# Analisis Base

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import friedmanchisquare
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

class DieboldMarianoTest:
    """
    Implementación del test de Diebold-Mariano para comparar pronósticos
    """
    @staticmethod
    def dm_test(errors1, errors2, h=1, crit="MSE", power=2):
        """
        Realiza el test de Diebold-Mariano
        
        Parameters:
        -----------
        errors1 : array-like
            Errores del primer modelo
        errors2 : array-like
            Errores del segundo modelo
        h : int
            Horizonte de predicción (para ajustar autocorrelación)
        crit : str
            Criterio de pérdida: "MSE", "MAE", "MAPE"
        power : int
            Potencia para la función de pérdida
            
        Returns:
        --------
        dm_stat : float
            Estadístico DM
        p_value : float
            P-valor (two-tailed)
        """
        errors1 = np.array(errors1)
        errors2 = np.array(errors2)
        
        # Calcular diferencias de pérdida
        if crit == "MSE":
            loss_diff = errors1**2 - errors2**2
        elif crit == "MAE":
            loss_diff = np.abs(errors1) - np.abs(errors2)
        elif crit == "MAPE":
            loss_diff = np.abs(errors1) - np.abs(errors2)
        else:
            loss_diff = errors1**power - errors2**power
        
        # Media de las diferencias
        mean_diff = np.mean(loss_diff)
        
        # Varianza de las diferencias (ajustada por autocorrelación)
        n = len(loss_diff)
        
        # Calcular varianza con corrección de Newey-West
        gamma0 = np.var(loss_diff, ddof=1)
        
        if h > 1:
            gamma_sum = 0
            for k in range(1, h):
                gamma_k = np.cov(loss_diff[:-k], loss_diff[k:])[0, 1]
                gamma_sum += (1 - k/h) * gamma_k
            variance = (gamma0 + 2 * gamma_sum) / n
        else:
            variance = gamma0 / n
        
        # Estadístico DM
        dm_stat = mean_diff / np.sqrt(variance) if variance > 0 else 0
        
        # P-valor (two-tailed)
        p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))
        
        return dm_stat, p_value


class ModelPerformanceAnalyzer:
    """
    Clase para análisis exhaustivo de rendimiento de modelos de predicción
    en diferentes escenarios de simulación.
    """
    
    def __init__(self):
        """
        Inicializa el analizador cargando los datos de los tres escenarios.
        """
        self.models = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
                      'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
        
        # Cargar datos con las rutas especificadas
        print("Cargando datos...")
        
        try:
            self.df_estacionario = pd.read_excel("./Datos/estacionario.xlsx")
            self.df_estacionario['Escenario'] = 'Estacionario_Lineal'
            print(f"✓ Estacionario: {len(self.df_estacionario)} filas")
            print(f"  Columnas: {self.df_estacionario.columns.tolist()}")
            
            self.df_no_estacionario = pd.read_excel("./Datos/no_estacionario.xlsx")
            self.df_no_estacionario['Escenario'] = 'No_Estacionario_Lineal'
            print(f"✓ No Estacionario: {len(self.df_no_estacionario)} filas")
            print(f"  Columnas: {self.df_no_estacionario.columns.tolist()}")
            
            self.df_no_lineal = pd.read_excel("./Datos/no_lineal.xlsx")
            self.df_no_lineal['Escenario'] = 'No_Lineal'
            print(f"✓ No Lineal: {len(self.df_no_lineal)} filas")
            print(f"  Columnas: {self.df_no_lineal.columns.tolist()}")
            
        except FileNotFoundError as e:
            print(f"ERROR: No se encontró el archivo - {e}")
            print("Verifica que los archivos estén en la carpeta './Datos/'")
            raise
        
        # Estandarizar nombres de columnas
        self._standardize_columns()
        
        # Combinar todos los datos
        self.df_all = pd.concat([self.df_estacionario, self.df_no_estacionario, 
                                 self.df_no_lineal], ignore_index=True)
        
        # Convertir tipos de datos críticos
        self._convert_data_types()
        
        print(f"\n✓ Datos combinados: {len(self.df_all)} observaciones totales")
        print(f"✓ Columnas finales: {self.df_all.columns.tolist()}")
        
    def _standardize_columns(self):
        """Estandariza nombres de columnas entre datasets"""
        # Para estacionario
        if 'Varianza error' in self.df_estacionario.columns:
            self.df_estacionario.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
        
        # Agregar columna 'Tipo de Modelo' si no existe en estacionario
        if 'Tipo de Modelo' not in self.df_estacionario.columns:
            # Crear tipo de modelo basado en valores AR y MA
            def create_model_type(row):
                ar_vals = row.get('Valores de AR', '')
                ma_vals = row.get('Valores MA', '')
                
                ar_str = str(ar_vals) if pd.notna(ar_vals) else ''
                ma_str = str(ma_vals) if pd.notna(ma_vals) else ''
                
                # Contar órdenes
                ar_order = len([x for x in ar_str.split(',') if x.strip() and x.strip() != '[]']) if ar_str else 0
                ma_order = len([x for x in ma_str.split(',') if x.strip() and x.strip() != '[]']) if ma_str else 0
                
                if ar_order > 0 and ma_order > 0:
                    return f'ARMA({ar_order},{ma_order})'
                elif ar_order > 0:
                    return f'AR({ar_order})'
                elif ma_order > 0:
                    return f'MA({ma_order})'
                else:
                    return 'Unknown'
            
            self.df_estacionario['Tipo de Modelo'] = self.df_estacionario.apply(create_model_type, axis=1)
        
        # Para no estacionario
        if 'Varianza error' in self.df_no_estacionario.columns:
            self.df_no_estacionario.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
        
        # Para no lineal
        if 'Varianza error' in self.df_no_lineal.columns:
            self.df_no_lineal.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
    
    def _convert_data_types(self):
        """Convierte tipos de datos para evitar errores de comparación"""
        # Convertir 'Paso' a numérico
        self.df_all['Paso'] = pd.to_numeric(self.df_all['Paso'], errors='coerce')
        
        # Convertir 'Varianza' a numérico
        self.df_all['Varianza'] = pd.to_numeric(self.df_all['Varianza'], errors='coerce')
        
        # Convertir columnas de modelos a numérico
        for model in self.models:
            self.df_all[model] = pd.to_numeric(self.df_all[model], errors='coerce')
        
        # Eliminar filas con valores NaN críticos
        critical_cols = ['Paso', 'Varianza'] + self.models
        self.df_all.dropna(subset=critical_cols, inplace=True)
        
        print(f"✓ Tipos de datos convertidos")
        print(f"✓ Filas después de limpieza: {len(self.df_all)}")
        
    def generate_full_report(self, output_dir='resultados_analisis'):
        """
        Genera reporte completo respondiendo a todas las preguntas clave.
        """
        import os
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        print("\n" + "="*80)
        print("INICIANDO ANÁLISIS COMPREHENSIVO DE MODELOS")
        print("="*80)
        
        # Crear archivo de reporte
        report_file = f"{output_dir}/reporte_completo.txt"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("REPORTE COMPLETO DE ANÁLISIS DE MODELOS DE PREDICCIÓN\n")
            f.write("="*80 + "\n\n")
        
        # 1. ANÁLISIS POR CARACTERÍSTICAS DEL DGP
        print("\n1. Analizando características del proceso generador...")
        self.analyze_dgp_characteristics(output_dir)
        
        # 2. ANÁLISIS POR DISTRIBUCIÓN DE ERRORES
        print("\n2. Analizando efecto de distribuciones...")
        self.analyze_distribution_effects(output_dir)
        
        # 3. ANÁLISIS POR HORIZONTE DE PREDICCIÓN
        print("\n3. Analizando horizonte de predicción...")
        self.analyze_horizon_effects(output_dir)
        
        # 4. ANÁLISIS DE INTERACCIONES COMPLEJAS
        print("\n4. Analizando interacciones complejas...")
        self.analyze_interactions(output_dir)
        
        # 5. ANÁLISIS DE ROBUSTEZ Y ESTABILIDAD
        print("\n5. Analizando robustez y estabilidad...")
        self.analyze_robustness(output_dir)
        
        # 6. ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA (DIEBOLD-MARIANO)
        print("\n6. Realizando tests de Diebold-Mariano...")
        self.analyze_statistical_significance_dm(output_dir)
        
        # 7. ANÁLISIS POR MODELO INDIVIDUAL
        print("\n7. Generando perfiles por modelo...")
        self.analyze_individual_models(output_dir)
        
        # 8. RECOMENDACIONES Y CONCLUSIONES
        print("\n8. Generando recomendaciones...")
        self.generate_recommendations(output_dir)
        
        print(f"\n{'='*80}")
        print(f"ANÁLISIS COMPLETO. Resultados guardados en: {output_dir}/")
        print(f"{'='*80}")
        
    def analyze_dgp_characteristics(self, output_dir):
        """
        1. ANÁLISIS DE CARACTERÍSTICAS DEL PROCESO GENERADOR
        """
        results = []
        
        # 1.1 Efecto de estacionaridad
        print("  - Analizando efecto de estacionaridad...")
        for model in self.models:
            est_mean = self.df_estacionario[model].mean()
            no_est_mean = self.df_no_estacionario[model].mean()
            diff = no_est_mean - est_mean
            pct_change = (diff / est_mean) * 100 if est_mean != 0 else 0
            
            results.append({
                'Modelo': model,
                'ECRPS_Estacionario': est_mean,
                'ECRPS_No_Estacionario': no_est_mean,
                'Diferencia': diff,
                'Cambio_%': pct_change
            })
        
        df_estacionaridad = pd.DataFrame(results)
        df_estacionaridad = df_estacionaridad.sort_values('Cambio_%')
        df_estacionaridad.to_csv(f'{output_dir}/1_efecto_estacionaridad.csv', index=False)
        
        # Visualización
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Gráfico de barras comparativas
        x = np.arange(len(self.models))
        width = 0.35
        axes[0].bar(x - width/2, df_estacionaridad['ECRPS_Estacionario'], 
                   width, label='Estacionario', alpha=0.8)
        axes[0].bar(x + width/2, df_estacionaridad['ECRPS_No_Estacionario'], 
                   width, label='No Estacionario', alpha=0.8)
        axes[0].set_xlabel('Modelo')
        axes[0].set_ylabel('ECRPS Promedio')
        axes[0].set_title('Rendimiento: Estacionario vs No Estacionario')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(df_estacionaridad['Modelo'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Gráfico de cambio porcentual
        colors = ['green' if x < 0 else 'red' for x in df_estacionaridad['Cambio_%']]
        axes[1].barh(df_estacionaridad['Modelo'], df_estacionaridad['Cambio_%'], color=colors, alpha=0.7)
        axes[1].set_xlabel('Cambio Porcentual (%)')
        axes[1].set_title('Impacto de No Estacionaridad\n(Negativo = Mejor en No Estacionario)')
        axes[1].axvline(x=0, color='black', linestyle='--', linewidth=0.8)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/1_estacionaridad.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 1.2 Efecto de no linealidad
        print("  - Analizando efecto de no linealidad...")
        results_nl = []
        for model in self.models:
            lin_mean = self.df_estacionario[model].mean()
            nl_mean = self.df_no_lineal[model].mean()
            diff = nl_mean - lin_mean
            pct_change = (diff / lin_mean) * 100 if lin_mean != 0 else 0
            
            results_nl.append({
                'Modelo': model,
                'ECRPS_Lineal': lin_mean,
                'ECRPS_No_Lineal': nl_mean,
                'Diferencia': diff,
                'Cambio_%': pct_change
            })
        
        df_linealidad = pd.DataFrame(results_nl)
        df_linealidad = df_linealidad.sort_values('Cambio_%')
        df_linealidad.to_csv(f'{output_dir}/1_efecto_no_linealidad.csv', index=False)
        
        # Visualización no linealidad
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        x = np.arange(len(self.models))
        axes[0].bar(x - width/2, df_linealidad['ECRPS_Lineal'], 
                   width, label='Lineal', alpha=0.8)
        axes[0].bar(x + width/2, df_linealidad['ECRPS_No_Lineal'], 
                   width, label='No Lineal', alpha=0.8)
        axes[0].set_xlabel('Modelo')
        axes[0].set_ylabel('ECRPS Promedio')
        axes[0].set_title('Rendimiento: Lineal vs No Lineal')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(df_linealidad['Modelo'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        colors = ['green' if x < 0 else 'red' for x in df_linealidad['Cambio_%']]
        axes[1].barh(df_linealidad['Modelo'], df_linealidad['Cambio_%'], color=colors, alpha=0.7)
        axes[1].set_xlabel('Cambio Porcentual (%)')
        axes[1].set_title('Impacto de No Linealidad\n(Negativo = Mejor en No Lineal)')
        axes[1].axvline(x=0, color='black', linestyle='--', linewidth=0.8)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/1_no_linealidad.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 1.3 Análisis por tipo de modelo
        print("  - Analizando efecto del tipo de modelo...")
        self.analyze_model_type_effect(output_dir)
        
    def analyze_model_type_effect(self, output_dir):
        """Analiza el efecto del tipo de modelo en el rendimiento"""
        
        # Análisis para datos estacionarios
        if 'Tipo de Modelo' in self.df_estacionario.columns:
            results_type = []
            for model in self.models:
                for model_type in self.df_estacionario['Tipo de Modelo'].unique():
                    subset = self.df_estacionario[self.df_estacionario['Tipo de Modelo'] == model_type]
                    if len(subset) > 0:
                        results_type.append({
                            'Modelo_Predictor': model,
                            'Tipo_Proceso': model_type,
                            'ECRPS_Mean': subset[model].mean(),
                            'ECRPS_Std': subset[model].std(),
                            'N_Obs': len(subset)
                        })
            
            df_type = pd.DataFrame(results_type)
            df_type.to_csv(f'{output_dir}/1_efecto_tipo_modelo.csv', index=False)
            
            # Crear heatmap para tipos más comunes
            common_types = df_type['Tipo_Proceso'].value_counts().head(10).index
            df_type_filtered = df_type[df_type['Tipo_Proceso'].isin(common_types)]
            
            if len(df_type_filtered) > 0:
                pivot = df_type_filtered.pivot_table(
                    index='Modelo_Predictor', 
                    columns='Tipo_Proceso', 
                    values='ECRPS_Mean'
                )
                
                fig, ax = plt.subplots(figsize=(14, 8))
                sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax, 
                           cbar_kws={'label': 'ECRPS'})
                ax.set_title('Rendimiento por Modelo Predictor y Tipo de Proceso', fontsize=14)
                ax.set_xlabel('Tipo de Proceso')
                ax.set_ylabel('Modelo Predictor')
                plt.tight_layout()
                plt.savefig(f'{output_dir}/1_heatmap_tipo_modelo.png', dpi=300, bbox_inches='tight')
                plt.close()
        
    def analyze_distribution_effects(self, output_dir):
        """
        2. ANÁLISIS DE EFECTOS DE DISTRIBUCIÓN
        """
        print("  - Analizando efectos de distribuciones...")
        
        results_dist = []
        for model in self.models:
            for dist in self.df_all['Distribución'].unique():
                if pd.notna(dist):
                    subset = self.df_all[self.df_all['Distribución'] == dist]
                    if len(subset) > 0:
                        results_dist.append({
                            'Modelo': model,
                            'Distribución': dist,
                            'ECRPS_Mean': subset[model].mean(),
                            'ECRPS_Std': subset[model].std(),
                            'ECRPS_Min': subset[model].min(),
                            'ECRPS_Max': subset[model].max()
                        })
        
        df_dist = pd.DataFrame(results_dist)
        df_dist.to_csv(f'{output_dir}/2_efecto_distribucion.csv', index=False)
        
        # Heatmap
        if len(df_dist) > 0:
            pivot = df_dist.pivot(index='Modelo', columns='Distribución', values='ECRPS_Mean')
            
            fig, ax = plt.subplots(figsize=(10, 8))
            sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax, cbar_kws={'label': 'ECRPS'})
            ax.set_title('Rendimiento por Modelo y Distribución', fontsize=14)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/2_heatmap_distribucion.png', dpi=300, bbox_inches='tight')
            plt.close()
        
        # Análisis por varianza
        print("  - Analizando efectos de varianza...")
        results_var = []
        varianzas_unicas = sorted([v for v in self.df_all['Varianza'].unique() if pd.notna(v)])
        
        for model in self.models:
            for var in varianzas_unicas:
                subset = self.df_all[self.df_all['Varianza'] == var]
                if len(subset) > 0:
                    results_var.append({
                        'Modelo': model,
                        'Varianza': var,
                        'ECRPS_Mean': subset[model].mean(),
                        'ECRPS_Std': subset[model].std()
                    })
        
        df_var = pd.DataFrame(results_var)
        df_var.to_csv(f'{output_dir}/2_efecto_varianza.csv', index=False)
        
        # Gráfico de líneas por varianza
        if len(df_var) > 0:
            fig, ax = plt.subplots(figsize=(12, 8))
            for model in self.models:
                data = df_var[df_var['Modelo'] == model].sort_values('Varianza')
                if len(data) > 0:
                    ax.plot(data['Varianza'], data['ECRPS_Mean'], marker='o', label=model, linewidth=2)
            
            ax.set_xlabel('Varianza', fontsize=12)
            ax.set_ylabel('ECRPS Promedio', fontsize=12)
            ax.set_title('Rendimiento según Nivel de Varianza', fontsize=14)
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/2_efecto_varianza.png', dpi=300, bbox_inches='tight')
            plt.close()
        
    def analyze_horizon_effects(self, output_dir):
        """
        3. ANÁLISIS DE HORIZONTE DE PREDICCIÓN
        """
        print("  - Analizando deterioro por horizonte...")
        
        results_horizon = []
        pasos_unicos = sorted([p for p in self.df_all['Paso'].unique() if pd.notna(p)])
        
        for model in self.models:
            for paso in pasos_unicos:
                subset = self.df_all[self.df_all['Paso'] == paso]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    std_val = subset[model].std()
                    cv_val = std_val / mean_val if mean_val != 0 and pd.notna(mean_val) else 0
                    
                    results_horizon.append({
                        'Modelo': model,
                        'Paso': int(paso),
                        'ECRPS_Mean': mean_val,
                        'ECRPS_Std': std_val,
                        'ECRPS_CV': cv_val
                    })
        
        df_horizon = pd.DataFrame(results_horizon)
        df_horizon.to_csv(f'{output_dir}/3_efecto_horizonte.csv', index=False)
        
        # Gráfico de deterioro
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # ECRPS promedio por paso
        for model in self.models:
            data = df_horizon[df_horizon['Modelo'] == model].sort_values('Paso')
            if len(data) > 0:
                axes[0].plot(data['Paso'], data['ECRPS_Mean'], marker='o', label=model, linewidth=2)
        
        axes[0].set_xlabel('Paso de Predicción', fontsize=12)
        axes[0].set_ylabel('ECRPS Promedio', fontsize=12)
        axes[0].set_title('Deterioro del Rendimiento por Horizonte', fontsize=14)
        axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0].grid(True, alpha=0.3)
        
        # Tasa de deterioro
        deterioro = []
        for model in self.models:
            data = df_horizon[df_horizon['Modelo'] == model].sort_values('Paso')
            if len(data) >= 2:
                paso_values = data['Paso'].tolist()
                ecrps_paso1 = data.iloc[0]['ECRPS_Mean']
                ecrps_paso_final = data.iloc[-1]['ECRPS_Mean']
                
                if pd.notna(ecrps_paso1) and pd.notna(ecrps_paso_final) and ecrps_paso1 != 0:
                    tasa = ((ecrps_paso_final - ecrps_paso1) / ecrps_paso1) * 100
                    deterioro.append({'Modelo': model, 'Deterioro_%': tasa})
        
        if deterioro:
            df_deterioro = pd.DataFrame(deterioro).sort_values('Deterioro_%')
            colors = ['green' if x < df_deterioro['Deterioro_%'].median() else 'red' 
                     for x in df_deterioro['Deterioro_%']]
            axes[1].barh(df_deterioro['Modelo'], df_deterioro['Deterioro_%'], color=colors, alpha=0.7)
            axes[1].set_xlabel(f'Deterioro Paso {pasos_unicos[0]}→{pasos_unicos[-1]} (%)', fontsize=12)
            axes[1].set_title('Tasa de Deterioro por Modelo', fontsize=14)
            axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/3_horizonte_prediccion.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de consistencia de ranking
        print("  - Analizando consistencia de ranking...")
        ranking_consistency = []
        for paso in pasos_unicos:
            subset = self.df_all[self.df_all['Paso'] == paso]
            if len(subset) > 0:
                ranks = subset[self.models].mean().rank()
                rank_dict = ranks.to_dict()
                rank_dict['Paso'] = int(paso)
                ranking_consistency.append(rank_dict)
        
        df_ranks = pd.DataFrame(ranking_consistency)
        df_ranks.to_csv(f'{output_dir}/3_ranking_por_paso.csv', index=False)
        
    def analyze_interactions(self, output_dir):
        """
        4. ANÁLISIS DE INTERACCIONES COMPLEJAS
        """
        print("  - Analizando interacciones Escenario × Distribución...")
        
        results_int = []
        for model in self.models:
            for escenario in self.df_all['Escenario'].unique():
                for dist in self.df_all['Distribución'].unique():
                    subset = self.df_all[(self.df_all['Escenario'] == escenario) & 
                                        (self.df_all['Distribución'] == dist)]
                    if len(subset) > 0:
                        results_int.append({
                            'Modelo': model,
                            'Escenario': escenario,
                            'Distribución': dist,
                            'ECRPS_Mean': subset[model].mean()
                        })
        
        df_int = pd.DataFrame(results_int)
        df_int.to_csv(f'{output_dir}/4_interacciones.csv', index=False)
        
        # Heatmap de interacciones para cada modelo
        for model in self.models[:3]:  # Solo primeros 3 por espacio
            model_data = df_int[df_int['Modelo'] == model]
            if len(model_data) > 0:
                pivot = model_data.pivot(
                    index='Escenario', columns='Distribución', values='ECRPS_Mean')
                
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax)
                ax.set_title(f'Interacción Escenario × Distribución: {model}', fontsize=12)
                plt.tight_layout()
                plt.savefig(f'{output_dir}/4_interaccion_{model.replace(" ", "_")}.png', 
                           dpi=300, bbox_inches='tight')
                plt.close()
        
        # Interacción triple: Escenario × Varianza × Paso
        print("  - Analizando interacción triple...")
        results_triple = []
        
        varianzas_unicas = sorted([v for v in self.df_all['Varianza'].unique() if pd.notna(v)])
        pasos_unicos = sorted([p for p in self.df_all['Paso'].unique() if pd.notna(p)])
        
        for model in self.models:
            for escenario in self.df_all['Escenario'].unique():
                for var in varianzas_unicas:
                    for paso in pasos_unicos:
                        subset = self.df_all[
                            (self.df_all['Escenario'] == escenario) & 
                            (self.df_all['Varianza'] == var) &
                            (self.df_all['Paso'] == paso)
                        ]
                        if len(subset) > 0:
                            results_triple.append({
                                'Modelo': model,
                                'Escenario': escenario,
                                'Varianza': var,
                                'Paso': int(paso),
                                'ECRPS_Mean': subset[model].mean()
                            })
        
        df_triple = pd.DataFrame(results_triple)
        df_triple.to_csv(f'{output_dir}/4_interaccion_triple.csv', index=False)
        
    def analyze_robustness(self, output_dir):
        """
        5. ANÁLISIS DE ROBUSTEZ Y ESTABILIDAD
        """
        print("  - Calculando métricas de robustez...")
        
        results_robust = []
        for model in self.models:
            ecrps_values = self.df_all[model]
            
            results_robust.append({
                'Modelo': model,
                'ECRPS_Mean': ecrps_values.mean(),
                'ECRPS_Std': ecrps_values.std(),
                'ECRPS_CV': ecrps_values.std() / ecrps_values.mean() if ecrps_values.mean() != 0 else 0,
                'ECRPS_Min': ecrps_values.min(),
                'ECRPS_Q25': ecrps_values.quantile(0.25),
                'ECRPS_Median': ecrps_values.median(),
                'ECRPS_Q75': ecrps_values.quantile(0.75),
                'ECRPS_Max': ecrps_values.max(),
                'ECRPS_IQR': ecrps_values.quantile(0.75) - ecrps_values.quantile(0.25)
            })
        
        df_robust = pd.DataFrame(results_robust)
        df_robust = df_robust.sort_values('ECRPS_CV')
        df_robust.to_csv(f'{output_dir}/5_robustez.csv', index=False)
        
        # Gráfico de robustez
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Coeficiente de variación
        axes[0, 0].barh(df_robust['Modelo'], df_robust['ECRPS_CV'], alpha=0.7)
        axes[0, 0].set_xlabel('Coeficiente de Variación')
        axes[0, 0].set_title('Estabilidad (Menor CV = Más Estable)')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Rango intercuartílico
        axes[0, 1].barh(df_robust['Modelo'], df_robust['ECRPS_IQR'], alpha=0.7, color='coral')
        axes[0, 1].set_xlabel('Rango Intercuartílico')
        axes[0, 1].set_title('Variabilidad (Menor IQR = Más Consistente)')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Boxplot comparativo
        data_box = [self.df_all[model] for model in self.models]
        bp = axes[1, 0].boxplot(data_box, labels=self.models, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('lightblue')
        axes[1, 0].set_ylabel('ECRPS')
        axes[1, 0].set_title('Distribución de ECRPS por Modelo')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].grid(True, alpha=0.3)
        
        # Scatter: Media vs Variabilidad
        axes[1, 1].scatter(df_robust['ECRPS_Mean'], df_robust['ECRPS_Std'], 
                          s=100, alpha=0.6, c=range(len(df_robust)), cmap='viridis')
        for idx, row in df_robust.iterrows():
            axes[1, 1].annotate(row['Modelo'], 
                               (row['ECRPS_Mean'], row['ECRPS_Std']),
                               fontsize=8, alpha=0.7)
        axes[1, 1].set_xlabel('ECRPS Promedio')
        axes[1, 1].set_ylabel('Desviación Estándar')
        axes[1, 1].set_title('Trade-off Rendimiento vs Estabilidad')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/5_robustez.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de peores casos
        print("  - Identificando peores casos...")
        worst_cases = []
        for model in self.models:
            df_temp = self.df_all.copy()
            df_temp['ECRPS'] = df_temp[model]
            worst = df_temp.nlargest(10, 'ECRPS')[
                ['Escenario', 'Tipo de Modelo', 'Distribución', 'Varianza', 'Paso', 'ECRPS']
            ]
            worst['Modelo_Predictor'] = model
            worst_cases.append(worst)
        
        df_worst = pd.concat(worst_cases, ignore_index=True)
        df_worst.to_csv(f'{output_dir}/5_peores_casos.csv', index=False)
        
    def analyze_statistical_significance_dm(self, output_dir):
        """
        6. ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA CON DIEBOLD-MARIANO
        """
        print("  - Realizando tests de Diebold-Mariano...")
        
        # Test de Friedman por escenario (para comparación general)
        results_friedman = []
        for escenario in self.df_all['Escenario'].unique():
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            data_matrix = subset[self.models].values
            
            try:
                statistic, p_value = friedmanchisquare(*[data_matrix[:, i] for i in range(len(self.models))])
                
                results_friedman.append({
                    'Escenario': escenario,
                    'Friedman_Statistic': statistic,
                    'P_Value': p_value,
                    'Significativo': 'Sí' if p_value < 0.05 else 'No'
                })
            except Exception as e:
                print(f"    Advertencia: Error en test de Friedman para {escenario}: {e}")
        
        if results_friedman:
            df_friedman = pd.DataFrame(results_friedman)
            df_friedman.to_csv(f'{output_dir}/6_test_friedman.csv', index=False)
        
        # Tests de Diebold-Mariano pareados
        print("  - Realizando tests pareados de Diebold-Mariano...")
        pairs = list(combinations(self.models, 2))
        dm_results = []
        
        for model1, model2 in pairs:
            # Calcular errores (usamos ECRPS directamente como métrica de pérdida)
            errors1 = self.df_all[model1].values
            errors2 = self.df_all[model2].values
            
            # Test de Diebold-Mariano
            dm_stat, p_value = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
            
            mean_diff = self.df_all[model1].mean() - self.df_all[model2].mean()
            
            # Determinar ganador
            if p_value < 0.05:
                if mean_diff < 0:
                    ganador = model1
                else:
                    ganador = model2
            else:
                ganador = 'Empate'
            
            dm_results.append({
                'Modelo_1': model1,
                'Modelo_2': model2,
                'Diferencia_Media': mean_diff,
                'DM_Statistic': dm_stat,
                'P_Value': p_value,
                'Significativo_0.05': 'Sí' if p_value < 0.05 else 'No',
                'Significativo_0.01': 'Sí' if p_value < 0.01 else 'No',
                'Ganador': ganador
            })
        
        df_dm = pd.DataFrame(dm_results)
        df_dm = df_dm.sort_values('P_Value')
        df_dm.to_csv(f'{output_dir}/6_tests_diebold_mariano.csv', index=False)
        
        # Matriz de p-valores (Diebold-Mariano)
        print("  - Creando matriz de p-valores...")
        p_matrix = np.ones((len(self.models), len(self.models)))
        for i, model1 in enumerate(self.models):
            for j, model2 in enumerate(self.models):
                if i != j:
                    errors1 = self.df_all[model1].values
                    errors2 = self.df_all[model2].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    p_matrix[i, j] = p_val
        
        fig, ax = plt.subplots(figsize=(12, 10))
        sns.heatmap(p_matrix, annot=True, fmt='.3f', cmap='RdYlGn', 
                   xticklabels=self.models, yticklabels=self.models, 
                   ax=ax, vmin=0, vmax=0.1, cbar_kws={'label': 'P-valor'})
        ax.set_title('Matriz de P-valores (Test de Diebold-Mariano)\nVerde = Diferencia Significativa', 
                    fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/6_matriz_pvalores_dm.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Dominancia estadística con Diebold-Mariano
        print("  - Analizando dominancia estadística...")
        dominance = []
        for model in self.models:
            wins = 0
            losses = 0
            ties = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05:
                        if mean_diff < 0:  # modelo es mejor (menor ECRPS)
                            wins += 1
                        else:
                            losses += 1
                    else:
                        ties += 1
            
            dominance.append({
                'Modelo': model,
                'Victorias_Significativas': wins,
                'Derrotas_Significativas': losses,
                'Empates': ties,
                'Score_Neto': wins - losses
            })
        
        df_dominance = pd.DataFrame(dominance)
        df_dominance = df_dominance.sort_values('Score_Neto', ascending=False)
        df_dominance.to_csv(f'{output_dir}/6_dominancia_estadistica_dm.csv', index=False)
        
        # Visualización de dominancia
        fig, ax = plt.subplots(figsize=(12, 6))
        x = np.arange(len(df_dominance))
        width = 0.25
        
        ax.bar(x - width, df_dominance['Victorias_Significativas'], 
               width, label='Victorias', color='green', alpha=0.7)
        ax.bar(x, df_dominance['Empates'], 
               width, label='Empates', color='gray', alpha=0.7)
        ax.bar(x + width, df_dominance['Derrotas_Significativas'], 
               width, label='Derrotas', color='red', alpha=0.7)
        
        ax.set_xlabel('Modelo')
        ax.set_ylabel('Número de Comparaciones')
        ax.set_title('Dominancia Estadística (Test Diebold-Mariano)')
        ax.set_xticks(x)
        ax.set_xticklabels(df_dominance['Modelo'], rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/6_dominancia_dm.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de Diebold-Mariano por escenario
        print("  - Analizando DM por escenario...")
        dm_by_scenario = []
        for escenario in self.df_all['Escenario'].unique():
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            
            for model1, model2 in combinations(self.models, 2):
                errors1 = subset[model1].values
                errors2 = subset[model2].values
                
                if len(errors1) > 0 and len(errors2) > 0:
                    dm_stat, p_value = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = subset[model1].mean() - subset[model2].mean()
                    
                    dm_by_scenario.append({
                        'Escenario': escenario,
                        'Modelo_1': model1,
                        'Modelo_2': model2,
                        'DM_Statistic': dm_stat,
                        'P_Value': p_value,
                        'Diferencia_Media': mean_diff,
                        'Significativo': 'Sí' if p_value < 0.05 else 'No'
                    })
        
        df_dm_scenario = pd.DataFrame(dm_by_scenario)
        df_dm_scenario.to_csv(f'{output_dir}/6_dm_por_escenario.csv', index=False)
    
    def analyze_individual_models(self, output_dir):
        """
        7. PERFILES INDIVIDUALES POR MODELO
        """
        print("  - Generando perfiles individuales...")
        
        for model in self.models:
            print(f"    > Analizando {model}...")
            
            # Crear subdirectorio para el modelo
            model_dir = f"{output_dir}/perfiles_modelos/{model.replace(' ', '_')}"
            import os
            os.makedirs(model_dir, exist_ok=True)
            
            # Reporte del modelo
            report = []
            report.append(f"="*80)
            report.append(f"PERFIL DETALLADO: {model}")
            report.append(f"="*80)
            report.append("")
            
            # Estadísticas generales
            report.append("1. ESTADÍSTICAS GENERALES")
            report.append("-" * 40)
            report.append(f"ECRPS Promedio Global: {self.df_all[model].mean():.6f}")
            report.append(f"Desviación Estándar: {self.df_all[model].std():.6f}")
            cv = self.df_all[model].std()/self.df_all[model].mean() if self.df_all[model].mean() != 0 else 0
            report.append(f"Coeficiente de Variación: {cv:.4f}")
            report.append(f"Mínimo: {self.df_all[model].min():.6f}")
            report.append(f"Mediana: {self.df_all[model].median():.6f}")
            report.append(f"Máximo: {self.df_all[model].max():.6f}")
            report.append("")
            
            # Ranking general
            mean_scores = self.df_all[self.models].mean()
            ranking = mean_scores.rank().astype(int)
            report.append(f"Ranking General: {ranking[model]}° de {len(self.models)}")
            report.append("")
            
            # Mejor escenario
            report.append("2. MEJORES ESCENARIOS")
            report.append("-" * 40)
            best_idx = self.df_all[model].idxmin()
            best_row = self.df_all.loc[best_idx]
            report.append(f"Mejor ECRPS: {best_row[model]:.6f}")
            report.append(f"  - Escenario: {best_row['Escenario']}")
            if 'Tipo de Modelo' in best_row:
                report.append(f"  - Tipo Modelo: {best_row['Tipo de Modelo']}")
            report.append(f"  - Distribución: {best_row['Distribución']}")
            report.append(f"  - Varianza: {best_row['Varianza']}")
            report.append(f"  - Paso: {best_row['Paso']}")
            report.append("")
            
            # Peor escenario
            report.append("3. PEORES ESCENARIOS")
            report.append("-" * 40)
            worst_idx = self.df_all[model].idxmax()
            worst_row = self.df_all.loc[worst_idx]
            report.append(f"Peor ECRPS: {worst_row[model]:.6f}")
            report.append(f"  - Escenario: {worst_row['Escenario']}")
            if 'Tipo de Modelo' in worst_row:
                report.append(f"  - Tipo Modelo: {worst_row['Tipo de Modelo']}")
            report.append(f"  - Distribución: {worst_row['Distribución']}")
            report.append(f"  - Varianza: {worst_row['Varianza']}")
            report.append(f"  - Paso: {worst_row['Paso']}")
            report.append("")
            
            # Rendimiento por escenario
            report.append("4. RENDIMIENTO POR ESCENARIO")
            report.append("-" * 40)
            for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
                subset = self.df_all[self.df_all['Escenario'] == escenario]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    report.append(f"{escenario}:")
                    report.append(f"  ECRPS: {mean_val:.6f} (Ranking: {int(rank)}°)")
            report.append("")
            
            # Fortalezas y debilidades
            report.append("5. FORTALEZAS Y DEBILIDADES")
            report.append("-" * 40)
            
            # Por distribución
            report.append("Por Distribución:")
            dist_performance = []
            for dist in self.df_all['Distribución'].unique():
                subset = self.df_all[self.df_all['Distribución'] == dist]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    dist_performance.append((dist, mean_val, rank))
            
            if dist_performance:
                dist_performance.sort(key=lambda x: x[2])
                report.append(f"  Mejor: {dist_performance[0][0]} (Ranking {int(dist_performance[0][2])}°)")
                report.append(f"  Peor: {dist_performance[-1][0]} (Ranking {int(dist_performance[-1][2])}°)")
            report.append("")
            
            # Por varianza
            report.append("Por Varianza:")
            var_performance = []
            for var in sorted(self.df_all['Varianza'].unique()):
                subset = self.df_all[self.df_all['Varianza'] == var]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    var_performance.append((var, mean_val, rank))
            
            if var_performance:
                var_performance.sort(key=lambda x: x[2])
                report.append(f"  Mejor: Varianza {var_performance[0][0]} (Ranking {int(var_performance[0][2])}°)")
                report.append(f"  Peor: Varianza {var_performance[-1][0]} (Ranking {int(var_performance[-1][2])}°)")
            report.append("")
            
            # Comparaciones con Diebold-Mariano
            report.append("6. COMPARACIONES ESTADÍSTICAS (DIEBOLD-MARIANO)")
            report.append("-" * 40)
            
            wins = 0
            losses = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05:
                        if mean_diff < 0:
                            wins += 1
                        else:
                            losses += 1
            
            report.append(f"Victorias significativas: {wins}")
            report.append(f"Derrotas significativas: {losses}")
            report.append(f"Score neto: {wins - losses}")
            report.append("")
            
            # Guardar reporte
            with open(f"{model_dir}/perfil_{model.replace(' ', '_')}.txt", 'w', encoding='utf-8') as f:
                f.write('\n'.join(report))
            
            # Visualizaciones del modelo
            self._create_model_visualizations(model, model_dir)
    
    def _create_model_visualizations(self, model, model_dir):
        """Crea visualizaciones específicas para un modelo"""
        
        # 1. Distribución de ECRPS
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Histograma
        axes[0, 0].hist(self.df_all[model], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
        axes[0, 0].axvline(self.df_all[model].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Media: {self.df_all[model].mean():.4f}')
        axes[0, 0].axvline(self.df_all[model].median(), color='green', linestyle='--', 
                          linewidth=2, label=f'Mediana: {self.df_all[model].median():.4f}')
        axes[0, 0].set_xlabel('ECRPS')
        axes[0, 0].set_ylabel('Frecuencia')
        axes[0, 0].set_title(f'Distribución de ECRPS - {model}')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Boxplot por escenario
        data_by_scenario = [self.df_all[self.df_all['Escenario'] == esc][model] 
                           for esc in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']]
        bp = axes[0, 1].boxplot(data_by_scenario, labels=['Est. Lin.', 'No Est. Lin.', 'No Lin.'], 
                               patch_artist=True)
        for patch, color in zip(bp['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
            patch.set_facecolor(color)
        axes[0, 1].set_ylabel('ECRPS')
        axes[0, 1].set_title(f'ECRPS por Escenario - {model}')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Rendimiento por paso
        paso_data = []
        for p in sorted(self.df_all['Paso'].unique()):
            subset = self.df_all[self.df_all['Paso'] == p]
            if len(subset) > 0:
                paso_data.append((p, subset[model].mean()))
        
        if paso_data:
            pasos, means = zip(*paso_data)
            axes[1, 0].plot(pasos, means, marker='o', linewidth=2, markersize=8, color='darkblue')
            axes[1, 0].set_xlabel('Paso de Predicción')
            axes[1, 0].set_ylabel('ECRPS Promedio')
            axes[1, 0].set_title(f'Rendimiento por Horizonte - {model}')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Heatmap: Distribución × Varianza
        pivot_data = []
        dist_labels = []
        var_labels = sorted(self.df_all['Varianza'].unique())
        
        for dist in self.df_all['Distribución'].unique():
            row = []
            for var in var_labels:
                subset = self.df_all[(self.df_all['Distribución'] == dist) & 
                                    (self.df_all['Varianza'] == var)]
                if len(subset) > 0:
                    row.append(subset[model].mean())
                else:
                    row.append(np.nan)
            if not all(np.isnan(row)):
                pivot_data.append(row)
                dist_labels.append(dist)
        
        if pivot_data:
            pivot_df = pd.DataFrame(pivot_data, index=dist_labels, columns=var_labels)
            
            sns.heatmap(pivot_df, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 1],
                       cbar_kws={'label': 'ECRPS'})
            axes[1, 1].set_title(f'ECRPS: Distribución × Varianza - {model}')
            axes[1, 1].set_xlabel('Varianza')
            axes[1, 1].set_ylabel('Distribución')
        
        plt.tight_layout()
        plt.savefig(f'{model_dir}/visualizaciones_{model.replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2. Comparación con otros modelos
        fig, ax = plt.subplots(figsize=(12, 8))
        
        means = self.df_all[self.models].mean().sort_values()
        colors = ['red' if m == model else 'steelblue' for m in means.index]
        bars = ax.barh(means.index, means.values, color=colors, alpha=0.7)
        
        # Destacar el modelo actual
        for i, bar in enumerate(bars):
            if means.index[i] == model:
                bar.set_edgecolor('black')
                bar.set_linewidth(3)
        
        ax.set_xlabel('ECRPS Promedio')
        ax.set_title(f'Comparación Global - {model} (Destacado en Rojo)')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{model_dir}/comparacion_{model.replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
    
    def generate_recommendations(self, output_dir):
        """
        8. GENERACIÓN DE RECOMENDACIONES
        """
        print("  - Generando recomendaciones estratégicas...")
        
        recommendations = []
        recommendations.append("="*80)
        recommendations.append("RECOMENDACIONES Y CONCLUSIONES")
        recommendations.append("="*80)
        recommendations.append("")
        
        # 1. Modelo campeón general
        overall_best = self.df_all[self.models].mean().idxmin()
        overall_worst = self.df_all[self.models].mean().idxmax()
        
        recommendations.append("1. MODELO CAMPEÓN GENERAL")
        recommendations.append("-" * 40)
        recommendations.append(f"Mejor rendimiento promedio: {overall_best}")
        recommendations.append(f"ECRPS: {self.df_all[overall_best].mean():.6f}")
        recommendations.append(f"Desviación Estándar: {self.df_all[overall_best].std():.6f}")
        recommendations.append("")
        recommendations.append(f"Peor rendimiento promedio: {overall_worst}")
        recommendations.append(f"ECRPS: {self.df_all[overall_worst].mean():.6f}")
        recommendations.append("")
        
        # 2. Modelos por escenario
        recommendations.append("2. RECOMENDACIONES POR ESCENARIO")
        recommendations.append("-" * 40)
        
        for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\n{escenario}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 3. Modelos por distribución
        recommendations.append("3. RECOMENDACIONES POR DISTRIBUCIÓN DE ERRORES")
        recommendations.append("-" * 40)
        
        for dist in self.df_all['Distribución'].unique():
            subset = self.df_all[self.df_all['Distribución'] == dist]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\nDistribución {dist}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 4. Modelos más robustos
        recommendations.append("4. MODELOS MÁS ROBUSTOS (MENOR VARIABILIDAD)")
        recommendations.append("-" * 40)
        
        cv_scores = {model: self.df_all[model].std() / self.df_all[model].mean() 
                    for model in self.models if self.df_all[model].mean() != 0}
        cv_sorted = sorted(cv_scores.items(), key=lambda x: x[1])
        
        for i, (model, cv) in enumerate(cv_sorted[:3], 1):
            recommendations.append(f"{i}. {model}: CV = {cv:.4f}")
        
        recommendations.append("")
        
        # 5. Modelos por horizonte
        recommendations.append("5. RECOMENDACIONES POR HORIZONTE DE PREDICCIÓN")
        recommendations.append("-" * 40)
        
        pasos_unicos = sorted(self.df_all['Paso'].unique())
        for paso in [pasos_unicos[0], pasos_unicos[len(pasos_unicos)//2], pasos_unicos[-1]]:
            subset = self.df_all[self.df_all['Paso'] == paso]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\nPaso {paso}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 6. Estrategia de ensamble
        recommendations.append("6. ESTRATEGIA DE ENSAMBLE SUGERIDA")
        recommendations.append("-" * 40)
        
        # Top 3 modelos complementarios
        top3 = self.df_all[self.models].mean().nsmallest(3)
        recommendations.append("Combinar los siguientes modelos:")
        for i, (model, score) in enumerate(top3.items(), 1):
            recommendations.append(f"{i}. {model} (ECRPS: {score:.6f})")
        
        recommendations.append("")
        recommendations.append("Justificación:")
        recommendations.append("  - Estos modelos muestran el mejor rendimiento promedio")
        recommendations.append("  - Un ensamble puede capturar fortalezas complementarias")
        recommendations.append("  - Reduce el riesgo de seleccionar un modelo subóptimo")
        
        recommendations.append("")
        
        # 7. Modelos con dominancia estadística
        recommendations.append("7. MODELOS CON DOMINANCIA ESTADÍSTICA")
        recommendations.append("-" * 40)
        
        dominance_scores = []
        for model in self.models:
            wins = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05 and mean_diff < 0:
                        wins += 1
            
            dominance_scores.append((model, wins))
        
        dominance_scores.sort(key=lambda x: x[1], reverse=True)
        
        recommendations.append("Modelos estadísticamente superiores (test Diebold-Mariano):")
        for i, (model, wins) in enumerate(dominance_scores[:5], 1):
            recommendations.append(f"{i}. {model}: {wins} victorias significativas")
        
        recommendations.append("")
        
        # 8. Reglas de decisión
        recommendations.append("8. REGLAS DE DECISIÓN SUGERIDAS")
        recommendations.append("-" * 40)
        recommendations.append("")
        
        # Reglas por escenario
        for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            if len(subset) > 0:
                top2 = subset[self.models].mean().nsmallest(2)
                
                if escenario == 'Estacionario_Lineal':
                    recommendations.append("SI el proceso es ESTACIONARIO y LINEAL:")
                elif escenario == 'No_Estacionario_Lineal':
                    recommendations.append("SI el proceso es NO ESTACIONARIO y LINEAL:")
                else:
                    recommendations.append("SI el proceso es NO LINEAL:")
                
                recommendations.append(f"  → Primera opción: {top2.index[0]}")
                recommendations.append(f"  → Segunda opción: {top2.index[1]}")
                recommendations.append("")
        
        # Reglas por distribución
        recommendations.append("SI la distribución de errores:")
        for dist in self.df_all['Distribución'].unique():
            subset = self.df_all[self.df_all['Distribución'] == dist]
            if len(subset) > 0:
                best = subset[self.models].mean().idxmin()
                recommendations.append(f"  • Es {dist} → Usar {best}")
        
        recommendations.append("")
        
        # Reglas por varianza
        recommendations.append("SI el nivel de varianza:")
        variances = sorted(self.df_all['Varianza'].unique())
        if len(variances) >= 2:
            low_var = variances[0]
            high_var = variances[-1]
            
            subset_low = self.df_all[self.df_all['Varianza'] == low_var]
            subset_high = self.df_all[self.df_all['Varianza'] == high_var]
            
            best_low = subset_low[self.models].mean().idxmin()
            best_high = subset_high[self.models].mean().idxmin()
            
            recommendations.append(f"  • Es bajo ({low_var}) → Usar {best_low}")
            recommendations.append(f"  • Es alto ({high_var}) → Usar {best_high}")
        
        recommendations.append("")
        
        # 9. Conclusiones finales
        recommendations.append("9. CONCLUSIONES PRINCIPALES")
        recommendations.append("-" * 40)
        recommendations.append("")
        recommendations.append(f"• El modelo {overall_best} muestra el mejor rendimiento general")
        recommendations.append(f"  con ECRPS promedio de {self.df_all[overall_best].mean():.6f}")
        recommendations.append("")
        
        # Análisis de robustez
        most_robust = min(cv_scores.items(), key=lambda x: x[1])[0]
        recommendations.append(f"• El modelo más robusto (menor CV) es {most_robust}")
        recommendations.append("")
        
        # Comparación estacionario vs no estacionario
        est_best = self.df_estacionario[self.models].mean().idxmin()
        no_est_best = self.df_no_estacionario[self.models].mean().idxmin()
        
        if est_best == no_est_best:
            recommendations.append(f"• {est_best} es consistentemente superior en procesos")
            recommendations.append("  estacionarios y no estacionarios")
        else:
            recommendations.append(f"• Para procesos estacionarios: preferir {est_best}")
            recommendations.append(f"• Para procesos no estacionarios: preferir {no_est_best}")
        recommendations.append("")
        
        # Análisis de no linealidad
        nl_best = self.df_no_lineal[self.models].mean().idxmin()
        recommendations.append(f"• Para procesos no lineales: {nl_best} es la mejor opción")
        recommendations.append("")
        
        # Recomendación de ensamble
        recommendations.append("• Se recomienda implementar un ENSAMBLE de los top 3 modelos")
        recommendations.append("  para maximizar robustez y rendimiento")
        recommendations.append("")
        
        # Consideraciones prácticas
        recommendations.append("10. CONSIDERACIONES PRÁCTICAS")
        recommendations.append("-" * 40)
        recommendations.append("")
        recommendations.append("Factores a considerar en la selección:")
        recommendations.append("  1. Costo computacional vs ganancia en precisión")
        recommendations.append("  2. Robustez ante cambios en la distribución de errores")
        recommendations.append("  3. Consistencia a través de horizontes de predicción")
        recommendations.append("  4. Facilidad de interpretación y explicabilidad")
        recommendations.append("  5. Disponibilidad de recursos para implementación")
        recommendations.append("")
        
        # Trade-offs identificados
        recommendations.append("Trade-offs identificados:")
        
        # Mejor vs más robusto
        if overall_best != most_robust:
            recommendations.append(f"  • Rendimiento vs Robustez: {overall_best} (mejor) vs {most_robust} (más robusto)")
        
        # Modelos especializados
        recommendations.append("  • Algunos modelos son especialistas en escenarios específicos")
        recommendations.append("  • Otros modelos son generalistas con buen rendimiento global")
        recommendations.append("")
        
        # Guardar recomendaciones
        with open(f'{output_dir}/8_recomendaciones.txt', 'w', encoding='utf-8') as f:
            f.write('\n'.join(recommendations))
        
        print('\n'.join(recommendations))


# ============================================================================
# CÓDIGO DE EJECUCIÓN PRINCIPAL
# ============================================================================

def main():
    """
    Función principal para ejecutar el análisis completo
    """
    print("\n" + "="*80)
    print("ANÁLISIS COMPREHENSIVO DE MODELOS DE PREDICCIÓN PROBABILÍSTICA")
    print("="*80 + "\n")
    
    # Crear analizador
    try:
        analyzer = ModelPerformanceAnalyzer()
    except FileNotFoundError:
        print("\nERROR: No se encontraron los archivos de datos")
        print("Verifica que existan los siguientes archivos:")
        print("  - ./Datos/estacionario.xlsx")
        print("  - ./Datos/no_estacionario.xlsx")
        print("  - ./Datos/no_lineal.xlsx")
        return
    except Exception as e:
        print(f"\nERROR al cargar datos: {e}")
        import traceback
        traceback.print_exc()
        return
    
    # Ejecutar análisis completo
    output_directory = 'resultados_analisis_completo'
    
    try:
        analyzer.generate_full_report(output_dir=output_directory)
        
        print(f"\n{'='*80}")
        print(f"✓ Análisis completado exitosamente")
        print(f"✓ Todos los resultados guardados en: {output_directory}/")
        print(f"{'='*80}\n")
        
        print("Archivos generados:")
        print("  📊 Análisis de características del DGP")
        print("  📈 Efectos de distribución y varianza")
        print("  🎯 Análisis de horizonte de predicción")
        print("  🔄 Interacciones complejas")
        print("  💪 Métricas de robustez")
        print("  📉 Tests de Diebold-Mariano")
        print("  👤 Perfiles individuales por modelo")
        print("  💡 Recomendaciones estratégicas")
        print("")
        
    except Exception as e:
        print(f"\n❌ ERROR durante el análisis: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


ANÁLISIS COMPREHENSIVO DE MODELOS DE PREDICCIÓN PROBABILÍSTICA

Cargando datos...
✓ Estacionario: 1320 filas
  Columnas: ['Paso', 'Valores de AR', 'Valores MA', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ No Estacionario: 840 filas
  Columnas: ['Paso', 'Tipo de Modelo', 'Valores de AR', 'Valores MA', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ No Lineal: 840 filas
  Columnas: ['Paso', 'Tipo de Modelo', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ Tipos de datos convertidos
✓ Filas después de limpieza: 2600

✓ Datos combinados: 2600 observaciones totales
✓ Columnas finales: ['Paso', 'Valores de AR'

# Pre analisis

In [12]:
import pandas as pd
import re
    
estacionario = pd.read_excel("./Datos/estacionario.xlsx")

estacionario = estacionario.drop_duplicates()
estacionario = estacionario[estacionario["Paso"] != "Promedio"]

def determinar_tipo_modelo_mejorado(row):
    """
    Determina el tipo de modelo (AR, MA, ARMA) y su orden a partir de los valores
    en las columnas 'Valores de AR' y 'Valores MA'.
    """
    ar_str = str(row['Valores de AR'])
    ma_str = str(row['Valores MA'])
    
    # Expresión regular para encontrar números (enteros o decimales, positivos o negativos)
    regex_numeros = r'-?\d+\.?\d*'
    
    # Cuenta cuántos números válidos hay en cada string
    p = len(re.findall(regex_numeros, ar_str))
    q = len(re.findall(regex_numeros, ma_str))
    
    if p > 0 and q == 0:
        return f"AR({p})"
    elif p == 0 and q > 0:
        return f"MA({q})"
    elif p > 0 and q > 0:
        return f"ARMA({p},{q})"
    else:
        return None # O "Ruido Blanco" si p=0 y q=0

# Aplica la función mejorada para crear la columna "Tipo de Modelo"
estacionario['Tipo de Modelo'] = estacionario.apply(determinar_tipo_modelo_mejorado, axis=1)

# Imprime los valores únicos de la columna Tipo de modelo para verificar
print("Valores únicos encontrados en 'Tipo de Modelo':")
print(estacionario['Tipo de Modelo'].unique())

# Ordena las columnas 'Paso' y 'Tipo de modelo' al inicio
cols = estacionario.columns.tolist()
# Aseguramos que las columnas existan antes de moverlas
if 'Paso' in cols:
    cols.insert(0, cols.pop(cols.index('Paso')))
if 'Tipo de Modelo' in cols:
    cols.insert(1, cols.pop(cols.index('Tipo de Modelo')))

estacionario = estacionario.reindex(columns=cols)


# Borra las columnas originales 'Valores de AR' y 'Valores MA'
estacionario = estacionario.drop(columns=['Valores de AR', 'Valores MA'])
estacionario["Escenario"] = "Estacionario_Lineal"

# Muestra el DataFrame resultante
estacionario

Valores únicos encontrados en 'Tipo de Modelo':
['AR(1)' 'AR(2)' 'MA(1)' 'MA(2)' 'ARMA(1,1)' 'ARMA(2,2)']


,Paso,Tipo de Modelo,Distribución,Varianza error,AREPD,AV-MCPS,Block Bootstrapping,DeepAR,EnCQR-LSTM,LSPM,LSPMW,MCPS,Sieve Bootstrap,Mejor Modelo,Escenario
0,1,AR(1),normal,0.2,0.294667,0.355344,0.248447,0.263419,0.306622,0.440706,0.431452,0.285427,0.248691,Block Bootstrapping,Estacionario_Lineal
2,2,AR(1),normal,0.2,0.604540,0.307449,0.254264,0.273001,0.565522,0.470424,0.474111,0.285430,0.254193,Sieve Bootstrap,Estacionario_Lineal
4,3,AR(1),normal,0.2,0.273622,0.276230,0.258388,0.315765,0.269452,0.520070,0.517876,0.337990,0.258039,Sieve Bootstrap,Estacionario_Lineal
6,4,AR(1),normal,0.2,0.261423,0.279697,0.254453,0.289443,0.269285,0.287989,0.288111,0.282999,0.254655,Block Bootstrapping,Estacionario_Lineal
8,5,AR(1),normal,0.2,0.626252,0.273680,0.254842,0.272827,0.639437,0.763960,0.753066,0.308347,0.254952,Block Bootstrapping,Estacionario_Lineal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,1,"ARMA(2,2)",mixture,3.0,1.082513,0.999066,0.953857,1.116455,1.053269,2.030504,2.165650,0.990087,0.954156,Block Bootstrapping,Estacionario_Lineal
1311,2,"ARMA(2,2)",mixture,3.0,1.903173,0.971148,0.954440,1.005615,1.518301,1.431610,1.522051,1.141614,0.954065,Sieve Bootstrap,Estacionario_Lineal
1313,3,"ARMA(2,2)",mixture,3.0,2.310542,1.021845,0.976235,1.002865,1.615073,1.026140,1.036051,1.484601,0.962417,Sieve Bootstrap,Estacionario_Lineal
1315,4,"ARMA(2,2)",mixture,3.0,1.324103,0.968827,0.961514,0.977739,1.072897,1.453428,1.530595,1.125230,0.960919,Sieve Bootstrap,Estacionario_Lineal


In [20]:
no_estacionario = pd.read_excel("./Datos/no_estacionario.xlsx")
no_estacionario.drop(columns=['Valores de AR', 'Valores MA'], inplace=True)
no_estacionario["Escenario"] = "No_Estacionario_Lineal"
no_estacionario = no_estacionario[no_estacionario["Paso"] != "Promedio"]
no_estacionario

,Paso,Tipo de Modelo,Distribución,Varianza error,AREPD,AV-MCPS,Block Bootstrapping,DeepAR,EnCQR-LSTM,LSPM,LSPMW,MCPS,Sieve Bootstrap,Mejor Modelo,Escenario
0,1,"ARIMA(0,1,0)",normal,0.2,1.860823,0.258474,0.253635,0.319481,0.488711,0.367279,0.360494,0.270816,0.273828,Block Bootstrapping,No_Estacionario_Lineal
1,2,"ARIMA(0,1,0)",normal,0.2,1.244128,0.528968,0.275061,0.438099,0.322919,0.426187,0.430296,0.576792,0.272952,Sieve Bootstrap,No_Estacionario_Lineal
2,3,"ARIMA(0,1,0)",normal,0.2,1.799818,0.864295,0.272406,0.291500,0.396481,0.642530,0.639134,0.269655,0.275661,MCPS,No_Estacionario_Lineal
3,4,"ARIMA(0,1,0)",normal,0.2,1.912421,0.481159,0.255186,0.291577,0.495882,0.341570,0.341227,0.533788,0.275948,Block Bootstrapping,No_Estacionario_Lineal
4,5,"ARIMA(0,1,0)",normal,0.2,2.822771,0.792130,0.257461,0.658698,1.291283,0.981902,0.969842,1.455485,0.338116,Block Bootstrapping,No_Estacionario_Lineal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1,"ARIMA(2,1,2)",mixture,3.0,76.766114,5.668568,0.965836,7.254422,13.176312,4.421885,4.173484,20.231134,2.612414,Block Bootstrapping,No_Estacionario_Lineal
835,2,"ARIMA(2,1,2)",mixture,3.0,80.630681,6.161741,0.974398,8.767931,9.287902,1.733689,1.596168,21.251698,1.956761,Block Bootstrapping,No_Estacionario_Lineal
836,3,"ARIMA(2,1,2)",mixture,3.0,86.539087,10.452450,0.982561,24.631292,18.639842,6.195609,5.953120,23.480752,3.623684,Block Bootstrapping,No_Estacionario_Lineal
837,4,"ARIMA(2,1,2)",mixture,3.0,93.057798,13.911382,0.958507,27.567728,17.852720,7.288522,6.952830,28.286507,4.681807,Block Bootstrapping,No_Estacionario_Lineal


In [21]:
no_lineal = pd.read_excel("./Datos/no_lineal.xlsx")
no_lineal = no_lineal[no_lineal["Paso"] != "Promedio"]
no_lineal["Escenario"] = "No_Lineal_Estacionario"
no_lineal

,Paso,Tipo de Modelo,Distribución,Varianza error,AREPD,AV-MCPS,Block Bootstrapping,DeepAR,EnCQR-LSTM,LSPM,LSPMW,MCPS,Sieve Bootstrap,Mejor Modelo,Escenario
0,1,"SETAR(2,1)",normal,0.2,0.257043,0.253521,0.251524,0.263274,0.257984,0.285655,0.282110,0.257015,0.251188,Sieve Bootstrap,No_Lineal_Estacionario
1,2,"SETAR(2,1)",normal,0.2,0.305723,0.383340,0.288529,0.297164,0.324101,0.316846,0.319675,0.347319,0.290022,Block Bootstrapping,No_Lineal_Estacionario
2,3,"SETAR(2,1)",normal,0.2,0.292055,0.258555,0.287265,0.275374,0.278881,0.320347,0.320181,0.270736,0.262183,AV-MCPS,No_Lineal_Estacionario
3,4,"SETAR(2,1)",normal,0.2,0.298469,0.269290,0.263802,0.255605,0.270449,0.290893,0.290581,0.329900,0.258734,DeepAR,No_Lineal_Estacionario
4,5,"SETAR(2,1)",normal,0.2,0.298007,0.368342,0.501202,0.323900,0.348571,0.326254,0.329508,0.423889,0.442319,AREPD,No_Lineal_Estacionario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1,"SETAR(2,3)",mixture,3.0,1.164445,0.992519,0.962026,0.989297,1.046459,0.971555,1.076860,1.003262,0.961513,Sieve Bootstrap,No_Lineal_Estacionario
835,2,"SETAR(2,3)",mixture,3.0,1.191648,1.034591,0.986347,1.077081,0.972263,0.957417,0.986525,0.963721,0.984072,LSPM,No_Lineal_Estacionario
836,3,"SETAR(2,3)",mixture,3.0,1.193252,1.387456,1.012627,0.981861,0.955903,0.987603,0.977201,1.041540,1.009812,EnCQR-LSTM,No_Lineal_Estacionario
837,4,"SETAR(2,3)",mixture,3.0,1.229893,1.182221,1.124342,0.983326,0.960088,1.036372,0.978720,1.029875,1.103310,EnCQR-LSTM,No_Lineal_Estacionario


In [22]:
# Une los tres DataFrames en uno solo uno debajo de otro
df_all = pd.concat([estacionario, no_estacionario, no_lineal], ignore_index=True)
# Guarda el DataFrame combinado en un archivo Excel
df_all.to_excel("./Datos/datos_combinados.xlsx", index=False)

# Analisis con la correcion del profe

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from scipy import stats
from itertools import combinations

# ============================================================================
# CONFIGURACIÓN
# ============================================================================
RUTA_DATOS = "./Datos/datos_combinados.xlsx"
CARPETA_RESULTADOS = "resultados_completos_media_mediana"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
ESCENARIOS_ESTACIONARIOS = ['Estacionario_Lineal', 'No_Lineal_Estacionario']
ESCENARIOS_NO_ESTACIONARIOS = ['No_Estacionario_Lineal']
ESCENARIOS_LINEALES = ['Estacionario_Lineal', 'No_Estacionario_Lineal']
ESCENARIOS_NO_LINEALES = ['No_Lineal_Estacionario']

# ============================================================================
# CLASE PARA TEST ESTADÍSTICO
# ============================================================================
class DieboldMarianoTest:
    @staticmethod
    def dm_test(errors1, errors2, h=1, power=2):
        # Implementación del test... (sin cambios)
        errors1, errors2 = np.array(errors1), np.array(errors2)
        loss_diff = (errors1**power) - (errors2**power)
        mean_diff = np.mean(loss_diff)
        n = len(loss_diff)
        gamma0 = np.var(loss_diff, ddof=1)
        if h > 1:
            gamma_sum = sum((1 - k/h) * np.cov(loss_diff[:-k], loss_diff[k:])[0, 1] for k in range(1, h))
            variance = (gamma0 + 2 * gamma_sum) / n
        else:
            variance = gamma0 / n
        dm_stat = mean_diff / np.sqrt(variance) if variance > 0 else 0
        p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))
        return dm_stat, p_value

# ============================================================================
# FUNCIONES DE ANÁLISIS Y VISUALIZACIÓN (MODIFICADAS)
# ============================================================================
def crear_directorio_resultados(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)
        print(f"Directorio '{nombre_carpeta}' creado.")

def guardar_grafico(nombre_archivo):
    ruta_completa = os.path.join(CARPETA_RESULTADOS, nombre_archivo)
    plt.savefig(ruta_completa, dpi=300, bbox_inches='tight')
    plt.close()

def graficar_comparacion_barras(promedios1, promedios2, orden, etiqueta1, etiqueta2, agg_method, nombre_archivo):
    """Grafica la comparación de barras para media o mediana."""
    fig, ax = plt.subplots(figsize=(14, 8))
    x = np.arange(len(orden))
    width = 0.35
    bars1 = ax.bar(x - width/2, promedios1[orden], width, label=etiqueta1, alpha=0.8, color='#3498db')
    bars2 = ax.bar(x + width/2, promedios2[orden], width, label=etiqueta2, alpha=0.8, color='#e74c3c')
    
    ylabel = f'ECRPS {"Promedio" if agg_method == "mean" else "Mediano"} (menor es mejor)'
    titulo = f'Comparación de Desempeño ({agg_method.capitalize()})'
    
    ax.set_xlabel('Modelos', fontsize=12, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(orden, rotation=45, ha='right')
    ax.legend(fontsize=11)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2., height, f'{height:.3f}', ha='center', va='bottom', fontsize=8)
    plt.tight_layout()
    guardar_grafico(nombre_archivo)

def generar_heatmap(data, agg_method, titulo_sufijo, nombre_archivo, figsize=(14, 8)):
    """Genera un heatmap basado en media o mediana."""
    fig, ax = plt.subplots(figsize=figsize)
    cbar_label = f'ECRPS {"Promedio" if agg_method == "mean" else "Mediano"}'
    titulo = f'Heatmap: {titulo_sufijo} ({agg_method.capitalize()})'
    
    sns.heatmap(data, annot=True, fmt='.3f', cmap='RdYlGn_r',
                cbar_kws={'label': cbar_label},
                linewidths=0.5, linecolor='gray', ax=ax)
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Modelos', fontsize=12, fontweight='bold')
    ax.set_ylabel(data.index.name, fontsize=12, fontweight='bold')
    plt.tight_layout()
    guardar_grafico(nombre_archivo)

def graficar_evolucion_metrica_por_tipo(df, metrica_eje_x, agg_method, xlabel, nombre_archivo_sufijo):
    """Genera gráficos de evolución para cada tipo de modelo, usando media o mediana."""
    valores_unicos = sorted(df[metrica_eje_x].unique())
    tipos_modelo_unicos = df['Tipo de Modelo'].unique()
    
    for tipo in tipos_modelo_unicos:
        df_tipo = df[df['Tipo de Modelo'] == tipo]
        fig, ax = plt.subplots(figsize=(12, 7))
        
        for modelo in MODELOS:
            agregados = [df_tipo[df_tipo[metrica_eje_x] == val][modelo].agg(agg_method) for val in valores_unicos]
            if not all(np.isnan(agregados)):
                ax.plot(valores_unicos, agregados, marker='o', linewidth=2, markersize=8, label=modelo, alpha=0.8)
        
        ylabel = f'ECRPS {"Promedio" if agg_method == "mean" else "Mediano"}'
        titulo = f'ECRPS vs {metrica_eje_x} ({agg_method.capitalize()}) - Tipo: {tipo}'
        
        ax.set_xlabel(xlabel, fontsize=12, fontweight='bold')
        ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
        ax.set_title(titulo, fontsize=13, fontweight='bold', pad=15)
        ax.legend(fontsize=9, loc='best', ncol=2)
        ax.grid(True, alpha=0.3, linestyle='--')
        if metrica_eje_x == 'Paso':
            ax.set_xticks(valores_unicos)
            
        plt.tight_layout()
        nombre_archivo_tipo = f'ecrps_vs_{nombre_archivo_sufijo}_tipo_{tipo.replace(" ", "_").lower()}_{agg_method}.png'
        guardar_grafico(nombre_archivo_tipo)

def analizar_robustez_estabilidad(df, agg_method):
    """Calcula y grafica métricas de robustez y estabilidad."""
    print(f" -> Analizando robustez y estabilidad (basado en {agg_method})...")
    
    if agg_method == 'mean':
        # Análisis basado en la media (como antes)
        metrics = [{'Modelo': m, 'Centralidad': df[m].mean(), 'Dispersion': df[m].std()} for m in MODELOS]
        df_robust = pd.DataFrame(metrics)
        label_centralidad = 'ECRPS Promedio (Rendimiento)'
        label_dispersion = 'Desviación Estándar (Estabilidad)'
        titulo_compromiso = 'Compromiso Rendimiento vs. Estabilidad (Media vs Std)'
        
    else: # agg_method == 'median'
        # Análisis basado en la mediana (más robusto a outliers)
        metrics = [{'Modelo': m, 'Centralidad': df[m].median(), 'Dispersion': df[m].quantile(0.75) - df[m].quantile(0.25)} for m in MODELOS]
        df_robust = pd.DataFrame(metrics)
        label_centralidad = 'ECRPS Mediano (Rendimiento Típico)'
        label_dispersion = 'Rango Intercuartílico (IQR - Estabilidad Robusta)'
        titulo_compromiso = 'Compromiso Rendimiento vs. Estabilidad (Mediana vs IQR)'

    # Gráfico de dispersión Rendimiento vs Estabilidad
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.scatterplot(data=df_robust, x='Centralidad', y='Dispersion', hue='Modelo', s=150, alpha=0.8, ax=ax)
    for _, row in df_robust.iterrows():
        ax.text(row['Centralidad'], row['Dispersion'], row['Modelo'], fontsize=9, ha='left', va='bottom')
    ax.set_xlabel(label_centralidad, fontweight='bold')
    ax.set_ylabel(label_dispersion, fontweight='bold')
    ax.set_title(titulo_compromiso, fontsize=14, fontweight='bold')
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend(title='Modelos', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    guardar_grafico(f"6_compromiso_rendimiento_estabilidad_{agg_method}.png")

# --- Funciones que no dependen de la agregación (se ejecutan una sola vez) ---
def graficar_densidades_individuales(df):
    """Crea un gráfico de densidad individual para cada modelo."""
    print(" -> Generando gráficos de densidad individuales (análisis único)...")
    all_ecrps_values = df[MODELOS].values.flatten()
    xlim_max = np.quantile(all_ecrps_values[~np.isnan(all_ecrps_values)], 0.995)
    for modelo in MODELOS:
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.kdeplot(df[modelo].dropna(), fill=True, color='teal', ax=ax, lw=2.5)
        mean_val, median_val = df[modelo].mean(), df[modelo].median()
        ax.axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.3f}')
        ax.axvline(median_val, color='green', linestyle=':', label=f'Mediana: {median_val:.3f}')
        ax.set_title(f'Distribución del ECRPS - Modelo: {modelo}', fontsize=14, fontweight='bold')
        ax.set_xlabel('ECRPS', fontweight='bold')
        ax.set_ylabel('Densidad', fontweight='bold')
        ax.set_xlim(left=0, right=xlim_max)
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        guardar_grafico(f"7_densidad_{modelo.replace(' ', '_').lower()}.png")

def realizar_test_diebold_mariano(df):
    """Realiza el test de Diebold-Mariano con corrección de Bonferroni."""
    print(" -> Realizando Test de Diebold-Mariano (análisis único)...")
    pairs = list(combinations(MODELOS, 2))
    alpha_bonferroni = 0.05 / len(pairs)
    dm_results = []
    for m1, m2 in pairs:
        e1, e2 = df[m1].dropna(), df[m2].dropna()
        min_len = min(len(e1), len(e2))
        _, p_value = DieboldMarianoTest.dm_test(e1[:min_len], e2[:min_len])
        winner = 'Empate' if p_value >= alpha_bonferroni else (m1 if df[m1].mean() < df[m2].mean() else m2)
        dm_results.append({'Modelo_1': m1, 'Modelo_2': m2, 'Ganador_Bonferroni': winner})
    
    # Heatmap de resultados
    result_matrix = pd.DataFrame(index=MODELOS, columns=MODELOS, data=0)
    for _, row in pd.DataFrame(dm_results).iterrows():
        if row['Ganador_Bonferroni'] == row['Modelo_1']:
            result_matrix.loc[row['Modelo_1'], row['Modelo_2']], result_matrix.loc[row['Modelo_2'], row['Modelo_1']] = 1, -1
        elif row['Ganador_Bonferroni'] == row['Modelo_2']:
            result_matrix.loc[row['Modelo_1'], row['Modelo_2']], result_matrix.loc[row['Modelo_2'], row['Modelo_1']] = -1, 1

    annot_matrix = result_matrix.applymap(lambda x: {1: 'Gana', -1: 'Pierde', 0: 'Empate'}[x])
    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(result_matrix.astype(float), annot=annot_matrix, fmt='s', cmap=['red', 'lightgray', 'green'], cbar=False, ax=ax)
    ax.set_title('Resultado Test Diebold-Mariano (con corrección de Bonferroni)', fontweight='bold')
    guardar_grafico("8_dm_heatmap_bonferroni.png")

# ============================================================================
# SCRIPT PRINCIPAL
# ============================================================================
def main():
    crear_directorio_resultados(CARPETA_RESULTADOS)
    try:
        df = pd.read_excel(RUTA_DATOS)
        print("✓ Datos cargados exitosamente.")
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo en la ruta '{RUTA_DATOS}'.")
        return

    # Bucle principal para ejecutar análisis por media y mediana
    for agg_method in ['mean', 'median']:
        print(f"\n{'='*80}\n--- INICIANDO ANÁLISIS BASADO EN LA {agg_method.upper()} ---\n{'='*80}")

        # --- ANÁLISIS 1: ESTACIONARIEDAD ---
        print(f" -> 1. Analizando por estacionariedad ({agg_method})...")
        df_est = df[df['Escenario'].isin(ESCENARIOS_ESTACIONARIOS)]
        df_no_est = df[df['Escenario'].isin(ESCENARIOS_NO_ESTACIONARIOS)]
        agregados_est = df_est[MODELOS].agg(agg_method)
        agregados_no_est = df_no_est[MODELOS].agg(agg_method)
        orden_est = (agregados_est + agregados_no_est).sort_values().index
        graficar_comparacion_barras(agregados_est, agregados_no_est, orden_est, 'Estacionarios', 'No Estacionarios', agg_method, f'1_comparacion_estacionariedad_{agg_method}.png')
        
        # --- ANÁLISIS 2: LINEALIDAD ---
        print(f" -> 2. Analizando por linealidad ({agg_method})...")
        df_lin = df[df['Escenario'].isin(ESCENARIOS_LINEALES)]
        df_no_lin = df[df['Escenario'].isin(ESCENARIOS_NO_LINEALES)]
        agregados_lin = df_lin[MODELOS].agg(agg_method)
        agregados_no_lin = df_no_lin[MODELOS].agg(agg_method)
        orden_lin = (agregados_lin + agregados_no_lin).sort_values().index
        graficar_comparacion_barras(agregados_lin, agregados_no_lin, orden_lin, 'Lineales', 'No Lineales', agg_method, f'2_comparacion_linealidad_{agg_method}.png')

        # --- ANÁLISIS 3: HEATMAPS GENERALES ---
        print(f" -> 3. Generando heatmaps generales ({agg_method})...")
        heatmap_esc_df = df.groupby('Escenario')[MODELOS].agg(agg_method)
        generar_heatmap(heatmap_esc_df, agg_method, 'Desempeño por Escenario', f'3_heatmap_escenario_{agg_method}.png', figsize=(14, 6))
        heatmap_dist_df = df.groupby('Distribución')[MODELOS].agg(agg_method)
        generar_heatmap(heatmap_dist_df, agg_method, 'Desempeño por Distribución', f'3_heatmap_distribucion_{agg_method}.png')

        # --- ANÁLISIS 4 & 5: EVOLUCIÓN VS VARIANZA Y PASO ---
        print(f" -> 4. Analizando ECRPS vs Varianza ({agg_method})...")
        graficar_evolucion_metrica_por_tipo(df, 'Varianza error', agg_method, 'Varianza error', 'varianza')
        print(f" -> 5. Analizando ECRPS vs Paso ({agg_method})...")
        graficar_evolucion_metrica_por_tipo(df, 'Paso', agg_method, 'Paso (Horizonte)', 'paso')
        
        # --- ANÁLISIS 6: ROBUSTEZ Y ESTABILIDAD ---
        analizar_robustez_estabilidad(df, agg_method)

    # --- ANÁLISIS QUE SE EJECUTAN UNA SOLA VEZ ---
    print(f"\n{'='*80}\n--- INICIANDO ANÁLISIS INDEPENDIENTES DE AGREGACIÓN ---\n{'='*80}")
    # --- ANÁLISIS 7: DENSIDAD DE ERRORES ---
    graficar_densidades_individuales(df)
    
    # --- ANÁLISIS 8: TEST DE DIEBOLD-MARIANO ---
    realizar_test_diebold_mariano(df)
    
    print(f"\n✓ Análisis completo. Resultados guardados en la carpeta '{CARPETA_RESULTADOS}'.")

if __name__ == "__main__":
    main()

Directorio 'resultados_completos_media_mediana' creado.
✓ Datos cargados exitosamente.

--- INICIANDO ANÁLISIS BASADO EN LA MEAN ---
 -> 1. Analizando por estacionariedad (mean)...
 -> 2. Analizando por linealidad (mean)...
 -> 3. Generando heatmaps generales (mean)...
 -> 4. Analizando ECRPS vs Varianza (mean)...
 -> 5. Analizando ECRPS vs Paso (mean)...
 -> Analizando robustez y estabilidad (basado en mean)...

--- INICIANDO ANÁLISIS BASADO EN LA MEDIAN ---
 -> 1. Analizando por estacionariedad (median)...
 -> 2. Analizando por linealidad (median)...
 -> 3. Generando heatmaps generales (median)...
 -> 4. Analizando ECRPS vs Varianza (median)...
 -> 5. Analizando ECRPS vs Paso (median)...
 -> Analizando robustez y estabilidad (basado en median)...

--- INICIANDO ANÁLISIS INDEPENDIENTES DE AGREGACIÓN ---
 -> Generando gráficos de densidad individuales (análisis único)...
 -> Realizando Test de Diebold-Mariano (análisis único)...


C:\Users\pleal\AppData\Local\Temp\ipykernel_16608\4105642524.py:199: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annot_matrix = result_matrix.applymap(lambda x: {1: 'Gana', -1: 'Pierde', 0: 'Empate'}[x])



✓ Análisis completo. Resultados guardados en la carpeta 'resultados_completos_media_mediana'.


# Machine Learning

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ============================================================================
# CONFIGURACIÓN
# ============================================================================
RUTA_DATOS = "./Datos/datos_combinados.xlsx"
CARPETA_RESULTADOS = "resultados_meta_modelo"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================
def crear_directorio_resultados(nombre_carpeta):
    """Crea la carpeta de resultados si no existe."""
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)
        print(f"Directorio '{nombre_carpeta}' creado.")

def guardar_grafico(nombre_archivo):
    """Guarda la figura actual en un archivo y la cierra."""
    ruta_completa = os.path.join(CARPETA_RESULTADOS, nombre_archivo)
    plt.savefig(ruta_completa, dpi=300, bbox_inches='tight')
    print(f" -> Gráfico guardado en: {ruta_completa}")
    plt.close()

def plot_feature_importance(model, feature_names, model_name):
    """Grafica la importancia de las características del modelo."""
    importances = model.feature_importances_
    df_importance = pd.DataFrame({
        'Característica': feature_names,
        'Importancia': importances
    }).sort_values(by='Importancia', ascending=True)

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(df_importance['Característica'], df_importance['Importancia'], color='steelblue')
    ax.set_xlabel('Importancia')
    ax.set_title(f'Importancia de Características - {model_name}')
    plt.tight_layout()
    guardar_grafico(f"feature_importance_{model_name.replace(' ', '_').lower()}.png")

def plot_confusion_matrix(y_true, y_pred, model_name, class_labels):
    """Grafica la matriz de confusión normalizada."""
    cm = confusion_matrix(y_true, y_pred, labels=class_labels, normalize='true')
    df_cm = pd.DataFrame(cm, index=class_labels, columns=class_labels)

    fig, ax = plt.subplots(figsize=(14, 12))
    sns.heatmap(df_cm, annot=True, fmt='.2f', cmap='Blues', ax=ax)
    ax.set_xlabel('Predicción del Recomendador', fontweight='bold')
    ax.set_ylabel('Mejor Modelo Real', fontweight='bold')
    ax.set_title(f'Matriz de Confusión Normalizada - {model_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    guardar_grafico(f"confusion_matrix_{model_name.replace(' ', '_').lower()}.png")

# ============================================================================
# SCRIPT PRINCIPAL
# ============================================================================
def main():
    """Función principal para crear y analizar el meta-modelo."""
    crear_directorio_resultados(CARPETA_RESULTADOS)
    
    # 1. Cargar y preparar los datos
    print("1. Cargando y preparando los datos para el meta-modelo...")
    try:
        df = pd.read_excel(RUTA_DATOS)
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo en la ruta '{RUTA_DATOS}'.")
        return

    features = ['Escenario', 'Distribución', 'Varianza error', 'Paso', 'Tipo de Modelo']
    df_meta = df[features + MODELOS].copy()
    df_meta['Mejor_Modelo'] = df_meta[MODELOS].idxmin(axis=1)
    df_meta.dropna(subset=features, inplace=True)

    X = df_meta[features]
    y = df_meta['Mejor_Modelo']
    
    # 2. Preprocesamiento de características
    print("2. Realizando preprocesamiento (One-Hot Encoding)...")
    categorical_features = ['Escenario', 'Distribución', 'Tipo de Modelo']
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_encoded_cats = encoder.fit_transform(X[categorical_features])
    
    # Obtener los nombres de las nuevas columnas codificadas
    encoded_feature_names = encoder.get_feature_names_out(categorical_features)
    
    # Combinar características numéricas y codificadas
    X_numeric = X.drop(columns=categorical_features)
    X_processed = np.hstack((X_numeric.values, X_encoded_cats))
    
    # Nombres de todas las características finales
    final_feature_names = list(X_numeric.columns) + list(encoded_feature_names)

    # 3. Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # 4. Definir y entrenar los modelos
    print("\n3. Entrenando y evaluando los modelos recomendadores...")
    models_to_train = {
        "Árbol de Decisión": DecisionTreeClassifier(max_depth=5, min_samples_leaf=20, random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    }
    
    class_labels = sorted(y.unique())

    for name, model in models_to_train.items():
        print(f"\n--- Analizando: {name} ---")
        
        # Entrenar
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_test)
        
        # Evaluar y mostrar reporte
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Precisión: {accuracy:.2%}")
        print("Reporte de Clasificación:")
        print(classification_report(y_test, y_pred, labels=class_labels))
        
        # Generar visualizaciones
        print("Generando visualizaciones...")
        plot_feature_importance(model, final_feature_names, name)
        plot_confusion_matrix(y_test, y_pred, name, class_labels)
        
        # Visualizar el árbol de decisión si corresponde
        if name == "Árbol de Decisión":
            fig, ax = plt.subplots(figsize=(25, 15))
            plot_tree(model, feature_names=final_feature_names, class_names=class_labels, 
                      filled=True, rounded=True, fontsize=10, ax=ax)
            ax.set_title("Árbol de Decisión para Recomendación de Modelos", fontsize=20)
            guardar_grafico("decision_tree_visualization.png")

    print("\n✓ Análisis del meta-modelo completado.")

if __name__ == "__main__":
    main()

Directorio 'resultados_meta_modelo' creado.
1. Cargando y preparando los datos para el meta-modelo...
2. Realizando preprocesamiento (One-Hot Encoding)...

3. Entrenando y evaluando los modelos recomendadores...

--- Analizando: Árbol de Decisión ---
Precisión: 45.83%
Reporte de Clasificación:
                     precision    recall  f1-score   support

              AREPD       0.00      0.00      0.00         9
            AV-MCPS       0.00      0.00      0.00        22
Block Bootstrapping       0.50      0.88      0.64       286
             DeepAR       0.00      0.00      0.00        31
         EnCQR-LSTM       0.18      0.29      0.22        35
               LSPM       0.00      0.00      0.00        42
              LSPMW       0.00      0.00      0.00        16
               MCPS       0.00      0.00      0.00        20
    Sieve Bootstrap       0.29      0.09      0.13       139

           accuracy                           0.46       600
          macro avg       0.11  

c:\Users\pleal\Documents\Unal\Tesis\Codigo\Prediccion_Probabilistica\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pleal\Documents\Unal\Tesis\Codigo\Prediccion_Probabilistica\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pleal\Documents\Unal\Tesis\Codigo\Prediccion_Probabilistica\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

 -> Gráfico guardado en: resultados_meta_modelo\feature_importance_árbol_de_decisión.png
 -> Gráfico guardado en: resultados_meta_modelo\confusion_matrix_árbol_de_decisión.png
 -> Gráfico guardado en: resultados_meta_modelo\decision_tree_visualization.png

--- Analizando: Gradient Boosting ---
Precisión: 43.17%
Reporte de Clasificación:
                     precision    recall  f1-score   support

              AREPD       0.00      0.00      0.00         9
            AV-MCPS       0.00      0.00      0.00        22
Block Bootstrapping       0.58      0.71      0.64       286
             DeepAR       0.21      0.26      0.23        31
         EnCQR-LSTM       0.21      0.20      0.20        35
               LSPM       0.18      0.10      0.12        42
              LSPMW       0.12      0.06      0.08        16
               MCPS       0.00      0.00      0.00        20
    Sieve Bootstrap       0.30      0.26      0.28       139

           accuracy                           0.4

# Analisis Escalonado

## Analisis Especifico

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from itertools import combinations
import warnings

warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Datos/datos_combinados.xlsx"
DIR_SALIDA = "./resultados_preguntas_profundizacion"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

COLORES_MODELOS = {
    'AREPD': '#e41a1c',
    'AV-MCPS': '#377eb8',
    'Block Bootstrapping': '#4daf4a',
    'DeepAR': '#984ea3',
    'EnCQR-LSTM': '#ff7f00',
    'LSPM': '#ffff33',
    'LSPMW': '#a65628',
    'MCPS': '#f781bf',
    'Sieve Bootstrap': '#999999'
}


# ============================================================================
# CLASE PRINCIPAL DE ANÁLISIS - PREGUNTAS DE PROFUNDIZACIÓN
# ============================================================================

class AnalizadorPreguntasProfundizacion:
    """Análisis específico para responder preguntas de profundización"""

    def __init__(self, ruta_datos):
        """Inicializa el analizador"""
        print("\n" + "=" * 80)
        print("ANÁLISIS DE PREGUNTAS DE PROFUNDIZACIÓN")
        print("=" * 80 + "\n")

        self.df = pd.read_excel(ruta_datos)
        self.modelos = MODELOS
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)

        # Extraer características del escenario
        self._extraer_caracteristicas()

        print(f"✓ Datos cargados: {self.df.shape[0]} filas, {self.df.shape[1]} columnas")
        print(f"✓ Modelos a analizar: {len(self.modelos)}")
        print(f"✓ Directorio de salida: {self.dir_salida}")
        print("\n" + "=" * 80 + "\n")

    def _extraer_caracteristicas(self):
        """Extrae características individuales del escenario"""
        self.df['Estacionario'] = self.df['Escenario'].apply(
            lambda x: 'Estacionario' if 'Estacionario' in x and 'No_Estacionario' not in x else 'No Estacionario'
        )

        self.df['Lineal'] = self.df['Escenario'].apply(
            lambda x: 'Lineal' if 'Lineal' in x and 'No_Lineal' not in x else 'No Lineal'
        )

    def ejecutar_analisis_completo(self):
        """Ejecuta todos los análisis para las preguntas"""
        print("\n" + "🔬" * 40 + "\n")

        # Pregunta 1: Punto de quiebre de AREPD
        print("1️⃣  Pregunta 1: Punto de quiebre de AREPD...")
        self._pregunta_1_punto_quiebre_arepd()

        # Pregunta 2: Robustez de Block Bootstrapping vs Sieve Bootstrap
        print("\n2️⃣  Pregunta 2: Zona de dominio Block Bootstrapping...")
        self._pregunta_2_zona_dominio_bb()

        # Pregunta 3: Deterioro acelerado AV-MCPS
        print("\n3️⃣  Pregunta 3: Deterioro de AV-MCPS por horizonte...")
        self._pregunta_3_deterioro_av_mcps()

        # Pregunta 4: Penalización Normal multiplicativa
        print("\n4️⃣  Pregunta 4: Efecto multiplicativo distribución Normal...")
        self._pregunta_4_penalizacion_normal()

        # Pregunta 5: Frontera de colapso Deep Learning
        print("\n5️⃣  Pregunta 5: Frontera de colapso Deep Learning...")
        self._pregunta_5_frontera_dl()

        # Pregunta 6: Consistencia "Mejor Modelo"
        print("\n6️⃣  Pregunta 6: Validación de 'Mejor Modelo'...")
        self._pregunta_6_consistencia_mejor_modelo()

        # Pregunta 7: Análisis de segunda derivada
        print("\n7️⃣  Pregunta 7: Aceleración del deterioro...")
        self._pregunta_7_segunda_derivada()

        # Pregunta 8: Interacción No Linealidad × Varianza
        print("\n8️⃣  Pregunta 8: Colapso LSPM con varianza alta...")
        self._pregunta_8_interaccion_nolineal_varianza()

        # Pregunta 9: Mapa de decisión operacional
        print("\n9️⃣  Pregunta 9: Mapa de decisión operacional...")
        self._pregunta_9_mapa_decision()

        print("\n" + "=" * 80)
        print("✅ ANÁLISIS DE PREGUNTAS COMPLETO")
        print(f"📁 Resultados guardados en: {self.dir_salida}")
        print("=" * 80 + "\n")

    # ========================================================================
    # PREGUNTA 1: PUNTO DE QUIEBRE DE AREPD
    # ========================================================================

    def _pregunta_1_punto_quiebre_arepd(self):
        """
        ¿Existe un punto de quiebre específico en la varianza del error donde 
        AREPD comienza su deterioro catastrófico?
        ¿Este punto es consistente entre distribuciones Normal y Mixture?
        """
        
        # Filtrar distribuciones de interés
        df_normal = self.df[self.df['Distribución'] == 'Normal'].copy()
        df_mixture = self.df[self.df['Distribución'] == 'Mixture'].copy()
        
        # Calcular rendimiento promedio por varianza para AREPD
        varianzas = sorted(self.df['Varianza error'].unique())
        
        arepd_normal = []
        arepd_mixture = []
        otros_normal = []
        otros_mixture = []
        
        for var in varianzas:
            # AREPD
            arepd_normal.append(df_normal[df_normal['Varianza error'] == var]['AREPD'].mean())
            arepd_mixture.append(df_mixture[df_mixture['Varianza error'] == var]['AREPD'].mean())
            
            # Promedio de otros modelos robustos para comparación
            otros_modelos = ['Block Bootstrapping', 'Sieve Bootstrap', 'LSPM']
            otros_normal.append(df_normal[df_normal['Varianza error'] == var][otros_modelos].mean().mean())
            otros_mixture.append(df_mixture[df_mixture['Varianza error'] == var][otros_modelos].mean().mean())
        
        # FIGURA 1.1: Evolución de AREPD vs modelos robustos
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
        
        # Normal
        ax1.plot(varianzas, arepd_normal, 'o-', label='AREPD', 
                color=COLORES_MODELOS['AREPD'], linewidth=3, markersize=10)
        ax1.plot(varianzas, otros_normal, 's--', label='Promedio Modelos Robustos',
                color='green', linewidth=2, markersize=8, alpha=0.7)
        ax1.set_xlabel('Varianza del Error', fontweight='bold', fontsize=12)
        ax1.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
        ax1.set_title('Distribución Normal: Punto de Quiebre AREPD', 
                     fontweight='bold', fontsize=13)
        ax1.legend(fontsize=11)
        ax1.grid(True, alpha=0.3)
        
        # Mixture
        ax2.plot(varianzas, arepd_mixture, 'o-', label='AREPD',
                color=COLORES_MODELOS['AREPD'], linewidth=3, markersize=10)
        ax2.plot(varianzas, otros_mixture, 's--', label='Promedio Modelos Robustos',
                color='green', linewidth=2, markersize=8, alpha=0.7)
        ax2.set_xlabel('Varianza del Error', fontweight='bold', fontsize=12)
        ax2.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
        ax2.set_title('Distribución Mixture: Punto de Quiebre AREPD',
                     fontweight='bold', fontsize=13)
        ax2.legend(fontsize=11)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_1_punto_quiebre_arepd_comparativo.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 1.2: Tasa de deterioro incremental
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Calcular tasas de cambio
        tasas_normal = np.diff(arepd_normal) / np.diff(varianzas)
        tasas_mixture = np.diff(arepd_mixture) / np.diff(varianzas)
        var_medias = [(varianzas[i] + varianzas[i+1])/2 for i in range(len(varianzas)-1)]
        
        ax.plot(var_medias, tasas_normal, 'o-', label='Normal', 
               color='red', linewidth=3, markersize=10)
        ax.plot(var_medias, tasas_mixture, 's-', label='Mixture',
               color='orange', linewidth=3, markersize=10)
        ax.axhline(y=0, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
        ax.set_xlabel('Varianza del Error (punto medio)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Tasa de Deterioro (ΔECRPS/ΔVarianza)', fontweight='bold', fontsize=12)
        ax.set_title('AREPD: Aceleración del Deterioro por Distribución\n(Mayor pendiente = Colapso más rápido)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=12, loc='upper left')
        ax.grid(True, alpha=0.3)
        
        # Identificar punto de máxima aceleración
        max_accel_normal_idx = np.argmax(tasas_normal)
        max_accel_mixture_idx = np.argmax(tasas_mixture)
        
        ax.annotate(f'Máxima aceleración\nVarianza ≈ {var_medias[max_accel_normal_idx]:.3f}',
                   xy=(var_medias[max_accel_normal_idx], tasas_normal[max_accel_normal_idx]),
                   xytext=(10, 20), textcoords='offset points',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', color='red', lw=2),
                   fontsize=10, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_2_tasa_deterioro_arepd.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # Calcular umbral de quiebre (donde la tasa supera 2x la mediana)
        umbral_normal = var_medias[max_accel_normal_idx] if len(var_medias) > 0 else None
        umbral_mixture = var_medias[max_accel_mixture_idx] if len(var_medias) > 0 else None
        
        print(f"   ✓ Punto de quiebre AREPD (Normal): Varianza ≈ {umbral_normal:.3f}")
        print(f"   ✓ Punto de quiebre AREPD (Mixture): Varianza ≈ {umbral_mixture:.3f}")
        print(f"   ✓ Diferencia entre distribuciones: {abs(umbral_normal - umbral_mixture):.3f}")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 2: ZONA DE DOMINIO BLOCK BOOTSTRAPPING
    # ========================================================================

    def _pregunta_2_zona_dominio_bb(self):
        """
        ¿En qué condiciones EXACTAS Block Bootstrapping supera a Sieve Bootstrap?
        """
        
        # Crear DataFrame de comparación directa
        df_comp = self.df.copy()
        df_comp['BB_mejor'] = df_comp['Block Bootstrapping'] < df_comp['Sieve Bootstrap']
        df_comp['Diferencia'] = df_comp['Sieve Bootstrap'] - df_comp['Block Bootstrapping']
        
        # FIGURA 2.1: Mapa de calor de superioridad
        fig, axes = plt.subplots(2, 2, figsize=(16, 14))
        axes = axes.flatten()
        
        escenarios_principales = [
            ('Estacionario', 'Lineal'),
            ('Estacionario', 'No Lineal'),
            ('No Estacionario', 'Lineal'),
            ('No Estacionario', 'No Lineal')
        ]
        
        for idx, (est, lin) in enumerate(escenarios_principales):
            ax = axes[idx]
            df_esc = df_comp[(df_comp['Estacionario'] == est) & (df_comp['Lineal'] == lin)]
            
            # Crear matriz de diferencias
            pivot = df_esc.pivot_table(
                values='Diferencia',
                index='Distribución',
                columns='Varianza error',
                aggfunc='mean'
            )
            
            sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn', center=0,
                       ax=ax, cbar_kws={'label': 'SB - BB (>0 = BB mejor)'},
                       linewidths=1, linecolor='gray', vmin=-0.02, vmax=0.02)
            ax.set_title(f'{est} + {lin}', fontweight='bold', fontsize=12)
            ax.set_xlabel('Varianza Error', fontweight='bold')
            ax.set_ylabel('Distribución', fontweight='bold')
        
        plt.suptitle('Zona de Dominio: Block Bootstrapping vs Sieve Bootstrap\n(Verde = BB domina, Rojo = SB domina)',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P2_1_zona_dominio_bb_heatmap.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 2.2: Frecuencia de dominio por condiciones
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Calcular % de casos donde BB es mejor
        resultados_dominio = []
        for est in ['Estacionario', 'No Estacionario']:
            for lin in ['Lineal', 'No Lineal']:
                df_esc = df_comp[(df_comp['Estacionario'] == est) & (df_comp['Lineal'] == lin)]
                pct_bb_mejor = (df_esc['BB_mejor'].sum() / len(df_esc) * 100) if len(df_esc) > 0 else 0
                resultados_dominio.append({
                    'Escenario': f'{est[:3]}+{lin[:3]}',
                    'Completo': f'{est} + {lin}',
                    'Pct_BB_Mejor': pct_bb_mejor
                })
        
        df_dominio = pd.DataFrame(resultados_dominio).sort_values('Pct_BB_Mejor', ascending=False)
        
        colors = ['green' if x > 50 else 'red' for x in df_dominio['Pct_BB_Mejor']]
        bars = ax.barh(df_dominio['Escenario'], df_dominio['Pct_BB_Mejor'],
                      color=colors, alpha=0.7, edgecolor='black', linewidth=2)
        ax.axvline(50, color='black', linestyle='--', linewidth=2, label='50% (Equilibrio)')
        ax.set_xlabel('% de casos donde BB supera a SB', fontweight='bold', fontsize=12)
        ax.set_title('Frecuencia de Dominio de Block Bootstrapping\n(>50% = BB generalmente mejor)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='x')
        ax.set_xlim(0, 100)
        
        for i, (bar, val) in enumerate(zip(bars, df_dominio['Pct_BB_Mejor'])):
            ax.text(val + 2, i, f'{val:.1f}%', va='center', fontweight='bold', fontsize=11)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P2_2_frecuencia_dominio_bb.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"   ✓ BB domina en: {df_comp['BB_mejor'].sum()} / {len(df_comp)} casos ({df_comp['BB_mejor'].sum()/len(df_comp)*100:.1f}%)")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 3: DETERIORO AV-MCPS POR HORIZONTE
    # ========================================================================

    def _pregunta_3_deterioro_av_mcps(self):
        """
        ¿El deterioro de AV-MCPS es lineal, cuadrático o exponencial?
        ¿Cambia según el nivel de varianza?
        """
        
        pasos = sorted(self.df['Paso'].unique())
        varianzas = sorted(self.df['Varianza error'].unique())
        
        # FIGURA 3.1: Ajuste de curvas de deterioro
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        # Seleccionar niveles de varianza representativos
        if len(varianzas) >= 4:
            var_seleccionadas = [varianzas[0], varianzas[len(varianzas)//3], 
                                varianzas[2*len(varianzas)//3], varianzas[-1]]
        else:
            var_seleccionadas = varianzas
        
        modelos_comparacion = ['AV-MCPS', 'LSPM', 'Block Bootstrapping']
        
        for idx, var in enumerate(var_seleccionadas[:4]):
            ax = axes[idx]
            df_var = self.df[self.df['Varianza error'] == var]
            
            for modelo in modelos_comparacion:
                valores = [df_var[df_var['Paso'] == p][modelo].mean() for p in pasos]
                ax.plot(pasos, valores, 'o-', label=modelo, 
                       linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo])
            
            ax.set_xlabel('Horizonte (Paso)', fontweight='bold', fontsize=11)
            ax.set_ylabel('ECRPS', fontweight='bold', fontsize=11)
            ax.set_title(f'Varianza = {var:.3f}', fontweight='bold', fontsize=12)
            ax.legend(fontsize=10)
            ax.grid(True, alpha=0.3)
        
        plt.suptitle('Evolución del Deterioro por Horizonte: AV-MCPS vs Modelos Estables',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P3_1_deterioro_av_mcps_curvas.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 3.2: Análisis de tipo de crecimiento
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Calcular R² para diferentes tipos de ajuste
        tipos_ajuste = []
        
        for var in varianzas:
            df_var = self.df[self.df['Varianza error'] == var]
            valores_av = [df_var[df_var['Paso'] == p]['AV-MCPS'].mean() for p in pasos]
            
            x = np.array(pasos)
            y = np.array(valores_av)
            
            # Ajuste lineal
            p_lin = np.polyfit(x, y, 1)
            y_lin = np.polyval(p_lin, x)
            r2_lin = 1 - (np.sum((y - y_lin)**2) / np.sum((y - np.mean(y))**2))
            
            # Ajuste cuadrático
            p_quad = np.polyfit(x, y, 2)
            y_quad = np.polyval(p_quad, x)
            r2_quad = 1 - (np.sum((y - y_quad)**2) / np.sum((y - np.mean(y))**2))
            
            # Ajuste exponencial (logarítmico)
            try:
                z = np.polyfit(x, np.log(y + 1e-10), 1)
                y_exp = np.exp(np.polyval(z, x))
                r2_exp = 1 - (np.sum((y - y_exp)**2) / np.sum((y - np.mean(y))**2))
            except:
                r2_exp = 0
            
            mejor_ajuste = max([('Lineal', r2_lin), ('Cuadrático', r2_quad), ('Exponencial', r2_exp)], 
                              key=lambda x: x[1])
            
            tipos_ajuste.append({
                'Varianza': var,
                'R2_Lineal': r2_lin,
                'R2_Cuadratico': r2_quad,
                'R2_Exponencial': r2_exp,
                'Mejor': mejor_ajuste[0]
            })
        
        df_ajustes = pd.DataFrame(tipos_ajuste)
        
        x_pos = np.arange(len(varianzas))
        width = 0.25
        
        ax.bar(x_pos - width, df_ajustes['R2_Lineal'], width, label='Lineal', alpha=0.8)
        ax.bar(x_pos, df_ajustes['R2_Cuadratico'], width, label='Cuadrático', alpha=0.8)
        ax.bar(x_pos + width, df_ajustes['R2_Exponencial'], width, label='Exponencial', alpha=0.8)
        
        ax.set_xlabel('Varianza del Error', fontweight='bold', fontsize=12)
        ax.set_ylabel('R² (Bondad de Ajuste)', fontweight='bold', fontsize=12)
        ax.set_title('AV-MCPS: Tipo de Deterioro por Nivel de Varianza\n(R² más alto = Mejor ajuste)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.set_xticks(x_pos)
        ax.set_xticklabels([f'{v:.3f}' for v in varianzas], rotation=45)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_ylim(0, 1.1)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P3_2_tipo_deterioro_av_mcps.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"   ✓ Tipo de deterioro predominante: {df_ajustes['Mejor'].mode()[0]}")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 4: PENALIZACIÓN NORMAL MULTIPLICATIVA
    # ========================================================================

    def _pregunta_4_penalizacion_normal(self):
        """
        ¿La penalización de la distribución Normal es aditiva o multiplicativa 
        con la no-estacionariedad?
        """
        
        # Calcular deterioro por distribución en cada escenario
        modelos_analisis = ['DeepAR', 'MCPS', 'LSPM', 'Block Bootstrapping']
        
        # FIGURA 4.1: Efecto aditivo vs multiplicativo
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        resultados_interaccion = []
        
        for idx, modelo in enumerate(modelos_analisis):
            ax = axes[idx]
            
            # Calcular penalización por escenario
            penalizaciones = []
            
            for est in ['Estacionario', 'No Estacionario']:
                df_est = self.df[self.df['Estacionario'] == est]
                
                # Rendimiento en Normal vs t-student
                rend_normal = df_est[df_est['Distribución'] == 'Normal'][modelo].mean()
                rend_tstudent = df_est[df_est['Distribución'] == 't-student'][modelo].mean()
                
                deterioro = ((rend_normal - rend_tstudent) / rend_tstudent) * 100
                penalizaciones.append({
                    'Estacionariedad': est,
                    'Deterioro_pct': deterioro
                })
            
            df_pen = pd.DataFrame(penalizaciones)
            
            # Calcular razón de efectos
            det_estacionario = df_pen[df_pen['Estacionariedad'] == 'Estacionario']['Deterioro_pct'].values[0]
            det_no_estacionario = df_pen[df_pen['Estacionariedad'] == 'No Estacionario']['Deterioro_pct'].values[0]
            
            razon = det_no_estacionario / det_estacionario if det_estacionario != 0 else 0
            es_multiplicativo = razon > 1.5  # Si el efecto es >50% mayor, es multiplicativo
            
            resultados_interaccion.append({
                'Modelo': modelo,
                'Razon': razon,
                'Tipo': 'Multiplicativo' if es_multiplicativo else 'Aditivo'
            })
            
            # Visualización
            colors = ['lightblue', 'coral']
            bars = ax.bar(df_pen['Estacionariedad'], df_pen['Deterioro_pct'], 
                         color=colors, alpha=0.7, edgecolor='black', linewidth=2)
            ax.set_ylabel('Deterioro por Normal (%)', fontweight='bold', fontsize=11)
            ax.set_title(f'{modelo}\nRazón: {razon:.2f}x ({("MULTIPLICATIVO" if es_multiplicativo else "ADITIVO")})',
                        fontweight='bold', fontsize=12)
            ax.grid(True, alpha=0.3, axis='y')
            ax.axhline(0, color='black', linestyle='-', linewidth=1)
            
            for bar, val in zip(bars, df_pen['Deterioro_pct']):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                       f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)
        
        plt.suptitle('Interacción: Distribución Normal × No-Estacionariedad\n(Razón >1.5 = Efecto Multiplicativo)',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P4_1_penalizacion_normal_interaccion.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 4.2: Resumen de tipos de interacción
        fig, ax = plt.subplots(figsize=(12, 8))
        
        df_interaccion = pd.DataFrame(resultados_interaccion).sort_values('Razon', ascending=False)
        
        colors_tipo = ['red' if x == 'Multiplicativo' else 'green' for x in df_interaccion['Tipo']]
        bars = ax.barh(df_interaccion['Modelo'], df_interaccion['Razon'],
                      color=colors_tipo, alpha=0.7, edgecolor='black', linewidth=2)
        ax.axvline(1.5, color='black', linestyle='--', linewidth=2, label='Umbral Multiplicativo (1.5x)')
        ax.axvline(1.0, color='gray', linestyle=':', linewidth=1.5, alpha=0.5)
        ax.set_xlabel('Razón de Efectos (No-Est / Est)', fontweight='bold', fontsize=12)
        ax.set_title('Clasificación del Tipo de Interacción por Modelo\n(Rojo = Multiplicativo, Verde = Aditivo)',
                    fontweight='bold', fontsize=14, pad=20)
        


PREGUNTA 1: ANÁLISIS DEL PUNTO DE QUIEBRE DE AREPD

📊 Datos disponibles:
   • Distribución Normal: 0 observaciones
   • Distribución Mixture: 0 observaciones
   • Niveles de varianza: 4
   • Rango: 0.2000 a 3.0000

❌ ERROR: No hay datos válidos para el análisis


## Analisis General Corregido*

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from itertools import combinations
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Datos/datos_combinados.xlsx"
DIR_SALIDA = "./resultados_base_completa_mejorado"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Colores únicos para 9 modelos
COLORES_MODELOS = {
    'AREPD': '#e41a1c',
    'AV-MCPS': '#377eb8',
    'Block Bootstrapping': '#4daf4a',
    'DeepAR': '#984ea3',
    'EnCQR-LSTM': '#ff7f00',
    'LSPM': '#ffff33',
    'LSPMW': '#a65628',
    'MCPS': '#f781bf',
    'Sieve Bootstrap': '#999999'
}

# Características para el meta-modelo
CARACTERISTICAS_META_MODELO = [
    'Estacionario', 'Lineal', 'Tipo de Modelo',
    'Distribución', 'Varianza error', 'Paso'
]
CARACTERISTICAS_NUMERICAS_META_MODELO = ['Varianza error', 'Paso']
CARACTERISTICAS_CATEGORICAS_META_MODELO = [
    'Estacionario', 'Lineal', 'Tipo de Modelo', 'Distribución'
]


# ============================================================================
# FUNCIONES AUXILIARES - TEST DIEBOLD-MARIANO (Sin cambios)
# ============================================================================

def diebold_mariano_test(errores1, errores2, h=1, alternative='two-sided'):
    """Test de Diebold-Mariano para comparar precisión de pronósticos"""
    e1 = np.asarray(errores1)
    e2 = np.asarray(errores2)

    if len(e1) != len(e2):
        raise ValueError("Los vectores de errores deben tener la misma longitud")

    n = len(e1)
    d = e1 - e2
    d_mean = np.mean(d)

    # Varianza con corrección de autocorrelación
    gamma_0 = np.var(d, ddof=1)
    gamma_sum = 0
    for k in range(1, h):
        if k < n:
            gamma_k = np.mean((d[:-k] - d_mean) * (d[k:] - d_mean))
            gamma_sum += 2 * gamma_k

    var_d = (gamma_0 + gamma_sum) / n

    # Corrección de Harvey-Leybourne-Newbold
    hlnc = np.sqrt((n + 1 - 2 * h + h * (h - 1) / n) / n)

    if var_d > 0:
        dm_stat = d_mean / np.sqrt(var_d)
        dm_stat_corrected = dm_stat * hlnc
    else:
        dm_stat = 0
        dm_stat_corrected = 0

    # P-valor
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.t.cdf(abs(dm_stat_corrected), df=n - 1))
    elif alternative == 'less':
        p_value = stats.t.cdf(dm_stat_corrected, df=n - 1)
    elif alternative == 'greater':
        p_value = 1 - stats.t.cdf(dm_stat_corrected, df=n - 1)
    else:
        raise ValueError("alternative debe ser 'two-sided', 'less' o 'greater'")

    return {
        'dm_statistic': dm_stat,
        'dm_statistic_corrected': dm_stat_corrected,
        'p_value': p_value,
        'mean_diff': d_mean,
        'modelo1_mejor': d_mean < 0,
        'n': n
    }


def comparaciones_multiples_dm(df, modelos, alpha=0.05):
    """Comparaciones múltiples con corrección de Bonferroni"""
    n_comparaciones = len(list(combinations(modelos, 2)))
    alpha_bonferroni = alpha / n_comparaciones

    resultados = []

    for modelo1, modelo2 in combinations(modelos, 2):
        try:
            dm_result = diebold_mariano_test(
                df[modelo1].values,
                df[modelo2].values,
                h=1,
                alternative='two-sided'
            )

            significativo = dm_result['p_value'] < alpha_bonferroni

            if significativo:
                if dm_result['mean_diff'] < 0:
                    ganador = modelo1
                else:
                    ganador = modelo2
            else:
                ganador = "No hay diferencia"

            resultados.append({
                'Modelo_1': modelo1,
                'Modelo_2': modelo2,
                'DM_Statistic': dm_result['dm_statistic_corrected'],
                'p_value': dm_result['p_value'],
                'p_value_bonferroni': alpha_bonferroni,
                'Significativo': significativo,
                'Ganador': ganador,
                'Diff_Media': dm_result['mean_diff']
            })

        except Exception as e:
            # print(f"Error en DM test entre {modelo1} y {modelo2}: {e}")
            continue

    return pd.DataFrame(resultados), alpha_bonferroni


def calcular_ranking_dm(df_comparaciones, modelos):
    """Calcula ranking basado en resultados DM"""
    n = len(modelos)
    matriz = pd.DataFrame(np.zeros((n, n)), index=modelos, columns=modelos)

    for _, row in df_comparaciones.iterrows():
        m1, m2 = row['Modelo_1'], row['Modelo_2']
        if row['Significativo']:
            if row['Ganador'] == m1:
                matriz.loc[m1, m2] = 1
                matriz.loc[m2, m1] = -1
            elif row['Ganador'] == m2:
                matriz.loc[m2, m1] = 1
                matriz.loc[m1, m2] = -1

    ranking_data = []
    for modelo in modelos:
        victorias = (matriz.loc[modelo] == 1).sum()
        derrotas = (matriz.loc[modelo] == -1).sum()
        empates = (matriz.loc[modelo] == 0).sum() - 1 # Excluir la comparación consigo mismo
        score = victorias - derrotas
        total_comparaciones = victorias + derrotas + empates if (victorias + derrotas + empates) > 0 else 1 # Evitar division by zero
        pct_victorias = (victorias / total_comparaciones * 100) if total_comparaciones > 0 else 0


        ranking_data.append({
            'Modelo': modelo,
            'Victorias': int(victorias),
            'Derrotas': int(derrotas),
            'Empates': int(empates),
            'Score': int(score),
            'Pct_Victorias': round(pct_victorias, 2)
        })

    df_ranking = pd.DataFrame(ranking_data)
    df_ranking = df_ranking.sort_values('Score', ascending=False).reset_index(drop=True)
    df_ranking['Rank'] = range(1, len(df_ranking) + 1)

    return df_ranking, matriz


# ============================================================================
# CLASE PRINCIPAL DE ANÁLISIS - MEJORADA
# ============================================================================

class AnalizadorBaseCompleta:
    """Análisis completo de la base de datos en 8 dimensiones + PFI/PDP/ICE"""

    def __init__(self, ruta_datos):
        """Inicializa el analizador"""
        print("\n" + "=" * 80)
        print("INICIANDO ANÁLISIS COMPLETO DE BASE DE DATOS - VERSIÓN MEJORADA")
        print("=" * 80 + "\n")

        self.df = pd.read_excel(ruta_datos)
        self.modelos = MODELOS
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)

        # Extraer características del escenario
        self._extraer_caracteristicas()

        # Preprocesar datos para meta-modelo
        self.preprocessor, self.X_processed = self._preprocess_meta_features()
        self.meta_models = {} # Almacenar meta-modelos entrenados
        self.pfi_results = {} # Almacenar resultados de PFI

        print(f"✓ Datos cargados: {self.df.shape[0]} filas, {self.df.shape[1]} columnas")
        print(f"✓ Modelos a analizar: {len(self.modelos)}")
        print(f"✓ Directorio de salida: {self.dir_salida}")
        print("\n" + "=" * 80 + "\n")

    def _extraer_caracteristicas(self):
        """Extrae características individuales del escenario (sin cambios)"""
        self.df['Estacionario'] = self.df['Escenario'].apply(
            lambda x: 'Estacionario' if 'Estacionario' in x and 'No_Estacionario' not in x else 'No Estacionario'
        )

        self.df['Lineal'] = self.df['Escenario'].apply(
            lambda x: 'Lineal' if 'Lineal' in x and 'No_Lineal' not in x else 'No Lineal'
        )

        print("✓ Características extraídas:")
        print(f"  - Estacionariedad: {self.df['Estacionario'].unique()}")
        print(f"  - Linealidad: {self.df['Lineal'].unique()}")
        print(f"  - Tipos de Modelo: {self.df['Tipo de Modelo'].unique()}")
        print(f"  - Distribuciones: {self.df['Distribución'].unique()}")
        print(f"  - Varianzas: {sorted(self.df['Varianza error'].unique())}")
        print(f"  - Pasos: {sorted(self.df['Paso'].unique())}")

    def _preprocess_meta_features(self):
        """
        Preprocesa las características para el meta-modelo (OneHotEncoding para categóricas).
        """
        numeric_features = CARACTERISTICAS_NUMERICAS_META_MODELO
        categorical_features = CARACTERISTICAS_CATEGORICAS_META_MODELO

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', numeric_features),
                # AÑADIR sparse_output=False AQUÍ PARA SOLUCIONAR EL ERROR
                ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
            ],
            remainder='drop'
        )
        
        # Ajustar y transformar X
        X = self.df[CARACTERISTICAS_META_MODELO]
        X_processed_array = preprocessor.fit_transform(X)
        
        # Obtener nombres de las características preprocesadas para PFI/PDP
        # Se debe usar get_feature_names_out para versiones recientes de sklearn
        try:
            ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
        except AttributeError:
             # Fallback para versiones más antiguas
            ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names(categorical_features)
            
        feature_names = numeric_features + list(ohe_feature_names)

        # Ahora la creación del DataFrame funcionará
        return preprocessor, pd.DataFrame(X_processed_array, columns=feature_names)


    def _train_meta_model(self, target_model_name):
        """
        Entrena un RandomForestRegressor para predecir el error de un modelo de pronóstico
        basado en las características de la simulación.
        """
        if target_model_name in self.meta_models:
            return self.meta_models[target_model_name]

        print(f"   Entrenando meta-modelo para {target_model_name}...")
        y = self.df[target_model_name]
        
        # Usamos un pipeline simplificado para que PFI/PDP puedan trabajar con el preprocesador
        # directamente si fuera necesario, aunque aquí ya pasamos X_processed.
        # En este caso, el preprocessor ya se usó para obtener X_processed.
        # Creamos solo el modelo RandomForestRegressor.
        
        meta_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        meta_model.fit(self.X_processed, y)
        self.meta_models[target_model_name] = meta_model
        print(f"   ✓ Meta-modelo entrenado para {target_model_name}.")
        return meta_model

    def ejecutar_analisis_completo(self):
        """Ejecuta todos los análisis, incluyendo los nuevos"""
        print("\n" + "🔬" * 40 + "\n")

        # 1. Impacto de Estacionariedad
        print("1️⃣  Analizando impacto de Estacionariedad...")
        self._analisis_estacionariedad()

        # 2. Impacto de Linealidad
        print("2️⃣  Analizando impacto de Linealidad...")
        self._analisis_linealidad()

        # 3. Efecto del Modelo Generador
        print("3️⃣  Analizando efecto del Modelo Generador...")
        self._analisis_modelo_generador()

        # 4. Influencia de Distribución
        print("4️⃣  Analizando influencia de Distribución...")
        self._analisis_distribucion()

        # 5. Impacto de Varianza
        print("5️⃣  Analizando impacto de Varianza...")
        self._analisis_varianza()

        # 6. Deterioro por Horizonte
        print("6️⃣  Analizando deterioro por Horizonte...")
        self._analisis_horizonte()

        # 7. Robustez y Estabilidad
        print("7️⃣  Analizando Robustez y Estabilidad...")
        self._analisis_robustez()

        # 8. Diferencias Estadísticamente Significativas
        print("8️⃣  Analizando Diferencias Estadísticamente Significativas...")
        self._analisis_significancia()

        # NUEVO: 9. Análisis de Impacto de Características con PFI
        print("\n9️⃣  Analizando Impacto de Características (Permutation Importance)...")
        self._analisis_impacto_pfi()

        # NUEVO: 10. Análisis de Variabilidad con PDP e ICE
        print("\n🔟 Analizando Variabilidad (PDP e ICE)...")
        self._analisis_variabilidad_pdp_ice()

        # Resumen ejecutivo
        print("\n✨ Generando Resumen Ejecutivo...")
        self._generar_resumen_ejecutivo()

        print("\n" + "=" * 80)
        print("✅ ANÁLISIS COMPLETO FINALIZADO")
        print(f"📁 Resultados guardados en: {self.dir_salida}")
        print("=" * 80 + "\n")

    # ========================================================================
    # 1. IMPACTO DE ESTACIONARIEDAD (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_estacionariedad(self):
        """Analiza el impacto de la estacionariedad"""

        # Calcular estadísticas por estacionariedad
        stats_est = []
        for modelo in self.modelos:
            for est in ['Estacionario', 'No Estacionario']:
                df_subset = self.df[self.df['Estacionario'] == est]
                stats_est.append({
                    'Modelo': modelo,
                    'Estacionariedad': est,
                    'Media': df_subset[modelo].mean(),
                    'Std': df_subset[modelo].std(),
                    'Mediana': df_subset[modelo].median()
                })

        df_stats = pd.DataFrame(stats_est)

        # FIGURA 1.1: Barras comparativas
        fig, ax = plt.subplots(figsize=(14, 9))
        pivot_media = df_stats.pivot(index='Modelo', columns='Estacionariedad', values='Media')
        pivot_media = pivot_media.sort_values('Estacionario')

        x = np.arange(len(pivot_media))
        width = 0.35

        ax.bar(x - width / 2, pivot_media['Estacionario'], width,
               label='Estacionario', color='lightblue', edgecolor='black', linewidth=1.5)
        ax.bar(x + width / 2, pivot_media['No Estacionario'], width,
               label='No Estacionario', color='lightcoral', edgecolor='black', linewidth=1.5)

        ax.set_xlabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Impacto de Estacionariedad: ECRPS Comparativo',
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xticks(x)
        ax.set_xticklabels(pivot_media.index, rotation=45, ha='right', fontsize=11)
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, axis='y')

        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_1_estacionariedad_comparativo.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 1.2: Cambio relativo (barras horizontales)
        fig, ax = plt.subplots(figsize=(12, 9))
        cambio_rel = ((pivot_media['No Estacionario'] - pivot_media['Estacionario']) /
                      pivot_media['Estacionario'] * 100)
        cambio_rel = cambio_rel.sort_values()

        colors = ['green' if x < 0 else 'red' for x in cambio_rel.values]
        bars = ax.barh(cambio_rel.index, cambio_rel.values, color=colors,
                       alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.set_xlabel('Cambio Relativo (%)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro en Datos No Estacionarios\n(Negativo = Mejora, Positivo = Deterioro)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')

        for i, (bar, val) in enumerate(zip(bars, cambio_rel.values)):
            ax.text(val + (3 if val > 0 else -3), i, f'{val:.1f}%',
                    va='center', ha='left' if val > 0 else 'right',
                    fontweight='bold', fontsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_2_estacionariedad_cambio_relativo.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para estacionariedad\n")

    # ========================================================================
    # 2. IMPACTO DE LINEALIDAD (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_linealidad(self):
        """Analiza el impacto de la linealidad"""

        # Calcular estadísticas
        stats_lin = []
        for modelo in self.modelos:
            for lin in ['Lineal', 'No Lineal']:
                df_subset = self.df[self.df['Lineal'] == lin]
                stats_lin.append({
                    'Modelo': modelo,
                    'Linealidad': lin,
                    'Media': df_subset[modelo].mean(),
                    'Std': df_subset[modelo].std(),
                    'Mediana': df_subset[modelo].median()
                })

        df_stats = pd.DataFrame(stats_lin)

        # FIGURA 2.1: Barras comparativas
        fig, ax = plt.subplots(figsize=(14, 9))
        pivot_media = df_stats.pivot(index='Modelo', columns='Linealidad', values='Media')
        pivot_media = pivot_media.sort_values('Lineal')

        x = np.arange(len(pivot_media))
        width = 0.35

        ax.bar(x - width / 2, pivot_media['Lineal'], width,
               label='Lineal', color='lightgreen', edgecolor='black', linewidth=1.5)
        ax.bar(x + width / 2, pivot_media['No Lineal'], width,
               label='No Lineal', color='orange', edgecolor='black', linewidth=1.5)

        ax.set_xlabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Impacto de Linealidad: ECRPS Comparativo',
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xticks(x)
        ax.set_xticklabels(pivot_media.index, rotation=45, ha='right', fontsize=11)
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, axis='y')

        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_1_linealidad_comparativo.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 2.2: Cambio relativo
        fig, ax = plt.subplots(figsize=(14, 10))
        cambio_rel = ((pivot_media['No Lineal'] - pivot_media['Lineal']) /
                      pivot_media['Lineal'] * 100)
        cambio_rel = cambio_rel.sort_values()

        colors = ['green' if x < 0 else 'red' for x in cambio_rel.values]
        bars = ax.barh(cambio_rel.index, cambio_rel.values, color=colors,
                       alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.set_xlabel('Cambio Relativo (%)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro en Datos No Lineales\n(Negativo = Mejora, Positivo = Deterioro)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')

        # Ajustar márgenes del eje x para dar espacio a las etiquetas
        x_min = min(cambio_rel.values)
        x_max = max(cambio_rel.values)
        x_range = x_max - x_min
        ax.set_xlim(x_min - x_range * 0.15, x_max + x_range * 0.15)

        for i, (bar, val) in enumerate(zip(bars, cambio_rel.values)):
            offset = x_range * 0.02
            ax.text(val + (offset if val > 0 else -offset), i, f'{val:.1f}%',
                    va='center', ha='left' if val > 0 else 'right',
                    fontweight='bold', fontsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_2_linealidad_cambio_relativo.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para linealidad\n")

    # ========================================================================
    # 3. EFECTO DEL MODELO GENERADOR (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_modelo_generador(self):
        """Analiza el efecto del modelo generador de datos"""

        pivot_media = self.df.groupby('Tipo de Modelo')[self.modelos].mean()
        tipos = self.df['Tipo de Modelo'].unique()

        # FIGURA 3.2: Heatmap normalizado (Z-scores)
        fig, ax = plt.subplots(figsize=(14, 10))

        pivot_norm = pivot_media.T.sub(pivot_media.T.mean(axis=1), axis=0).div(pivot_media.T.std(axis=1), axis=0)

        sns.heatmap(pivot_norm, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                    ax=ax, cbar_kws={'label': 'Z-Score'},
                    linewidths=0.5, linecolor='gray', vmin=-2, vmax=2)
        ax.set_xlabel('Tipo de Modelo Generador', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo de Predicción', fontweight='bold', fontsize=12)
        ax.set_title('ECRPS Relativo (Z-Score por Modelo)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', rotation=0, labelsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_2_modelo_generador_zscore.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 3.3: Variabilidad por tipo
        fig, ax = plt.subplots(figsize=(12, 8))

        rankings = []
        for tipo in tipos:
            df_tipo = self.df[self.df['Tipo de Modelo'] == tipo]
            medias = df_tipo[self.modelos].mean().sort_values()
            rankings.append({
                'Tipo': tipo,
                'Mejor_Modelo': medias.index[0],
                'Mejor_ECRPS': medias.values[0],
                'Peor_Modelo': medias.index[-1],
                'Peor_ECRPS': medias.values[-1],
                'Rango': medias.values[-1] - medias.values[0]
            })

        df_rankings = pd.DataFrame(rankings).sort_values('Rango', ascending=False)

        y_pos = np.arange(len(df_rankings))
        bars = ax.barh(y_pos, df_rankings['Rango'].values,
                       color='steelblue', alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(df_rankings['Tipo'].values, fontsize=10)
        ax.set_xlabel('Rango de ECRPS (Max - Min)', fontweight='bold', fontsize=12)
        ax.set_title('Variabilidad por Tipo de Generador',
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')

        for i, (bar, val) in enumerate(zip(bars, df_rankings['Rango'].values)):
            ax.text(val + 0.001, i, f'{val:.3f}', va='center', fontweight='bold', fontsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_3_modelo_generador_variabilidad.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para modelo generador\n")

    # ========================================================================
    # 4. INFLUENCIA DE LA DISTRIBUCIÓN (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_distribucion(self):
        """Analiza la influencia de la distribución de errores"""

        pivot_media = self.df.groupby('Distribución')[self.modelos].mean()
        pivot_std = self.df.groupby('Distribución')[self.modelos].std()

        # FIGURA 4.1: Heatmap de rendimiento
        fig, ax = plt.subplots(figsize=(14, 10))

        sns.heatmap(pivot_media.T, annot=True, fmt='.3f', cmap='RdYlGn_r',
                    ax=ax, cbar_kws={'label': 'ECRPS Promedio'},
                    linewidths=0.5, linecolor='gray')
        ax.set_xlabel('Distribución de Errores', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo de Predicción', fontweight='bold', fontsize=12)
        ax.set_title('ECRPS por Distribución de Errores',
                     fontweight='bold', fontsize=14, pad=20)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', rotation=0, labelsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_1_distribucion_heatmap_rendimiento.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 4.2: Heatmap de variabilidad
        fig, ax = plt.subplots(figsize=(14, 10))

        sns.heatmap(pivot_std.T, annot=True, fmt='.3f', cmap='YlOrRd',
                    ax=ax, cbar_kws={'label': 'Desviación Estándar'},
                    linewidths=0.5, linecolor='gray')
        ax.set_xlabel('Distribución de Errores', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo de Predicción', fontweight='bold', fontsize=12)
        ax.set_title('Variabilidad por Distribución de Errores',
                     fontweight='bold', fontsize=14, pad=20)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', rotation=0, labelsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_2_distribucion_heatmap_variabilidad.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para distribución\n")

    # ========================================================================
    # 5. IMPACTO DE VARIANZA (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_varianza(self):
        """Analiza el impacto del nivel de varianza (ruido)"""

        varianzas = sorted(self.df['Varianza error'].unique())

        # FIGURA 5.1: Líneas de tendencia
        fig, ax = plt.subplots(figsize=(14, 8))

        for modelo in self.modelos:
            medias = [self.df[self.df['Varianza error'] == v][modelo].mean()
                      for v in varianzas]
            ax.plot(varianzas, medias, marker='o', label=modelo,
                    linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo])

        ax.set_xlabel('Nivel de Varianza', fontweight='bold', fontsize=12)
        ax.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro con Aumento de Varianza',
                     fontweight='bold', fontsize=14, pad=20)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_xticks(varianzas)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_1_varianza_tendencias.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 5.2: Tasa de crecimiento
        fig, ax = plt.subplots(figsize=(12, 8))

        tasas_crecimiento = {}
        for modelo in self.modelos:
            medias = [self.df[self.df['Varianza error'] == v][modelo].mean()
                      for v in varianzas]
            if len(medias) > 1:
                pendiente = (medias[-1] - medias[0]) / (varianzas[-1] - varianzas[0])
                tasas_crecimiento[modelo] = pendiente

        tc_sorted = dict(sorted(tasas_crecimiento.items(), key=lambda x: x[1]))

        colors_tc = ['green' if v < np.median(list(tc_sorted.values())) else 'red'
                     for v in tc_sorted.values()]
        bars = ax.barh(range(len(tc_sorted)), list(tc_sorted.values()),
                       color=colors_tc, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.set_yticks(range(len(tc_sorted)))
        ax.set_yticklabels(list(tc_sorted.keys()), fontsize=10)
        ax.set_xlabel('Tasa de Crecimiento del Error', fontweight='bold', fontsize=12)
        ax.set_title('Sensibilidad al Ruido\n(Menor = Más Robusto)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.axvline(np.median(list(tc_sorted.values())), color='black',
                   linestyle='--', linewidth=2, label='Mediana')
        ax.grid(True, alpha=0.3, axis='x')
        ax.legend(fontsize=11)

        for i, (bar, val) in enumerate(zip(bars, tc_sorted.values())):
            ax.text(val + (0.0001 if val > 0 else -0.0001), i, f'{val:.4f}',
                    va='center', ha='left' if val > 0 else 'right',
                    fontweight='bold', fontsize=9)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_2_varianza_tasa_crecimiento.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para varianza\n")

    # ========================================================================
    # 6. DETERIORO POR HORIZONTE (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_horizonte(self):
        """Analiza el deterioro del rendimiento con el horizonte de predicción"""

        pasos = sorted(self.df['Paso'].unique())

        # FIGURA 6.1: Evolución paso a paso
        fig, ax = plt.subplots(figsize=(14, 8))

        for modelo in self.modelos:
            medias = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            ax.plot(pasos, medias, marker='o', label=modelo,
                    linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo])

        ax.set_xlabel('Horizonte de Predicción (Paso)', fontweight='bold', fontsize=12)
        ax.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Evolución del ECRPS por Horizonte',
                     fontweight='bold', fontsize=14, pad=20)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_xticks(pasos)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_1_horizonte_evolucion.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 6.2: Tasa de deterioro
        fig, ax = plt.subplots(figsize=(12, 8))

        tasas_deterioro = {}
        for modelo in self.modelos:
            medias = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            if len(medias) > 1:
                pendiente = (medias[-1] - medias[0]) / (pasos[-1] - pasos[0])
                tasas_deterioro[modelo] = pendiente

        td_sorted = dict(sorted(tasas_deterioro.items(), key=lambda x: x[1]))

        colors_td = ['green' if v < np.median(list(td_sorted.values())) else 'red'
                     for v in td_sorted.values()]
        bars = ax.barh(range(len(td_sorted)), list(td_sorted.values()),
                       color=colors_td, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.set_yticks(range(len(td_sorted)))
        ax.set_yticklabels(list(td_sorted.keys()), fontsize=10)
        ax.set_xlabel('Tasa de Deterioro por Paso', fontweight='bold', fontsize=12)
        ax.set_title('Velocidad de Deterioro\n(Menor = Más Estable)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.grid(True, alpha=0.3, axis='x')

        for i, (bar, val) in enumerate(zip(bars, td_sorted.values())):
            ax.text(val + (0.0001 if val > 0 else -0.0001), i, f'{val:.4f}',
                    va='center', ha='left' if val > 0 else 'right',
                    fontweight='bold', fontsize=9)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_2_horizonte_tasa_deterioro.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para horizonte\n")

    # ========================================================================
    # 7. ROBUSTEZ Y ESTABILIDAD (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_robustez(self):
        """Analiza la robustez y estabilidad de los modelos"""

        # Calcular métricas de robustez
        metricas_robustez = []

        for modelo in self.modelos:
            std_global = self.df[modelo].std()
            cv = (self.df[modelo].std() / self.df[modelo].mean()) * 100
            q75, q25 = self.df[modelo].quantile([0.75, 0.25])
            iqr = q75 - q25
            std_entre_escenarios = self.df.groupby('Escenario')[modelo].mean().std()
            std_entre_dist = self.df.groupby('Distribución')[modelo].mean().std()
            std_entre_var = self.df.groupby('Varianza error')[modelo].mean().std()

            metricas_robustez.append({
                'Modelo': modelo,
                'Std_Global': std_global,
                'CV': cv,
                'IQR': iqr,
                'Std_Escenarios': std_entre_escenarios,
                'Std_Distribuciones': std_entre_dist,
                'Std_Varianzas': std_entre_var
            })

        df_robustez = pd.DataFrame(metricas_robustez)

        # FIGURA 7.2: Coeficiente de variación
        fig, ax = plt.subplots(figsize=(12, 8))

        df_sorted = df_robustez.sort_values('CV')
        colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, len(df_sorted)))
        bars = ax.barh(df_sorted['Modelo'], df_sorted['CV'],
                       color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        ax.set_xlabel('Coeficiente de Variación (%)', fontweight='bold', fontsize=12)
        ax.set_title('Variabilidad Relativa\n(Menor = Más Consistente)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')

        for i, (bar, val) in enumerate(zip(bars, df_sorted['CV'].values)):
            ax.text(val + 1, i, f'{val:.1f}%', va='center', fontweight='bold', fontsize=9)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_2_robustez_coef_variacion.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # Guardar para usar después
        self.df_robustez = df_robustez

        print("   ✓ 1 figura generada para robustez\n")

    # ========================================================================
    # 8. DIFERENCIAS ESTADÍSTICAMENTE SIGNIFICATIVAS (Sin cambios funcionales, solo prints)
    # ========================================================================

    def _analisis_significancia(self):
        """Análisis de diferencias estadísticamente significativas con Test DM"""
        print("\n" + "=" * 80)
        print("REALIZANDO TEST DE DIEBOLD-MARIANO")
        print("=" * 80 + "\n")

        # Realizar comparaciones múltiples
        df_comparaciones, alpha_bonf = comparaciones_multiples_dm(
            self.df, self.modelos, alpha=0.05
        )

        print(f"   Número de comparaciones: {len(df_comparaciones)}")
        print(f"   Alpha corregido (Bonferroni): {alpha_bonf:.6f}")
        print(f"   Comparaciones significativas: {df_comparaciones['Significativo'].sum()}")

        # Calcular ranking
        df_ranking, matriz_sup = calcular_ranking_dm(df_comparaciones, self.modelos)

        # FIGURA 8.2: Matriz de superioridad
        fig, ax = plt.subplots(figsize=(14, 12))

        sns.heatmap(matriz_sup, annot=True, fmt='.0f', cmap='RdYlGn',
                    center=0, ax=ax, cbar_kws={'label': 'Superioridad'},
                    vmin=-1, vmax=1, linewidths=1, linecolor='gray',
                    annot_kws={'fontsize': 10, 'fontweight': 'bold'})
        ax.set_title('Matriz de Superioridad\n(1=Superior, -1=Inferior, 0=Sin diferencia)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xlabel('Modelo Comparado', fontsize=12, fontweight='bold')
        ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
        ax.tick_params(labelsize=10)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '8_2_significancia_matriz_superioridad.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # Guardar para usar después
        self.df_ranking = df_ranking
        self.df_comparaciones = df_comparaciones

        print(f"\n   ✓ Ranking guardado: Top 3")
        for i, row in df_ranking.head(3).iterrows():
            print(f"      {row['Rank']}. {row['Modelo']} - Score: {row['Score']} "
                  f"(V:{row['Victorias']}, D:{row['Derrotas']}, E:{row['Empates']})")

        print("\n   ✓ 1 figura generada para significancia\n")

    # ========================================================================
    # 9. NUEVO: ANÁLISIS DE IMPACTO DE CARACTERÍSTICAS CON PFI
    # ========================================================================

    def _analisis_impacto_pfi(self):
        """
        Calcula la Permutation Feature Importance (PFI) para cada modelo
        y guarda los resultados.
        """
        all_pfi_scores = {}

        for modelo in self.modelos:
            print(f"   Calculando PFI para el modelo: {modelo}")
            meta_model = self._train_meta_model(modelo)
            
            # Utilizar permutation_importance de sklearn
            # n_repeats es el número de veces que se permuta una característica
            # random_state para reproducibilidad
            # scoring: 'neg_mean_squared_error' si es regresión y queremos minimizar error, 
            #           'r2' si queremos maximizar r2. Para errores, el valor más bajo es mejor, 
            #           así que un scoring que aumente con un peor modelo es más intuitivo para el "impacto".
            #           Si el error es MSE, neg_mean_squared_error es adecuado.
            
            result = permutation_importance(
                meta_model, self.X_processed, self.df[modelo],
                n_repeats=10, random_state=42, n_jobs=-1,
                scoring='neg_mean_squared_error' # Usar neg_mean_squared_error
            )
            
            # Los valores importances_mean ya están en la escala de la métrica de scoring.
            # Como usamos neg_mean_squared_error, un valor más negativo significa mejor rendimiento.
            # La importancia es la disminución en el scoring cuando la característica se permuta.
            # Una mayor disminución (más positiva) significa más importancia.
            
            sorted_idx = result.importances_mean.argsort()[::-1] # Ordenar de mayor a menor importancia

            model_pfi = {}
            for i in sorted_idx:
                feature_name = self.X_processed.columns[i]
                model_pfi[feature_name] = result.importances_mean[i]

            all_pfi_scores[modelo] = model_pfi
            print(f"   ✓ PFI calculado para {modelo}\n")

        self.pfi_results = all_pfi_scores
        print("   ✓ PFI calculado para todos los modelos.")

        # Opcional: Graficar el PFI promedio global
        if self.pfi_results:
            # Calcular el PFI promedio para cada característica en todos los modelos
            avg_pfi = pd.DataFrame(self.pfi_results).mean(axis=1).sort_values(ascending=True)

            fig, ax = plt.subplots(figsize=(12, 8))
            bars = ax.barh(avg_pfi.index, avg_pfi.values, color='steelblue', alpha=0.7)
            ax.set_xlabel('Importancia de Permutación (Disminución en Neg. MSE)', fontweight='bold', fontsize=12)
            ax.set_title('Importancia Global de Características (Promedio PFI)', fontweight='bold', fontsize=14, pad=20)
            ax.grid(True, alpha=0.3, axis='x')

            for i, (bar, val) in enumerate(zip(bars, avg_pfi.values)):
                ax.text(val, i, f' {val:.4f}', va='center', ha='left', fontweight='bold', fontsize=10)

            plt.tight_layout()
            plt.savefig(self.dir_salida / '9_1_pfi_global_promedio.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("   ✓ 1 figura (PFI Global Promedio) generada para impacto de características.\n")


    # ========================================================================
    # 10. NUEVO: ANÁLISIS DE VARIABILIDAD CON PDP e ICE
    # ========================================================================

    def _analisis_variabilidad_pdp_ice(self):
        """
        Genera Partial Dependence Plots (PDP) e Individual Conditional Expectation (ICE) plots
        para el modelo con mejor ranking y sus características más importantes.
        """
        if not hasattr(self, 'df_ranking') or self.df_ranking.empty:
            print("   ⚠️ No se encontró el ranking de modelos. Saltando PDP/ICE.")
            return

        # Seleccionar el modelo con el mejor ranking DM
        best_model_name = self.df_ranking.iloc[0]['Modelo']
        print(f"   Generando PDP/ICE para el modelo con mejor ranking: {best_model_name}")

        meta_model = self._train_meta_model(best_model_name)

        # Obtener las características más importantes de PFI para este modelo
        if best_model_name in self.pfi_results:
            sorted_features = list(self.pfi_results[best_model_name].keys())
            # Tomar las top N características para visualización
            top_features_for_pdp = sorted_features[:min(3, len(sorted_features))]
            
            # Asegurarse de que las características elegidas para PDP existen en X_processed
            # Y que no sean características one-hot-encoded si queremos el nombre original
            
            # Mapear nombres de características de OneHotEncoder a sus originales
            original_feature_names = CARACTERISTICAS_META_MODELO
            processed_feature_names = self.X_processed.columns
            
            features_to_plot = []
            for f in top_features_for_pdp:
                # Si es una característica numérica, se usa directamente
                if f in CARACTERISTICAS_NUMERICAS_META_MODELO:
                    features_to_plot.append(f)
                # Si es una característica categórica (posiblemente one-hot encoded), 
                # Intentar mapearla a su nombre original.
                else:
                    for orig_cat_feat in CARACTERISTICAS_CATEGORICAS_META_MODELO:
                        if f.startswith(orig_cat_feat + '_'): # Es una columna one-hot
                            if orig_cat_feat not in features_to_plot: # Añadir solo la característica original una vez
                                features_to_plot.append(orig_cat_feat)
                            break
                    else: # Si no se encontró como categórica original, añadir tal cual
                        if f not in features_to_plot:
                            features_to_plot.append(f)
            
            # Asegurarse de que no haya duplicados y que sean características válidas para PDP
            features_to_plot = list(dict.fromkeys(features_to_plot)) # Eliminar duplicados manteniendo el orden
            
            # Filtrar a solo características presentes en CARACTERISTICAS_META_MODELO (las originales)
            features_to_plot = [f for f in features_to_plot if f in CARACTERISTICAS_META_MODELO]
            
        else:
            print(f"   ⚠️ No se encontraron resultados PFI para {best_model_name}. Usando características predefinidas.")
            features_to_plot = ['Varianza error', 'Paso', 'Distribución'] # fallback

        # Convertir características categóricas originales a índices para PDP si no están en X_processed
        # PartialDependenceDisplay puede manejar ColumnTransformer si se pasa el pipeline completo.
        # Aquí, usaremos X_processed y el modelo directamente, por lo que las categorías deben ser tratadas.
        # Para características categóricas, PDP puede agrupar automáticamente si se le da el nombre de la columna original
        # y si X es el DataFrame original pre-transformado.
        
        # Para simplificar la visualización con PartialDependenceDisplay y el X_processed
        # Se puede intentar plotear las columnas one-hot si son las más importantes
        # o agruparlas de nuevo al nombre original.
        # Aquí intentaremos plotear las características originales del CARACTERISTICAS_META_MODELO

        pdp_indices = []
        for feat_name in features_to_plot:
            if feat_name in CARACTERISTICAS_NUMERICAS_META_MODELO:
                # Obtener el índice de la columna numérica en X_processed
                try:
                    pdp_indices.append(self.X_processed.columns.get_loc(feat_name))
                except KeyError:
                    # Fallback si el nombre exacto de la columna numérica no está en X_processed por alguna razón
                    pdp_indices.append(CARACTERISTICAS_NUMERICAS_META_MODELO.index(feat_name))

            elif feat_name in CARACTERISTICAS_CATEGORICAS_META_MODELO:
                # Para categóricas, PartialDependenceDisplay puede tomar el nombre de la característica original
                # si se le pasa el ColumnTransformer y el pipeline completo.
                # Como ya preprocesamos X_processed, necesitamos identificar las columnas OHE correspondientes
                # para agruparlas, o pasar el preprocessor al display.
                # Usaremos la estrategia de identificar las columnas OHE.
                
                # En este caso particular, PartialDependenceDisplay es más fácil de usar si se le pasa
                # el objeto preprocessor y el modelo dentro de un pipeline.
                # Adaptaremos la llamada.

                # Si es categórica, necesitamos pasar el índice de las columnas one-hot encoded correspondientes
                # O si estamos graficando solo una, podemos usar su nombre original y dejar que el ColumnTransformer lo maneje
                
                # Una forma más sencilla con ColumnTransformer es pasar el pipeline completo al PartialDependenceDisplay
                # Creamos un pipeline que incluye el preprocesador y el meta_model
                
                feature_indices = []
                for col_name in self.X_processed.columns:
                    if col_name.startswith(feat_name + '_'):
                        feature_indices.append(self.X_processed.columns.get_loc(col_name))
                
                if feature_indices:
                    pdp_indices.append(feature_indices)
                else: # Si no se encontraron columnas OHE, es posible que sea una característica de fallback o un error
                    # Intentar añadirla por su nombre, si PartialDependenceDisplay puede resolverlo
                    if feat_name in self.X_processed.columns:
                        pdp_indices.append(self.X_processed.columns.get_loc(feat_name))
                    
            else: # Fallback para características que no se mapearon bien
                print(f"   Advertencia: No se pudo mapear '{feat_name}' para PDP/ICE.")

        if not pdp_indices:
            print("   ⚠️ No se encontraron características válidas para generar PDP/ICE. Saltando.")
            return
        
        # Crear un pipeline para PartialDependenceDisplay
        full_pipeline = Pipeline(steps=[('preprocessor', self.preprocessor),
                                        ('regressor', meta_model)])

        # Ajustar el pipeline (ya lo hemos hecho con X_processed, pero para PartialDependenceDisplay se necesita el pipeline)
        # Necesitamos el X original y el y original para el pipeline completo
        X_original_for_pipeline = self.df[CARACTERISTICAS_META_MODELO]
        y_original_for_pipeline = self.df[best_model_name]
        full_pipeline.fit(X_original_for_pipeline, y_original_for_pipeline)


        # Generar PDP e ICE plots
        for i, feature_name in enumerate(features_to_plot):
            print(f"      Generando PDP/ICE para '{feature_name}'...")
            fig, ax = plt.subplots(figsize=(10, 6))
            
            # PartialDependenceDisplay puede manejar tanto nombres de características originales
            # como índices de características ya transformadas.
            # Al pasar el pipeline completo y el X_original, puede aplicar el preprocessor internamente.

            try:
                # Si la característica es numérica, podemos usar n_jobs=-1 para acelerar
                # Si es categórica, no se usa grid_resolution tan directamente
                
                # Determinar si es una característica numérica o categórica original
                is_categorical = feature_name in CARACTERISTICAS_CATEGORICAS_META_MODELO

                if is_categorical:
                    # Para categóricas, PartialDependenceDisplay toma los nombres originales
                    # y puede hacer un "grid" sobre las categorías únicas.
                    # El `kind='both'` intenta mostrar ICE y PDP
                    PartialDependenceDisplay.from_estimator(
                        full_pipeline,
                        X_original_for_pipeline, # Pasar el DataFrame original
                        features=[feature_name], # Pasar el nombre de la característica original
                        kind='both', # Muestra PDP e ICE
                        ax=ax,
                        feature_names=CARACTERISTICAS_META_MODELO, # Nombres de las características originales
                        ice_lines_kw={"color": "darkblue", "alpha": 0.2, "linewidth": 0.8},
                        pd_line_kw={"color": "red", "linewidth": 4, "alpha": 0.8, "label": "PDP (Mean)"},
                        n_jobs=-1 # Usar todos los cores
                    )
                else: # Numérica
                     PartialDependenceDisplay.from_estimator(
                        full_pipeline,
                        X_original_for_pipeline,
                        features=[feature_name],
                        kind='both',
                        ax=ax,
                        feature_names=CARACTERISTICAS_META_MODELO,
                        ice_lines_kw={"color": "darkblue", "alpha": 0.2, "linewidth": 0.8},
                        pd_line_kw={"color": "red", "linewidth": 4, "alpha": 0.8, "label": "PDP (Mean)"},
                        grid_resolution=30, # Puntos en la rejilla para numéricas
                        n_jobs=-1
                    )
                
                ax.set_title(f'PDP e ICE para {best_model_name}: {feature_name}', fontweight='bold', fontsize=14, pad=20)
                ax.set_xlabel(feature_name, fontweight='bold', fontsize=12)
                ax.set_ylabel(f'Predicción de Error para {best_model_name}', fontweight='bold', fontsize=12)
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.savefig(self.dir_salida / f'10_1_pdp_ice_{best_model_name}_{feature_name.replace(" ", "_")}.png',
                            dpi=300, bbox_inches='tight')
                plt.close()
                print(f"      ✓ PDP/ICE generado para '{feature_name}'.")

            except Exception as e:
                print(f"      ❌ Error generando PDP/ICE para '{feature_name}': {e}")
                plt.close() # Cerrar figura en caso de error

        print("   ✓ PDP e ICE generados para las características más importantes del mejor modelo.\n")

    # ========================================================================
    # 9. RESUMEN EJECUTIVO (Actualizado para usar PFI)
    # ========================================================================

    def _generar_resumen_ejecutivo(self):
        """Genera un resumen ejecutivo consolidado, ahora con PFI para impacto."""
        print("\n" + "=" * 80)
        print("GENERANDO RESUMEN EJECUTIVO")
        print("=" * 80 + "\n")

        pasos = sorted(self.df['Paso'].unique())
        distribuciones = self.df['Distribución'].unique()
        varianzas = sorted(self.df['Varianza error'].unique())

        # FIGURA 9.2: Comparación multidimensional (sin cambios)
        fig, ax = plt.subplots(figsize=(14, 10))

        top5_modelos = self.df_ranking.head(5)['Modelo'].tolist()

        # Nuevas dimensiones
        caracteristicas_eval = ['Ranking DM', 'Estabilidad por Paso',
                                'Estabilidad por Distribución', 'Estabilidad por Varianza']
        matriz_resumen = []

        for modelo in top5_modelos:
            fila = []

            # 1. Ranking DM (normalizado)
            rank_pos = self.df_ranking[self.df_ranking['Modelo'] == modelo].index[0]
            # Invertir y normalizar: un rank 1 debe ser 100, un rank N debe ser 0.
            rank_norm = 100 * (1 - rank_pos / (len(self.df_ranking) - 1)) if len(self.df_ranking) > 1 else 100
            fila.append(rank_norm)

            # 2. Estabilidad por Paso (inversa de la std)
            stds_paso = [self.df[self.df['Paso'] == p][modelo].std() for p in pasos]
            min_std_paso_all_models = min([np.mean([self.df[self.df['Paso'] == p][m].std() for p in pasos]) for m in self.modelos if len(pasos) > 0]) if len(pasos) > 0 else 0
            max_std_paso_all_models = max([np.mean([self.df[self.df['Paso'] == p][m].std() for p in pasos]) for m in self.modelos if len(pasos) > 0]) if len(pasos) > 0 else 1
            if max_std_paso_all_models == min_std_paso_all_models:
                est_paso = 100
            else:
                est_paso = 100 * (1 - (np.mean(stds_paso) - min_std_paso_all_models) / (max_std_paso_all_models - min_std_paso_all_models))
            fila.append(est_paso)

            # 3. Estabilidad por Distribución
            stds_dist = [self.df[self.df['Distribución'] == d][modelo].std() for d in distribuciones]
            min_std_dist_all_models = min([np.mean([self.df[self.df['Distribución'] == d][m].std() for d in distribuciones]) for m in self.modelos if len(distribuciones) > 0]) if len(distribuciones) > 0 else 0
            max_std_dist_all_models = max([np.mean([self.df[self.df['Distribución'] == d][m].std() for d in distribuciones]) for m in self.modelos if len(distribuciones) > 0]) if len(distribuciones) > 0 else 1
            if max_std_dist_all_models == min_std_dist_all_models:
                est_dist = 100
            else:
                est_dist = 100 * (1 - (np.mean(stds_dist) - min_std_dist_all_models) / (max_std_dist_all_models - min_std_dist_all_models))
            fila.append(est_dist)

            # 4. Estabilidad por Varianza
            stds_var = [self.df[self.df['Varianza error'] == v][modelo].std() for v in varianzas]
            min_std_var_all_models = min([np.mean([self.df[self.df['Varianza error'] == v][m].std() for v in varianzas]) for m in self.modelos if len(varianzas) > 0]) if len(varianzas) > 0 else 0
            max_std_var_all_models = max([np.mean([self.df[self.df['Varianza error'] == v][m].std() for v in varianzas]) for m in self.modelos if len(varianzas) > 0]) if len(varianzas) > 0 else 1
            if max_std_var_all_models == min_std_var_all_models:
                est_var = 100
            else:
                est_var = 100 * (1 - (np.mean(stds_var) - min_std_var_all_models) / (max_std_var_all_models - min_std_var_all_models))
            fila.append(est_var)

            matriz_resumen.append(fila)

        df_heatmap = pd.DataFrame(matriz_resumen, columns=caracteristicas_eval, index=top5_modelos)

        sns.heatmap(df_heatmap, annot=True, fmt='.1f', cmap='RdYlGn',
                    ax=ax, cbar_kws={'label': 'Score Normalizado (0-100)'},
                    linewidths=2, linecolor='white', vmin=0, vmax=100,
                    annot_kws={'fontsize': 11, 'fontweight': 'bold'})
        ax.set_title('Perfil Multidimensional - Top 5 Modelos',
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xlabel('Dimensión de Evaluación', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo', fontweight='bold', fontsize=12)
        ax.tick_params(labelsize=11)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '9_2_resumen_perfil_multidimensional.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        # FIGURA 9.3: Impacto de características - AHORA CON PFI
        fig, ax = plt.subplots(figsize=(12, 8))

        if self.pfi_results:
            # Calcular el PFI promedio para cada característica en todos los modelos
            # ya se calculó en _analisis_impacto_pfi, aquí lo recuperamos
            avg_pfi_series = pd.DataFrame(self.pfi_results).mean(axis=1)

            # Asegurar que el total sea 100% para este gráfico
            total_pfi = avg_pfi_series.sum()
            if total_pfi > 0:
                impactos_norm = (avg_pfi_series / total_pfi) * 100
            else:
                impactos_norm = pd.Series(0.0, index=avg_pfi_series.index)


            # Ordenar por impacto
            impactos_sorted = impactos_norm.sort_values(ascending=True) # Ascending para barh

            nombres_imp = impactos_sorted.index
            valores_imp = impactos_sorted.values

            colors_imp = plt.cm.Reds(np.linspace(0.3, 0.9, len(impactos_sorted)))
            bars = ax.barh(nombres_imp, valores_imp, color=colors_imp, alpha=0.8,
                           edgecolor='black', linewidth=1.5)
            ax.set_xlabel('Impacto Normalizado PFI (%)', fontweight='bold', fontsize=12)
            ax.set_title('Impacto de Características en el ECRPS (PFI Promedio)\n(Total = 100%)',
                         fontweight='bold', fontsize=14, pad=20)
            ax.grid(True, alpha=0.3, axis='x')
            ax.set_xlim(0, max(valores_imp) * 1.15)

            for i, (bar, val) in enumerate(zip(bars, valores_imp)):
                ax.text(val + 1, i, f' {val:.1f}%', va='center', fontweight='bold', fontsize=11)

            # Agregar suma total
            ax.text(0.98, 0.02, f'Suma Total: {sum(valores_imp):.1f}%',
                    transform=ax.transAxes, fontsize=11, fontweight='bold',
                    ha='right', va='bottom',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
        else:
            ax.text(0.5, 0.5, "No hay datos de PFI para mostrar.",
                    horizontalalignment='center', verticalalignment='center',
                    transform=ax.transAxes, fontsize=12, color='red')
            ax.set_title('Impacto de Características en el ECRPS (PFI Promedio)\n(Total = 100%)',
                         fontweight='bold', fontsize=14, pad=20)


        plt.tight_layout()
        plt.savefig(self.dir_salida / '9_3_resumen_impacto_caracteristicas.png',
                    dpi=300, bbox_inches='tight')
        plt.close()

        print("   ✓ 2 figuras generadas para resumen ejecutivo")
        print()


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def main():
    """Función principal de ejecución"""
    print("\n" + "█" * 80)
    print("█" + " " * 78 + "█")
    print("█" + " " * 10 + "ANÁLISIS COMPLETO DE BASE DE DATOS - VERSIÓN MEJORADA" + " " * 9 + "█")
    print("█" + " " * 78 + "█")
    print("█" * 80 + "\n")

    try:
        # Crear instancia del analizador
        analizador = AnalizadorBaseCompleta(RUTA_DATOS)

        # Ejecutar análisis completo
        analizador.ejecutar_analisis_completo()

        print("\n" + "█" * 80)
        print("█" + " " * 78 + "█")
        print("█" + " " * 20 + "✅ ANÁLISIS COMPLETADO EXITOSAMENTE" + " " * 23 + "█")
        print("█" + " " * 78 + "█")
        print("█" * 80 + "\n")

        total_figures = 15 # Figuras originales
        if hasattr(analizador, 'pfi_results') and analizador.pfi_results:
            total_figures += 1 # PFI global promedio
        if hasattr(analizador, 'df_ranking') and not analizador.df_ranking.empty:
            best_model = analizador.df_ranking.iloc[0]['Modelo']
            if best_model in analizador.pfi_results:
                num_pdp_ice_plots = min(3, len(list(analizador.pfi_results[best_model].keys())))
                total_figures += num_pdp_ice_plots
        
        print(f"📊 TOTAL DE FIGURAS GENERADAS: {total_figures} imágenes PNG")
        print("\n📁 ESTRUCTURA DE RESULTADOS:")
        print(f"   {DIR_SALIDA}/")
        print("   ├── 1.1: Estacionariedad - Comparativo")
        print("   ├── 1.2: Estacionariedad - Cambio Relativo")
        print("   ├── 2.1: Linealidad - Comparativo")
        print("   ├── 2.2: Linealidad - Cambio Relativo")
        print("   ├── 3.2: Modelo Generador - Z-Score")
        print("   ├── 3.3: Modelo Generador - Variabilidad")
        print("   ├── 4.1: Distribución - Heatmap ECRPS")
        print("   ├── 4.2: Distribución - Heatmap Variabilidad")
        print("   ├── 5.1: Varianza - Tendencias")
        print("   ├── 5.2: Varianza - Tasa de Crecimiento")
        print("   ├── 6.1: Horizonte - Evolución")
        print("   ├── 6.2: Horizonte - Tasa de Deterioro")
        print("   ├── 7.2: Robustez - Coeficiente de Variación")
        print("   ├── 8.2: Significancia - Matriz de Superioridad")
        print("   ├── 9.1: PFI Global Promedio (NUEVO)")
        print("   ├── 9.2: Resumen - Perfil Multidimensional")
        print("   ├── 9.3: Resumen - Impacto de Características (PFI Normalizado) (MODIFICADO)")
        print("   └── 10.1: PDP e ICE para el Mejor Modelo (x3) (NUEVO)")
        print("\n" + "=" * 80 + "\n")

    except FileNotFoundError:
        print(f"\n❌ ERROR: No se encontró el archivo {RUTA_DATOS}")
        print("   Por favor, verifica que el archivo existe y la ruta es correcta.\n")
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {str(e)}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


████████████████████████████████████████████████████████████████████████████████
█                                                                              █
█          ANÁLISIS COMPLETO DE BASE DE DATOS - VERSIÓN MEJORADA         █
█                                                                              █
████████████████████████████████████████████████████████████████████████████████


INICIANDO ANÁLISIS COMPLETO DE BASE DE DATOS - VERSIÓN MEJORADA

✓ Características extraídas:
  - Estacionariedad: ['Estacionario' 'No Estacionario']
  - Linealidad: ['Lineal' 'No Lineal']
  - Tipos de Modelo: ['AR(1)' 'AR(2)' 'MA(1)' 'MA(2)' 'ARMA(1,1)' 'ARMA(2,2)' 'ARIMA(0,1,0)'
 'ARIMA(1,1,0)' 'ARIMA(2,1,0)' 'ARIMA(0,1,1)' 'ARIMA(0,1,2)'
 'ARIMA(1,1,1)' 'ARIMA(2,1,2)' 'SETAR(2,1)' 'TAR(2,1)' 'EXPAR(2,1)'
 'BILINEAR(1)' 'SETAR(2,2)' 'TAR(2,2)' 'SETAR(2,3)']
  - Distribuciones: ['normal' 'uniform' 'exponential' 't-student' 'mixture']
  - Varianzas: [np.float64(0.2), np.float64(0.5), np.floa

# Preguntas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from itertools import combinations
import math
import warnings

warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Datos/datos_combinados.xlsx"
DIR_SALIDA = "./resultados_preguntas_profundizacion"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

COLORES_MODELOS = {
    'AREPD': '#e41a1c',
    'AV-MCPS': '#377eb8',
    'Block Bootstrapping': '#4daf4a',
    'DeepAR': '#984ea3',
    'EnCQR-LSTM': '#ff7f00',
    'LSPM': '#ffff33',
    'LSPMW': '#a65628',
    'MCPS': '#f781bf',
    'Sieve Bootstrap': '#999999'
}


# ============================================================================
# CLASE PRINCIPAL DE ANÁLISIS - PREGUNTAS DE PROFUNDIZACIÓN
# ============================================================================

class AnalizadorPreguntasProfundizacion:
    """Análisis específico para responder preguntas de profundización"""

    def __init__(self, ruta_datos):
        """Inicializa el analizador"""
        print("\n" + "=" * 80)
        print("ANÁLISIS DE PREGUNTAS DE PROFUNDIZACIÓN")
        print("=" * 80 + "\n")

        self.df = pd.read_excel(ruta_datos)
        self.modelos = MODELOS
        self.COLORES_MODELOS = COLORES_MODELOS
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)

        # Extraer características del escenario
        self._extraer_caracteristicas()

        print(f"✓ Datos cargados: {self.df.shape[0]} filas, {self.df.shape[1]} columnas")
        print(f"✓ Modelos a analizar: {len(self.modelos)}")
        print(f"✓ Directorio de salida: {self.dir_salida}")
        print("\n" + "=" * 80 + "\n")

    def _extraer_caracteristicas(self):
        """Extrae características individuales del escenario"""
        self.df['Estacionario'] = self.df['Escenario'].apply(
            lambda x: 'Estacionario' if 'Estacionario' in x and 'No_Estacionario' not in x else 'No Estacionario'
        )

        self.df['Lineal'] = self.df['Escenario'].apply(
            lambda x: 'Lineal' if 'Lineal' in x and 'No_Lineal' not in x else 'No Lineal'
        )

    def ejecutar_analisis_completo(self):
        """Ejecuta todos los análisis para las preguntas"""
        print("\n" + "🔬" * 40 + "\n")

        # Pregunta 1: Punto de quiebre de AREPD
        print("1️⃣  Pregunta 1: Punto de quiebre de AREPD...")
        self._pregunta_1_punto_quiebre_arepd()

        # Pregunta 2: Robustez de Block Bootstrapping vs Sieve Bootstrap
        print("\n2️⃣  Pregunta 2: Zona de dominio Block Bootstrapping...")
        self._pregunta_2_zona_dominio_bb()

        # Pregunta 3: Deterioro acelerado AV-MCPS
        print("\n3️⃣  Pregunta 3: Deterioro de AV-MCPS por horizonte...")
        self._pregunta_3_deterioro_av_mcps()

        # Pregunta 4: Penalización Normal multiplicativa
        print("\n4️⃣  Pregunta 4: Efecto multiplicativo distribución Normal...")
        self._pregunta_4_penalizacion_normal()

        # Pregunta 5: Frontera de colapso Deep Learning
        print("\n5️⃣  Pregunta 5: Frontera de colapso Deep Learning...")
        self._pregunta_5_frontera_dl()

        # Pregunta 6: Consistencia "Mejor Modelo"
        print("\n6️⃣  Pregunta 6: Validación de 'Mejor Modelo'...")
        self._pregunta_6_consistencia_mejor_modelo()

        # Pregunta 7: Análisis de segunda derivada
        print("\n7️⃣  Pregunta 7: Aceleración del deterioro...")
        self._pregunta_7_segunda_derivada()

        # Pregunta 8: Interacción No Linealidad × Varianza
        print("\n8️⃣  Pregunta 8: Colapso LSPM con varianza alta...")
        self._pregunta_8_interaccion_nolineal_varianza()

        # Pregunta 9: Mapa de decisión operacional
        print("\n9️⃣  Pregunta 9: Mapa de decisión operacional...")
        self._pregunta_9_mapa_decision()

        print("\n" + "=" * 80)
        print("✅ ANÁLISIS DE PREGUNTAS COMPLETO")
        print(f"📁 Resultados guardados en: {self.dir_salida}")
        print("=" * 80 + "\n")

    # ========================================================================
    # PREGUNTA 1: PUNTO DE QUIEBRE DE AREPD
    # ========================================================================

    def _pregunta_1_punto_quiebre_arepd(self):
        """
        ¿Existe un punto de quiebre en la varianza del error donde AREPD se deteriora?
        ¿Este punto es consistente entre todas las distribuciones presentes?
        """
        
        # --- MODIFICACIÓN: Detectar todas las distribuciones únicas automáticamente ---
        distribuciones = self.df['Distribución'].unique()
        varianzas = sorted(self.df['Varianza error'].unique())
        
        # Diccionario para almacenar los resultados de cada distribución
        resultados_por_dist = {dist: {'arepd': [], 'otros_robustos': []} for dist in distribuciones}
        
        # Modelos robustos para comparación
        otros_modelos = ['Block Bootstrapping', 'Sieve Bootstrap', 'LSPM']
        
        # Calcular el rendimiento promedio para cada distribución y varianza
        for dist in distribuciones:
            df_dist = self.df[self.df['Distribución'] == dist]
            for var in varianzas:
                df_var = df_dist[df_dist['Varianza error'] == var]
                
                # Rendimiento de AREPD
                resultados_por_dist[dist]['arepd'].append(df_var['AREPD'].mean())
                
                # Rendimiento promedio de otros modelos robustos
                resultados_por_dist[dist]['otros_robustos'].append(df_var[otros_modelos].mean().mean())

        # --- MODIFICACIÓN: Gráfico dinámico para N distribuciones ---
        # FIGURA 1.1: Evolución de AREPD vs modelos robustos por distribución
        n_dist = len(distribuciones)
        n_cols = 2
        n_rows = math.ceil(n_dist / n_cols)
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(8 * n_cols, 6 * n_rows), squeeze=False)
        axes = axes.flatten()

        for idx, dist in enumerate(distribuciones):
            ax = axes[idx]
            arepd_perf = resultados_por_dist[dist]['arepd']
            otros_perf = resultados_por_dist[dist]['otros_robustos']
            
            ax.plot(varianzas, arepd_perf, 'o-', label='AREPD', 
                    color=self.COLORES_MODELOS.get('AREPD', 'blue'), linewidth=3, markersize=8)
            ax.plot(varianzas, otros_perf, 's--', label='Promedio Modelos Robustos',
                    color='green', linewidth=2, markersize=7, alpha=0.7)
            
            ax.set_xlabel('Varianza del Error', fontweight='bold', fontsize=12)
            ax.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
            ax.set_title(f'Distribución {dist.capitalize()}: Punto de Quiebre AREPD', 
                         fontweight='bold', fontsize=13)
            ax.legend(fontsize=11)
            ax.grid(True, alpha=0.3)
        
        # Ocultar ejes no utilizados si el número de distribuciones es impar
        for idx in range(n_dist, len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_1_punto_quiebre_arepd_comparativo.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # --- MODIFICACIÓN: Gráfico de comparación de tasas con N distribuciones ---
        # FIGURA 1.2: Tasa de deterioro incremental
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Usar un ciclo de colores de Matplotlib para distinguir las líneas
        colors = plt.cm.viridis(np.linspace(0, 1, n_dist))
        var_medias = [(varianzas[i] + varianzas[i+1]) / 2 for i in range(len(varianzas) - 1)]
        
        puntos_quiebre = {}

        for idx, dist in enumerate(distribuciones):
            arepd_perf = resultados_por_dist[dist]['arepd']
            
            # Calcular tasas de cambio (derivada numérica)
            if len(varianzas) > 1 and np.diff(varianzas).any():
                tasas = np.diff(arepd_perf) / np.diff(varianzas)
            else:
                tasas = []

            if len(tasas) > 0:
                ax.plot(var_medias, tasas, 'o-', label=f'{dist.capitalize()}', 
                       color=colors[idx], linewidth=2.5, markersize=8)
                
                # Identificar y almacenar el punto de máxima aceleración del deterioro
                max_accel_idx = np.argmax(tasas)
                puntos_quiebre[dist] = var_medias[max_accel_idx]
            else:
                puntos_quiebre[dist] = None


        ax.axhline(y=0, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
        ax.set_xlabel('Varianza del Error (punto medio)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Tasa de Deterioro (ΔECRPS/ΔVarianza)', fontweight='bold', fontsize=12)
        ax.set_title('AREPD: Aceleración del Deterioro por Distribución\n(Mayor pendiente = Colapso más rápido)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=12, loc='upper left', title='Distribuciones')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_2_tasa_deterioro_arepd.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # --- MODIFICACIÓN: Imprimir resultados para todas las distribuciones ---
        print("--- Análisis de Punto de Quiebre para AREPD ---")
        for dist, umbral in puntos_quiebre.items():
            if umbral is not None:
                print(f"   ✓ Punto de quiebre AREPD ({dist.capitalize()}): Varianza ≈ {umbral:.3f}")
            else:
                print(f"   ! No se pudo calcular el punto de quiebre para {dist.capitalize()}.")
        
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 2: ZONA DE DOMINIO BLOCK BOOTSTRAPPING
    # ========================================================================

    def _pregunta_2_zona_dominio_bb(self):
        """
        ¿En qué condiciones EXACTAS Block Bootstrapping supera a Sieve Bootstrap?
        """
        
        # Crear DataFrame de comparación directa
        df_comp = self.df.copy()
        df_comp['BB_mejor'] = df_comp['Block Bootstrapping'] < df_comp['Sieve Bootstrap']
        df_comp['Diferencia'] = df_comp['Sieve Bootstrap'] - df_comp['Block Bootstrapping']
        
        # FIGURA 2.1: Mapa de calor de superioridad
        # >>> INICIO DE MODIFICACIÓN 1 <<<
        fig, axes = plt.subplots(1, 3, figsize=(24, 7))
        axes = axes.flatten()
        
        escenarios_principales = [
            ('Estacionario', 'Lineal'),
            ('Estacionario', 'No Lineal'),
            ('No Estacionario', 'Lineal')
        ]
        # >>> FIN DE MODIFICACIÓN 1 <<<
        
        for idx, (est, lin) in enumerate(escenarios_principales):
            ax = axes[idx]
            df_esc = df_comp[(df_comp['Estacionario'] == est) & (df_comp['Lineal'] == lin)]
            
            # Crear matriz de diferencias
            pivot = df_esc.pivot_table(
                values='Diferencia',
                index='Distribución',
                columns='Varianza error',
                aggfunc='mean'
            )
            
            sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn', center=0,
                       ax=ax, cbar_kws={'label': 'SB - BB (>0 = BB mejor)'},
                       linewidths=1, linecolor='gray', vmin=-0.02, vmax=0.02)
            ax.set_title(f'{est} + {lin}', fontweight='bold', fontsize=12)
            ax.set_xlabel('Varianza Error', fontweight='bold')
            ax.set_ylabel('Distribución', fontweight='bold')
        
        plt.suptitle('Zona de Dominio: Block Bootstrapping vs Sieve Bootstrap\n(Verde = BB domina, Rojo = SB domina)',
                    fontweight='bold', fontsize=16, y=1.03)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P2_1_zona_dominio_bb_heatmap.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 2.2: Frecuencia de dominio por condiciones
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Calcular % de casos donde BB es mejor
        resultados_dominio = []
        # >>> INICIO DE MODIFICACIÓN 1 (cont.) <<<
        # Se usan los mismos 3 escenarios definidos para la figura 2.1
        for est, lin in escenarios_principales:
            df_esc = df_comp[(df_comp['Estacionario'] == est) & (df_comp['Lineal'] == lin)]
            pct_bb_mejor = (df_esc['BB_mejor'].sum() / len(df_esc) * 100) if len(df_esc) > 0 else 0
            resultados_dominio.append({
                'Escenario': f'{est[:3]}+{lin[:3]}',
                'Completo': f'{est} + {lin}',
                'Pct_BB_Mejor': pct_bb_mejor
            })
        # >>> FIN DE MODIFICACIÓN 1 (cont.) <<<
        
        df_dominio = pd.DataFrame(resultados_dominio).sort_values('Pct_BB_Mejor', ascending=False)
        
        colors = ['green' if x > 50 else 'red' for x in df_dominio['Pct_BB_Mejor']]
        bars = ax.barh(df_dominio['Escenario'], df_dominio['Pct_BB_Mejor'],
                      color=colors, alpha=0.7, edgecolor='black', linewidth=2)
        ax.axvline(50, color='black', linestyle='--', linewidth=2, label='50% (Equilibrio)')
        ax.set_xlabel('% de casos donde BB supera a SB', fontweight='bold', fontsize=12)
        ax.set_title('Frecuencia de Dominio de Block Bootstrapping\n(>50% = BB generalmente mejor)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='x')
        ax.set_xlim(0, 100)
        
        for i, (bar, val) in enumerate(zip(bars, df_dominio['Pct_BB_Mejor'])):
            ax.text(val + 2, i, f'{val:.1f}%', va='center', fontweight='bold', fontsize=11)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P2_2_frecuencia_dominio_bb.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"   ✓ BB domina en: {df_comp['BB_mejor'].sum()} / {len(df_comp)} casos ({df_comp['BB_mejor'].sum()/len(df_comp)*100:.1f}%)")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 3: DETERIORO AV-MCPS POR HORIZONTE
    # ========================================================================

    def _pregunta_3_deterioro_av_mcps(self):
        """
        ¿El deterioro de AV-MCPS es lineal, cuadrático o exponencial?
        ¿Cambia según el nivel de varianza?
        """
        
        pasos = sorted(self.df['Paso'].unique())
        varianzas = sorted(self.df['Varianza error'].unique())
        
        # FIGURA 3.1: Ajuste de curvas de deterioro
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        # Seleccionar niveles de varianza representativos
        if len(varianzas) >= 4:
            var_seleccionadas = [varianzas[0], varianzas[len(varianzas)//3], 
                                varianzas[2*len(varianzas)//3], varianzas[-1]]
        else:
            var_seleccionadas = varianzas
        
        modelos_comparacion = ['AV-MCPS', 'LSPM', 'Block Bootstrapping']
        
        for idx, var in enumerate(var_seleccionadas[:4]):
            ax = axes[idx]
            df_var = self.df[self.df['Varianza error'] == var]
            
            for modelo in modelos_comparacion:
                valores = [df_var[df_var['Paso'] == p][modelo].mean() for p in pasos]
                ax.plot(pasos, valores, 'o-', label=modelo, 
                       linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo])
            
            ax.set_xlabel('Horizonte (Paso)', fontweight='bold', fontsize=11)
            ax.set_ylabel('ECRPS', fontweight='bold', fontsize=11)
            ax.set_title(f'Varianza = {var:.3f}', fontweight='bold', fontsize=12)
            ax.legend(fontsize=10)
            ax.grid(True, alpha=0.3)
        
        plt.suptitle('Evolución del Deterioro por Horizonte: AV-MCPS vs Modelos Estables',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P3_1_deterioro_av_mcps_curvas.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 3.2: Análisis de tipo de crecimiento
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Calcular R² para diferentes tipos de ajuste
        tipos_ajuste = []
        
        for var in varianzas:
            df_var = self.df[self.df['Varianza error'] == var]
            valores_av = [df_var[df_var['Paso'] == p]['AV-MCPS'].mean() for p in pasos]
            
            x = np.array(pasos)
            y = np.array(valores_av)
            
            # Ajuste lineal
            p_lin = np.polyfit(x, y, 1)
            y_lin = np.polyval(p_lin, x)
            r2_lin = 1 - (np.sum((y - y_lin)**2) / np.sum((y - np.mean(y))**2))
            
            # Ajuste cuadrático
            p_quad = np.polyfit(x, y, 2)
            y_quad = np.polyval(p_quad, x)
            r2_quad = 1 - (np.sum((y - y_quad)**2) / np.sum((y - np.mean(y))**2))
            
            # Ajuste exponencial (logarítmico)
            try:
                z = np.polyfit(x, np.log(y + 1e-10), 1)
                y_exp = np.exp(np.polyval(z, x))
                r2_exp = 1 - (np.sum((y - y_exp)**2) / np.sum((y - np.mean(y))**2))
            except:
                r2_exp = 0
            
            mejor_ajuste = max([('Lineal', r2_lin), ('Cuadrático', r2_quad), ('Exponencial', r2_exp)], 
                              key=lambda x: x[1])
            
            tipos_ajuste.append({
                'Varianza': var,
                'R2_Lineal': r2_lin,
                'R2_Cuadratico': r2_quad,
                'R2_Exponencial': r2_exp,
                'Mejor': mejor_ajuste[0]
            })
        
        df_ajustes = pd.DataFrame(tipos_ajuste)
        
        x_pos = np.arange(len(varianzas))
        width = 0.25
        
        ax.bar(x_pos - width, df_ajustes['R2_Lineal'], width, label='Lineal', alpha=0.8)
        ax.bar(x_pos, df_ajustes['R2_Cuadratico'], width, label='Cuadrático', alpha=0.8)
        ax.bar(x_pos + width, df_ajustes['R2_Exponencial'], width, label='Exponencial', alpha=0.8)
        
        ax.set_xlabel('Varianza del Error', fontweight='bold', fontsize=12)
        ax.set_ylabel('R² (Bondad de Ajuste)', fontweight='bold', fontsize=12)
        ax.set_title('AV-MCPS: Tipo de Deterioro por Nivel de Varianza\n(R² más alto = Mejor ajuste)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.set_xticks(x_pos)
        ax.set_xticklabels([f'{v:.3f}' for v in varianzas], rotation=45)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_ylim(0, 1.1)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P3_2_tipo_deterioro_av_mcps.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"   ✓ Tipo de deterioro predominante: {df_ajustes['Mejor'].mode()[0]}")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 4: PENALIZACIÓN NORMAL MULTIPLICATIVA
    # ========================================================================
    def _pregunta_4_penalizacion_normal(self):
        """
        ¿La penalización de la distribución Normal es aditiva o multiplicativa 
        con la no-estacionariedad?
        """
        
        # --- CORRECCIÓN: Se definen los nombres reales de los escenarios del Excel ---
        # Se usan estos nombres para filtrar el DataFrame correctamente.
        escenario_estacionario = 'Estacionario_Lineal'
        escenario_no_estacionario = 'No_Estacionario_Lineal'
        
        # Modelos a analizar
        modelos_analisis = ['DeepAR', 'MCPS', 'LSPM', 'Block Bootstrapping']
        
        # FIGURA 4.1: Efecto aditivo vs multiplicativo
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        resultados_interaccion = []
        
        for idx, modelo in enumerate(modelos_analisis):
            ax = axes[idx]
            
            # Calcular penalización por escenario
            penalizaciones = []
            
            # --- CORRECCIÓN: Se itera sobre los escenarios correctos y se usan etiquetas limpias para los gráficos ---
            escenarios_a_comparar = {
                'Estacionario': escenario_estacionario,
                'No Estacionario': escenario_no_estacionario
            }
            
            for etiqueta, nombre_escenario in escenarios_a_comparar.items():
                
                # --- CORRECCIÓN: Filtrar por la columna 'Escenario' con los nombres correctos ---
                df_escenario = self.df[self.df['Escenario'] == nombre_escenario]
                
                # --- CORRECCIÓN: Usar 'normal' y 'mixture' en minúsculas ---
                rend_normal = df_escenario[df_escenario['Distribución'] == 'normal'][modelo].mean()
                rend_mixture = df_escenario[df_escenario['Distribución'] == 'mixture'][modelo].mean()
                
                # Calcular el deterioro porcentual. Se añade una guarda contra la división por cero.
                if rend_mixture != 0:
                    deterioro = ((rend_normal - rend_mixture) / rend_mixture) * 100
                else:
                    deterioro = 0.0 # O float('inf') si se prefiere, pero 0 es más seguro para graficar
                
                penalizaciones.append({
                    'Estacionariedad': etiqueta, # Usar la etiqueta limpia para el gráfico
                    'Deterioro_pct': deterioro
                })
            
            df_pen = pd.DataFrame(penalizaciones)
            
            # Calcular razón de efectos
            det_estacionario = df_pen[df_pen['Estacionariedad'] == 'Estacionario']['Deterioro_pct'].values[0]
            det_no_estacionario = df_pen[df_pen['Estacionariedad'] == 'No Estacionario']['Deterioro_pct'].values[0]
            
            # Evitar división por cero si el deterioro en el caso estacionario fue nulo
            razon = det_no_estacionario / det_estacionario if det_estacionario != 0 else 0
            es_multiplicativo = razon > 1.5  # Si el efecto es >50% mayor, se considera multiplicativo
            
            resultados_interaccion.append({
                'Modelo': modelo,
                'Razon': razon,
                'Tipo': 'Multiplicativo' if es_multiplicativo else 'Aditivo'
            })
            
            # Visualización
            colors = ['lightblue', 'coral']
            bars = ax.bar(df_pen['Estacionariedad'], df_pen['Deterioro_pct'], 
                         color=colors, alpha=0.7, edgecolor='black', linewidth=2)
            ax.set_ylabel('Deterioro por Dist. Normal (%)', fontweight='bold', fontsize=11)
            ax.set_title(f'{modelo}\nRazón: {razon:.2f}x ({("MULTIPLICATIVO" if es_multiplicativo else "ADITIVO")})',
                        fontweight='bold', fontsize=12)
            ax.grid(True, alpha=0.3, axis='y')
            ax.axhline(0, color='black', linestyle='-', linewidth=1)
            
            for bar, val in zip(bars, df_pen['Deterioro_pct']):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width() / 2., height + 1,
                       f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)
        
        plt.suptitle('Interacción: Distribución Normal × No-Estacionariedad\n(Razón > 1.5 = Efecto Multiplicativo)',
                    fontweight='bold', fontsize=14, y=0.995)
        
        # >>> INICIO DE MODIFICACIÓN 2 <<<
        # Se añade más padding para evitar que los elementos se superpongan
        plt.tight_layout(pad=3.0)
        # >>> FIN DE MODIFICACIÓN 2 <<<
        
        plt.savefig(self.dir_salida / 'P4_1_penalizacion_normal_interaccion.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 4.2: Resumen de tipos de interacción
        fig, ax = plt.subplots(figsize=(12, 8))
        
        df_interaccion = pd.DataFrame(resultados_interaccion).sort_values('Razon', ascending=False)
        
        colors_tipo = ['red' if x == 'Multiplicativo' else 'green' for x in df_interaccion['Tipo']]
        bars = ax.barh(df_interaccion['Modelo'], df_interaccion['Razon'],
                      color=colors_tipo, alpha=0.7, edgecolor='black', linewidth=2)
        ax.axvline(1.5, color='black', linestyle='--', linewidth=2, label='Umbral Multiplicativo (1.5x)')
        ax.axvline(1.0, color='gray', linestyle=':', linewidth=1.5, alpha=0.5)
        ax.set_xlabel('Razón de Efectos (No-Est / Est)', fontweight='bold', fontsize=12)
        ax.set_title('Clasificación del Tipo de Interacción por Modelo\n(Rojo = Multiplicativo, Verde = Aditivo)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val, tipo) in enumerate(zip(bars, df_interaccion['Razon'], df_interaccion['Tipo'])):
            ax.text(val + 0.05, i, f'{val:.2f}x ({tipo})', 
                   va='center', fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P4_2_clasificacion_interaccion.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        modelos_multiplicativos = df_interaccion[df_interaccion['Tipo'] == 'Multiplicativo']['Modelo'].tolist()
        print(f"   ✓ Modelos con efecto multiplicativo: {modelos_multiplicativos}")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 5: FRONTERA DE COLAPSO DEEP LEARNING
    # ========================================================================

    def _pregunta_5_frontera_dl(self):
        """
        ¿Existe una 'frontera de colapso' para DeepAR y EnCQR-LSTM donde su 
        rendimiento cae por debajo de métodos estadísticos simples?
        """
        
        modelos_dl = ['DeepAR', 'EnCQR-LSTM']
        modelos_estadisticos = ['Block Bootstrapping', 'Sieve Bootstrap', 'LSPM']
        
        varianzas = sorted(self.df['Varianza error'].unique())
        
        # FIGURA 5.1: Evolución comparativa con varianza
        fig, axes = plt.subplots(1, 2, figsize=(16, 7))
        
        for idx, modelo_dl in enumerate(modelos_dl):
            ax = axes[idx]
            
            # Calcular promedios por varianza
            dl_vals = [self.df[self.df['Varianza error'] == v][modelo_dl].mean() for v in varianzas]
            
            # Promedio de modelos estadísticos
            est_vals = []
            for v in varianzas:
                df_v = self.df[self.df['Varianza error'] == v]
                est_vals.append(df_v[modelos_estadisticos].mean().mean())
            
            # Plotear
            ax.plot(varianzas, dl_vals, 'o-', label=f'{modelo_dl} (DL)',
                   color=COLORES_MODELOS[modelo_dl], linewidth=3, markersize=10)
            ax.plot(varianzas, est_vals, 's--', label='Promedio Estadísticos',
                   color='green', linewidth=2.5, markersize=8, alpha=0.7)
            
            # Identificar punto de cruce
            diferencias = np.array(dl_vals) - np.array(est_vals)
            if np.any(diferencias > 0):
                idx_cruce_candidatos = np.where(diferencias > 0)[0]
                if len(idx_cruce_candidatos) > 0:
                    idx_cruce = idx_cruce_candidatos[0]
                    var_cruce = varianzas[idx_cruce]
                    ax.axvline(var_cruce, color='red', linestyle=':', linewidth=2, alpha=0.7)
                    ax.annotate(f'Frontera de colapso\nVarianza ≈ {var_cruce:.3f}',
                               xy=(var_cruce, dl_vals[idx_cruce]),
                               xytext=(20, -30), textcoords='offset points',
                               bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8),
                               arrowprops=dict(arrowstyle='->', color='red', lw=2),
                               fontsize=10, fontweight='bold')
            
            ax.set_xlabel('Varianza del Error', fontweight='bold', fontsize=12)
            ax.set_ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
            ax.set_title(f'Frontera de Colapso: {modelo_dl}',
                        fontweight='bold', fontsize=13)
            ax.legend(fontsize=11)
            ax.grid(True, alpha=0.3)
        
        plt.suptitle('Identificación de Frontera donde Deep Learning < Métodos Estadísticos',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P5_1_frontera_colapso_dl.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 5.2: Brecha de rendimiento por escenario
        # >>> INICIO DE MODIFICACIÓN 3 <<<
        fig, ax = plt.subplots(figsize=(16, 9)) # Ajustar tamaño para mejor visualización
        
        escenarios_unicos = self.df['Escenario'].unique()
        
        brechas = []
        for esc in escenarios_unicos:
            df_esc = self.df[self.df['Escenario'] == esc]
            
            for modelo_dl in modelos_dl:
                dl_mean = df_esc[modelo_dl].mean()
                est_mean = df_esc[modelos_estadisticos].mean().mean()
                
                brecha_pct = ((dl_mean - est_mean) / est_mean) * 100
                
                brechas.append({
                    'Escenario': esc.replace("_", " "), # Nombres más legibles
                    'Modelo_DL': modelo_dl,
                    'Brecha_pct': brecha_pct
                })
        
        df_brechas = pd.DataFrame(brechas)
        
        # Pivot para visualización
        pivot_brechas = df_brechas.pivot(index='Escenario', columns='Modelo_DL', values='Brecha_pct')
        pivot_brechas = pivot_brechas.sort_values(by='DeepAR', ascending=False)
        
        x = np.arange(len(pivot_brechas))
        width = 0.35
        
        # Se cambia barh por bar para un gráfico vertical
        ax.bar(x - width/2, pivot_brechas['DeepAR'], width, 
               label='DeepAR', color=COLORES_MODELOS['DeepAR'], alpha=0.8, edgecolor='black')
        ax.bar(x + width/2, pivot_brechas['EnCQR-LSTM'], width,
               label='EnCQR-LSTM', color=COLORES_MODELOS['EnCQR-LSTM'], alpha=0.8, edgecolor='black')
        
        # Se cambia axvline por axhline
        ax.axhline(0, color='black', linestyle='-', linewidth=2)
        ax.axhline(50, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Umbral Crítico (+50%)')
        
        # Se configuran los ejes para el gráfico vertical
        ax.set_xticks(x)
        ax.set_xticklabels(pivot_brechas.index, rotation=45, ha='right', fontsize=9)
        ax.set_ylabel('Brecha vs Métodos Estadísticos (%)', fontweight='bold', fontsize=12)
        ax.set_xlabel('Escenario', fontweight='bold', fontsize=12)
        ax.set_title('Escenarios donde Deep Learning Colapsa\n(>0% = DL peor que Estadísticos)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=11, loc='upper right')
        ax.grid(True, alpha=0.3, axis='y') # Grid en el eje Y
        # >>> FIN DE MODIFICACIÓN 3 <<<
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P5_2_brecha_dl_por_escenario.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"   ✓ Escenarios críticos identificados: {len(pivot_brechas[pivot_brechas['DeepAR'] > 50])}")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 6: CONSISTENCIA "MEJOR MODELO"
    # ========================================================================

    def _pregunta_6_consistencia_mejor_modelo(self):
        """
        ¿Con qué frecuencia el modelo marcado como 'Mejor Modelo' coincide con el 
        que REALMENTE tiene el menor error numérico?
        """
        
        # Verificar si existe la columna "Mejor Modelo"
        if 'Mejor Modelo' not in self.df.columns:
            print("   ⚠️  Columna 'Mejor Modelo' no encontrada. Saltando análisis.\n")
            return
        
        # Identificar el modelo con menor error en cada fila
        self.df['Modelo_Min_Real'] = self.df[self.modelos].idxmin(axis=1)
        self.df['Coincide'] = self.df['Mejor Modelo'] == self.df['Modelo_Min_Real']
        
        # FIGURA 6.1: Tasa de coincidencia global
        fig, ax = plt.subplots(figsize=(10, 6))
        
        tasa_coincidencia = self.df['Coincide'].mean() * 100
        
        labels = ['Coincide', 'No Coincide']
        sizes = [tasa_coincidencia, 100 - tasa_coincidencia]
        colors = ['#4CAF50', '#F44336']
        explode = (0.1, 0)
        
        ax.pie(sizes, explode=explode, labels=labels, colors=colors,
               autopct='%1.1f%%', shadow=True, startangle=90,
               textprops={'fontsize': 14, 'fontweight': 'bold'})
        ax.set_title('Consistencia de "Mejor Modelo" con Menor Error Real\n',
                    fontweight='bold', fontsize=14, pad=20)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P6_1_consistencia_mejor_modelo_global.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 6.2: Coincidencia por escenario y distribución
        fig, axes = plt.subplots(1, 2, figsize=(16, 7))
        
        # Por Escenario
        ax1 = axes[0]
        coincidencia_esc = self.df.groupby('Escenario')['Coincide'].mean() * 100
        coincidencia_esc = coincidencia_esc.sort_values(ascending=True)
        
        colors_esc = ['green' if x > 70 else 'orange' if x > 50 else 'red' 
                      for x in coincidencia_esc.values]
        bars1 = ax1.barh(range(len(coincidencia_esc)), coincidencia_esc.values,
                        color=colors_esc, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax1.set_yticks(range(len(coincidencia_esc)))
        ax1.set_yticklabels([x[:30] + '...' if len(x) > 30 else x for x in coincidencia_esc.index], 
                           fontsize=8)
        ax1.set_xlabel('% de Coincidencia', fontweight='bold', fontsize=11)
        ax1.set_title('Consistencia por Escenario', fontweight='bold', fontsize=12)
        ax1.axvline(70, color='green', linestyle='--', linewidth=1.5, alpha=0.5, label='Umbral Bueno (70%)')
        ax1.axvline(50, color='orange', linestyle='--', linewidth=1.5, alpha=0.5, label='Umbral Aceptable (50%)')
        ax1.legend(fontsize=9)
        ax1.grid(True, alpha=0.3, axis='x')
        
        # Por Distribución
        ax2 = axes[1]
        coincidencia_dist = self.df.groupby('Distribución')['Coincide'].mean() * 100
        coincidencia_dist = coincidencia_dist.sort_values(ascending=False)
        
        colors_dist = ['green' if x > 70 else 'orange' if x > 50 else 'red' 
                       for x in coincidencia_dist.values]
        bars2 = ax2.bar(range(len(coincidencia_dist)), coincidencia_dist.values,
                       color=colors_dist, alpha=0.7, edgecolor='black', linewidth=2)
        ax2.set_xticks(range(len(coincidencia_dist)))
        ax2.set_xticklabels(coincidencia_dist.index, rotation=45, ha='right')
        ax2.set_ylabel('% de Coincidencia', fontweight='bold', fontsize=11)
        ax2.set_title('Consistencia por Distribución', fontweight='bold', fontsize=12)
        ax2.axhline(70, color='green', linestyle='--', linewidth=1.5, alpha=0.5)
        ax2.axhline(50, color='orange', linestyle='--', linewidth=1.5, alpha=0.5)
        ax2.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars2, coincidencia_dist.values):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)
        
        plt.suptitle('Análisis de Consistencia de "Mejor Modelo" por Condiciones',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P6_2_consistencia_por_condiciones.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"   ✓ Tasa de coincidencia global: {tasa_coincidencia:.1f}%")
        print(f"   ✓ Distribución con mayor consistencia: {coincidencia_dist.idxmax()} ({coincidencia_dist.max():.1f}%)")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 7: ANÁLISIS DE SEGUNDA DERIVADA
    # ========================================================================

    def _pregunta_7_segunda_derivada(self):
        """
        ¿Qué modelos muestran 'deterioro acelerado' (segunda derivada positiva) 
        vs 'deterioro constante' al aumentar Pasos?
        """
        
        pasos = sorted(self.df['Paso'].unique())
        
        if len(pasos) < 3:
            print("   ⚠️  Insuficientes pasos para análisis de segunda derivada. Saltando.\n")
            return
        
        # FIGURA 7.1: Aceleración del deterioro
        fig, ax = plt.subplots(figsize=(14, 8))
        
        aceleraciones = []
        
        for modelo in self.modelos:
            valores = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            
            # Primera derivada (velocidad de cambio)
            primera_deriv = np.diff(valores)
            
            # Segunda derivada (aceleración)
            if len(primera_deriv) > 1:
                segunda_deriv = np.diff(primera_deriv)
                aceleracion_media = np.mean(segunda_deriv)
                
                # Clasificación
                if aceleracion_media > 0.001:
                    tipo = 'Acelerado'
                elif aceleracion_media < -0.001:
                    tipo = 'Desacelerado'
                else:
                    tipo = 'Constante'
                
                aceleraciones.append({
                    'Modelo': modelo,
                    'Aceleracion': aceleracion_media,
                    'Tipo': tipo
                })
        
        df_aceleraciones = pd.DataFrame(aceleraciones).sort_values('Aceleracion', ascending=False)
        
        # Colores según tipo
        color_map = {'Acelerado': 'red', 'Constante': 'orange', 'Desacelerado': 'green'}
        colors = [color_map[t] for t in df_aceleraciones['Tipo']]
        
        bars = ax.barh(df_aceleraciones['Modelo'], df_aceleraciones['Aceleracion'],
                      color=colors, alpha=0.7, edgecolor='black', linewidth=2)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.set_xlabel('Aceleración del Deterioro (Segunda Derivada)', fontweight='bold', fontsize=12)
        ax.set_title('Clasificación de Modelos por Aceleración del Deterioro\n(Rojo=Acelerado, Verde=Desacelerado, Naranja=Constante)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val, tipo) in enumerate(zip(bars, df_aceleraciones['Aceleracion'], 
                                                  df_aceleraciones['Tipo'])):
            ax.text(val + (0.0001 if val > 0 else -0.0001), i, f'{val:.4f} ({tipo})',
                   va='center', ha='left' if val > 0 else 'right',
                   fontweight='bold', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P7_1_segunda_derivada_aceleracion.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 7.2: Comparación de primeras y segundas derivadas
        fig, axes = plt.subplots(2, 1, figsize=(14, 12))
        
        # >>> INICIO DE MODIFICACIÓN 4 <<<
        # Mapear cada modelo a su tipo de aceleración para el estilo de línea
        modelo_a_tipo = df_aceleraciones.set_index('Modelo')['Tipo'].to_dict()
        tipo_a_estilo = {'Acelerado': '-', 'Desacelerado': '--', 'Constante': ':'}

        # Primera derivada
        ax1 = axes[0]
        for modelo in self.modelos:
            valores = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            primera_deriv = np.diff(valores)
            pasos_deriv = pasos[1:]
            
            tipo_modelo = modelo_a_tipo.get(modelo, 'Constante')
            linestyle = tipo_a_estilo[tipo_modelo]
            
            ax1.plot(pasos_deriv, primera_deriv, 'o-', label=modelo,
                    linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo],
                    linestyle=linestyle)
        
        ax1.axhline(0, color='black', linestyle=':', linewidth=1.5, alpha=0.5)
        ax1.set_xlabel('Paso', fontweight='bold', fontsize=11)
        ax1.set_ylabel('Primera Derivada (Velocidad)', fontweight='bold', fontsize=11)
        ax1.set_title('Velocidad de Deterioro por Paso\n(Acelerado:—, Desacelerado:--, Constante:..)',
                     fontweight='bold', fontsize=12)
        ax1.legend(fontsize=9, ncol=3) # Aumentar columnas para que quepan todos
        ax1.grid(True, alpha=0.3)
        
        # Segunda derivada
        ax2 = axes[1]
        for modelo in self.modelos:
            valores = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            primera_deriv = np.diff(valores)
            
            if len(primera_deriv) > 1:
                segunda_deriv = np.diff(primera_deriv)
                pasos_deriv2 = pasos[2:]
                
                tipo_modelo = modelo_a_tipo.get(modelo, 'Constante')
                linestyle = tipo_a_estilo[tipo_modelo]
                
                ax2.plot(pasos_deriv2, segunda_deriv, 's-', label=modelo,
                        linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo],
                        linestyle=linestyle)
        
        ax2.axhline(0, color='black', linestyle=':', linewidth=1.5, alpha=0.5)
        ax2.set_xlabel('Paso', fontweight='bold', fontsize=11)
        ax2.set_ylabel('Segunda Derivada (Aceleración)', fontweight='bold', fontsize=11)
        ax2.set_title('Aceleración del Deterioro por Paso\n(Valores positivos = Deterioro acelerado)',
                     fontweight='bold', fontsize=12)
        ax2.legend(fontsize=9, ncol=3) # Aumentar columnas
        ax2.grid(True, alpha=0.3)
        # >>> FIN DE MODIFICACIÓN 4 <<<
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P7_2_comparacion_derivadas.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        modelos_vulnerables = df_aceleraciones[df_aceleraciones['Tipo'] == 'Acelerado']['Modelo'].tolist()
        modelos_robustos = df_aceleraciones[df_aceleraciones['Tipo'] == 'Desacelerado']['Modelo'].tolist()
        print(f"   ✓ Modelos con deterioro acelerado: {len(modelos_vulnerables)}")
        print(f"   ✓ Modelos con deterioro desacelerado: {len(modelos_robustos)}")
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 8: INTERACCIÓN NO LINEALIDAD × VARIANZA
    # ========================================================================

    def _pregunta_8_interaccion_nolineal_varianza(self):
        """
        ¿La ventaja de LSPM/LSPMW en escenarios no lineales DESAPARECE cuando 
        la varianza del error supera cierto umbral?
        """
        
        # Filtrar escenarios no lineales
        df_nolineal = self.df[self.df['Lineal'] == 'No Lineal'].copy()
        
        if df_nolineal.empty:
            print("   ⚠️  No hay datos para escenarios no lineales. Saltando análisis.\n")
            return
            
        varianzas = sorted(df_nolineal['Varianza error'].unique())
        
        # >>> INICIO DE MODIFICACIÓN 5 <<<
        # Se redefine el cálculo para hacer una comparación más clara entre los extremos
        # Se usa el primer cuartil (Q1) para "Varianza Baja" y el tercer cuartil (Q3) para "Varianza Alta"
        if len(varianzas) > 3:
            q1, q3 = np.percentile(varianzas, [25, 75])
            df_baja = df_nolineal[df_nolineal['Varianza error'] <= q1]
            df_alta = df_nolineal[df_nolineal['Varianza error'] >= q3]
        else: # Fallback para pocos datos
            q2 = np.percentile(varianzas, 50)
            df_baja = df_nolineal[df_nolineal['Varianza error'] <= q2]
            df_alta = df_nolineal[df_nolineal['Varianza error'] >= q2]

        grupos_dfs = {'Varianza Baja': df_baja, 'Varianza Alta': df_alta}
        # >>> FIN DE MODIFICACIÓN 5 <<<

        # FIGURA 8.1: Ranking por grupo de varianza
        fig, axes = plt.subplots(1, 2, figsize=(16, 7))
        
        for idx, (grupo, df_grupo) in enumerate(grupos_dfs.items()):
            ax = axes[idx]
            
            if df_grupo.empty:
                ax.text(0.5, 0.5, 'Sin datos', ha='center', va='center', fontsize=12)
                ax.set_title(f'Ranking en Escenarios No Lineales\n{grupo}', fontweight='bold', fontsize=12)
                continue

            # Calcular medias y ranking
            medias = df_grupo[self.modelos].mean().sort_values()
            
            colors = ['gold' if m in ['LSPM', 'LSPMW'] else 'steelblue' for m in medias.index]
            bars = ax.barh(range(len(medias)), medias.values, color=colors,
                          alpha=0.7, edgecolor='black', linewidth=1.5)
            
            ax.set_yticks(range(len(medias)))
            ax.set_yticklabels(medias.index, fontsize=10)
            ax.set_xlabel('ECRPS Promedio', fontweight='bold', fontsize=11)
            ax.set_title(f'Ranking en Escenarios No Lineales\n{grupo}',
                        fontweight='bold', fontsize=12)
            ax.grid(True, alpha=0.3, axis='x')
            
            # Destacar posición de LSPM/LSPMW
            pos_lspm = list(medias.index).index('LSPM') + 1
            pos_lspmw = list(medias.index).index('LSPMW') + 1
            
            ax.text(0.02, 0.98, f'LSPM: Rank {pos_lspm}\nLSPMW: Rank {pos_lspmw}',
                   transform=ax.transAxes, fontsize=11, fontweight='bold',
                   va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
        
        plt.suptitle('¿LSPM/LSPMW Pierden Ventaja con Alta Varianza?',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P8_1_interaccion_nolineal_varianza_ranking.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 8.2: Cambio de posición en ranking
        fig, ax = plt.subplots(figsize=(14, 8))
        
        cambios_ranking = []
        
        # >>> INICIO DE MODIFICACIÓN 5 (cont.) <<<
        # Se usan los dataframes df_baja y df_alta definidos previamente
        for modelo in self.modelos:
            if df_baja.empty or df_alta.empty: continue
            
            ranking_baja = df_baja[self.modelos].mean().rank().loc[modelo]
            ranking_alta = df_alta[self.modelos].mean().rank().loc[modelo]
            
            cambio = ranking_alta - ranking_baja  # Positivo = empeoró posición
            
            cambios_ranking.append({
                'Modelo': modelo,
                'Cambio': cambio,
                'Empeora': cambio > 0
            })
        # >>> FIN DE MODIFICACIÓN 5 (cont.) <<<

        if not cambios_ranking:
            print("   ⚠️  No se pudo generar el gráfico de cambio de ranking.\n")
            return

        df_cambios = pd.DataFrame(cambios_ranking).sort_values('Cambio', ascending=True)
        
        colors_cambio = ['red' if x > 2 else 'orange' if x > 0 else 'green' 
                        for x in df_cambios['Cambio']]
        bars = ax.barh(df_cambios['Modelo'], df_cambios['Cambio'],
                      color=colors_cambio, alpha=0.7, edgecolor='black', linewidth=2)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.axvline(3, color='red', linestyle='--', linewidth=1.5, alpha=0.5, 
                  label='Cambio Crítico (>3 posiciones)')
        ax.set_xlabel('Cambio en Ranking (Varianza Alta - Varianza Baja)', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro de Posición con Alta Varianza en Escenarios No Lineales\n(Positivo = Pierde posiciones, Negativo = Gana posiciones)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val, modelo) in enumerate(zip(bars, df_cambios['Cambio'], df_cambios['Modelo'])):
            label_text = f'{val:+.1f}'
            if modelo in ['LSPM', 'LSPMW']:
                label_text += ' ⭐'
            ax.text(val + (0.1 if val > 0 else -0.1), i, label_text,
                   va='center', ha='left' if val > 0 else 'right',
                   fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P8_2_cambio_ranking_lspm.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis específico de LSPM/LSPMW
        cambio_lspm = df_cambios[df_cambios['Modelo'] == 'LSPM']['Cambio'].values[0]
        cambio_lspmw = df_cambios[df_cambios['Modelo'] == 'LSPMW']['Cambio'].values[0]
        
        print(f"   ✓ Cambio de ranking LSPM: {cambio_lspm:+.1f} posiciones")
        print(f"   ✓ Cambio de ranking LSPMW: {cambio_lspmw:+.1f} posiciones")
        
        if cambio_lspm > 3 or cambio_lspmw > 3:
            print("   ⚠️  ADVERTENCIA: LSPM/LSPMW pierden >3 posiciones con alta varianza")
        else:
            print("   ✓ LSPM/LSPMW mantienen ventaja incluso con alta varianza")
        
        print("   ✓ 2 figuras generadas\n")

    # ========================================================================
    # PREGUNTA 9: MAPA DE DECISIÓN OPERACIONAL
    # ========================================================================

    def _pregunta_9_mapa_decision(self):
        """
        ¿Podemos construir una 'regla de decisión' simple para elegir modelo 
        basada en 3 variables: Estacionariedad, Distribución y Varianza?
        """
        
        # Crear categorías de varianza
        varianzas = sorted(self.df['Varianza error'].unique())
        q33, q67 = np.percentile(varianzas, [33, 67])
        
        self.df['Nivel_Varianza'] = pd.cut(
            self.df['Varianza error'],
            bins=[-np.inf, q33, q67, np.inf],
            labels=['Baja', 'Media', 'Alta']
        )
        
        # FIGURA 9.1: Árbol de decisión visual (heatmap 3D colapsado)
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        niveles_var = ['Baja', 'Media', 'Alta']
        estacionariedades = ['Estacionario', 'No Estacionario']
        
        for idx_est, est in enumerate(estacionariedades):
            for idx_var,niv_var in enumerate(niveles_var):
                ax = axes[idx_est, idx_var]
                
                # Filtrar datos
                df_filtrado = self.df[
                    (self.df['Estacionario'] == est) & 
                    (self.df['Nivel_Varianza'] == niv_var)
                ]
                
                if len(df_filtrado) == 0:
                    ax.text(0.5, 0.5, 'Sin datos', ha='center', va='center',
                           transform=ax.transAxes, fontsize=12)
                    ax.set_title(f'{est} + Varianza {niv_var}', fontweight='bold', fontsize=11)
                    continue
                
                # Calcular mejor modelo por distribución
                mejores_por_dist = {}
                for dist in df_filtrado['Distribución'].unique():
                    df_dist = df_filtrado[df_filtrado['Distribución'] == dist]
                    medias = df_dist[self.modelos].mean()
                    mejor = medias.idxmin()
                    mejores_por_dist[dist] = mejor
                
                # Crear matriz para heatmap
                distribuciones = sorted(df_filtrado['Distribución'].unique())
                matriz_decision = pd.DataFrame(index=distribuciones, columns=['Mejor Modelo'])
                
                for dist in distribuciones:
                    matriz_decision.loc[dist, 'Mejor Modelo'] = mejores_por_dist.get(dist, 'N/A')
                
                # Asignar colores por modelo
                modelo_a_color = {modelo: idx for idx, modelo in enumerate(self.modelos)}
                matriz_numerica = matriz_decision['Mejor Modelo'].map(
                    lambda x: modelo_a_color.get(x, -1)
                ).values.reshape(-1, 1)
                
                im = ax.imshow(matriz_numerica, cmap='tab10', aspect='auto', vmin=0, vmax=len(self.modelos)-1)
                
                # Configurar ejes
                ax.set_yticks(range(len(distribuciones)))
                ax.set_yticklabels(distribuciones, fontsize=9)
                ax.set_xticks([])
                ax.set_title(f'{est}\nVarianza {niv_var}', fontweight='bold', fontsize=11)
                
                # Añadir texto con nombre del modelo
                for i, dist in enumerate(distribuciones):
                    modelo = mejores_por_dist.get(dist, 'N/A')
                    ax.text(0, i, modelo, ha='center', va='center',
                           fontweight='bold', fontsize=9, color='white',
                           bbox=dict(boxstyle='round', facecolor='black', alpha=0.6))
        
        plt.suptitle('Mapa de Decisión Operacional: Mejor Modelo por Condiciones\n(Cada celda muestra el modelo óptimo)',
                    fontweight='bold', fontsize=14, y=0.995)
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P9_1_mapa_decision_operacional.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 9.2: Reglas de decisión simplificadas
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Calcular frecuencia de "mejor modelo" en cada combinación
        reglas = []
        
        for est in estacionariedades:
            for niv_var in niveles_var:
                df_comb = self.df[
                    (self.df['Estacionario'] == est) & 
                    (self.df['Nivel_Varianza'] == niv_var)
                ]
                
                if len(df_comb) == 0:
                    continue
                
                # Encontrar modelo con menor error promedio
                medias = df_comb[self.modelos].mean()
                mejor_modelo = medias.idxmin()
                mejor_ecrps = medias.min()
                
                # Calcular % de veces que es el mejor
                cuenta_mejor = 0
                for idx, row in df_comb.iterrows():
                    if row[self.modelos].idxmin() == mejor_modelo:
                        cuenta_mejor += 1
                
                frecuencia = (cuenta_mejor / len(df_comb)) * 100 if len(df_comb) > 0 else 0
                
                reglas.append({
                    'Condicion': f'{est[:3]}+Var_{niv_var}',
                    'Completo': f'{est} + Varianza {niv_var}',
                    'Mejor_Modelo': mejor_modelo,
                    'ECRPS': mejor_ecrps,
                    'Frecuencia': frecuencia
                })
        
        df_reglas = pd.DataFrame(reglas).sort_values('Frecuencia', ascending=True)
        
        # Visualización
        y_pos = np.arange(len(df_reglas))
        colors_reglas = [COLORES_MODELOS.get(m, 'gray') for m in df_reglas['Mejor_Modelo']]
        
        bars = ax.barh(y_pos, df_reglas['Frecuencia'], color=colors_reglas,
                      alpha=0.7, edgecolor='black', linewidth=1.5)
        
        ax.set_yticks(y_pos)
        ax.set_yticklabels(df_reglas['Condicion'], fontsize=10)
        ax.set_xlabel('Frecuencia de Optimalidad (%)', fontweight='bold', fontsize=12)
        ax.set_title('Confiabilidad de Reglas de Decisión\n(% de casos donde el modelo recomendado es óptimo)',
                    fontweight='bold', fontsize=14, pad=20)
        ax.axvline(70, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Umbral Alta Confianza (70%)')
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='x')
        
        # Añadir etiquetas con modelo recomendado
        for i, (bar, modelo, freq) in enumerate(zip(bars, df_reglas['Mejor_Modelo'], 
                                                     df_reglas['Frecuencia'])):
            ax.text(freq + 2, i, f'{modelo} ({freq:.0f}%)',
                   va='center', fontweight='bold', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P9_2_confiabilidad_reglas.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 9.3: Árbol de decisión textual
        fig, ax = plt.subplots(figsize=(14, 10))
        ax.axis('off')
        
        # Generar reglas textuales
        texto_reglas = "ÁRBOL DE DECISIÓN OPERACIONAL\n" + "="*60 + "\n\n"
        
        for idx, row in df_reglas.iterrows():
            confianza = "ALTA" if row['Frecuencia'] > 70 else "MEDIA" if row['Frecuencia'] > 50 else "BAJA"
            texto_reglas += f"📌 SI: {row['Completo']}\n"
            texto_reglas += f"   → USAR: {row['Mejor_Modelo']}\n"
            texto_reglas += f"   → Confianza: {confianza} ({row['Frecuencia']:.1f}%)\n"
            texto_reglas += f"   → ECRPS esperado: {row['ECRPS']:.4f}\n\n"
        
        # Añadir reglas generales
        texto_reglas += "\n" + "="*60 + "\n"
        texto_reglas += "REGLAS GENERALES SIMPLIFICADAS:\n\n"
        
        # Mejor modelo global
        mejor_global = df_reglas.loc[df_reglas['ECRPS'].idxmin(), 'Mejor_Modelo']
        texto_reglas += f"🏆 Modelo más robusto (recomendación por defecto): {mejor_global}\n\n"
        
        # Modelos por características
        mejor_no_est = df_reglas[df_reglas['Completo'].str.contains('No Estacionario')].iloc[0]['Mejor_Modelo'] if len(df_reglas[df_reglas['Completo'].str.contains('No Estacionario')]) > 0 else 'N/A'
        mejor_alta_var = df_reglas[df_reglas['Completo'].str.contains('Alta')].iloc[0]['Mejor_Modelo'] if len(df_reglas[df_reglas['Completo'].str.contains('Alta')]) > 0 else 'N/A'
        
        texto_reglas += f"🔴 Para datos NO estacionarios: {mejor_no_est}\n"
        texto_reglas += f"🔴 Para varianza ALTA: {mejor_alta_var}\n"
        
        ax.text(0.05, 0.95, texto_reglas, transform=ax.transAxes,
               fontsize=10, verticalalignment='top', fontfamily='monospace',
               bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P9_3_arbol_decision_textual.png',
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # Guardar reglas en CSV
        df_reglas.to_csv(self.dir_salida / 'reglas_decision.csv', index=False)
        
        print(f"   ✓ Modelo más robusto: {mejor_global}")
        print(f"   ✓ Reglas de alta confianza (>70%): {len(df_reglas[df_reglas['Frecuencia'] > 70])}")
        print(f"   ✓ Archivo CSV generado: reglas_decision.csv")
        print("   ✓ 3 figuras generadas\n")


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def main():
    """Función principal de ejecución"""
    print("\n" + "█" * 80)
    print("█" + " " * 78 + "█")
    print("█" + " " * 15 + "ANÁLISIS DE PREGUNTAS DE PROFUNDIZACIÓN" + " " * 23 + "█")
    print("█" + " " * 78 + "█")
    print("█" * 80 + "\n")

    try:
        # Crear instancia del analizador
        analizador = AnalizadorPreguntasProfundizacion(RUTA_DATOS)

        # Ejecutar análisis completo
        analizador.ejecutar_analisis_completo()

        print("\n" + "█" * 80)
        print("█" + " " * 78 + "█")
        print("█" + " " * 20 + "✅ ANÁLISIS COMPLETADO EXITOSAMENTE" + " " * 23 + "█")
        print("█" + " " * 78 + "█")
        print("█" * 80 + "\n")

        print("📊 RESUMEN DE FIGURAS GENERADAS POR PREGUNTA:\n")
        print("   Pregunta 1 (Punto de quiebre AREPD): 2 figuras")
        print("   Pregunta 2 (Zona de dominio BB): 2 figuras")
        print("   Pregunta 3 (Deterioro AV-MCPS): 2 figuras")
        print("   Pregunta 4 (Penalización Normal): 2 figuras")
        print("   Pregunta 5 (Frontera DL): 2 figuras")
        print("   Pregunta 6 (Consistencia Mejor Modelo): 2 figuras")
        print("   Pregunta 7 (Segunda derivada): 2 figuras")
        print("   Pregunta 8 (Interacción No-Lineal × Varianza): 2 figuras")
        print("   Pregunta 9 (Mapa de decisión): 3 figuras")
        print("\n   📁 TOTAL: 19 figuras PNG + 1 archivo CSV (reglas_decision.csv)")
        
        print("\n📁 ESTRUCTURA DE RESULTADOS:")
        print(f"   {DIR_SALIDA}/")
        print("   ├── P1_1: Punto de quiebre AREPD - Comparativo")
        print("   ├── P1_2: Tasa de deterioro AREPD")
        print("   ├── P2_1: Zona de dominio BB - Heatmap")
        print("   ├── P2_2: Frecuencia de dominio BB")
        print("   ├── P3_1: Deterioro AV-MCPS - Curvas")
        print("   ├── P3_2: Tipo de deterioro AV-MCPS")
        print("   ├── P4_1: Penalización Normal - Interacción")
        print("   ├── P4_2: Clasificación de interacción")
        print("   ├── P5_1: Frontera de colapso DL")
        print("   ├── P5_2: Brecha DL por escenario")
        print("   ├── P6_1: Consistencia Mejor Modelo - Global")
        print("   ├── P6_2: Consistencia por condiciones")
        print("   ├── P7_1: Segunda derivada - Aceleración")
        print("   ├── P7_2: Comparación de derivadas")
        print("   ├── P8_1: Interacción No-Lineal × Varianza - Ranking")
        print("   ├── P8_2: Cambio de ranking LSPM")
        print("   ├── P9_1: Mapa de decisión operacional")
        print("   ├── P9_2: Confiabilidad de reglas")
        print("   ├── P9_3: Árbol de decisión textual")
        print("   └── reglas_decision.csv")
        print("\n" + "=" * 80 + "\n")

    except FileNotFoundError:
        print(f"\n❌ ERROR: No se encontró el archivo {RUTA_DATOS}")
        print("   Por favor, verifica que el archivo existe y la ruta es correcta.\n")
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {str(e)}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


████████████████████████████████████████████████████████████████████████████████
█                                                                              █
█               ANÁLISIS DE PREGUNTAS DE PROFUNDIZACIÓN                       █
█                                                                              █
████████████████████████████████████████████████████████████████████████████████


ANÁLISIS DE PREGUNTAS DE PROFUNDIZACIÓN

✓ Datos cargados: 2000 filas, 17 columnas
✓ Modelos a analizar: 9
✓ Directorio de salida: resultados_preguntas_profundizacion



🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬

1️⃣  Pregunta 1: Punto de quiebre de AREPD...
--- Análisis de Punto de Quiebre para AREPD ---
   ✓ Punto de quiebre AREPD (Normal): Varianza ≈ 0.350
   ✓ Punto de quiebre AREPD (Uniform): Varianza ≈ 0.750
   ✓ Punto de quiebre AREPD (Exponential): Varianza ≈ 0.350
   ✓ Punto de quiebre AREPD (T-student): Varianza ≈ 0.750
   ✓ Punto de quiebre AREPD (Mixture): Varianza ≈ 0.350
  